In [11]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [12]:
from sympy import *
from IPython.display import display
import time
import pickle
import datetime
%env USE_SYMENGINE 1

epsilon, epsiloninv, u, v, a, theta = symbols('\\epsilon, \\eta, u, v, a, \\theta', real=True)


q = symbols('q', real=False)
q_conj = symbols('\\tilde{q}', real=False)


W_plus = Symbol('W_+', real=True)
W_minus = Symbol('W_-', real=True)
w_plus = Symbol('w_+', real=True)
w_minus = Symbol('w_-', real=True)
V_sym = Symbol('V')
N_sym = Symbol('N')
A_sym = Symbol('A')
phi_sym = Symbol('\\phi')
n_sym = Symbol('n')
a_sym = Symbol('a')

differentiable_symbols = [u, v, a, theta, q, q_conj, w_plus, w_minus, W_plus, W_minus, V_sym, N_sym, A_sym, phi_sym, n_sym, a_sym]


env: USE_SYMENGINE=1


In [13]:

def var_deriv_name(var):
    if "_{x" in var.name:
        return var.name[:-1] + 'x}'
    else:
        return '(' + var.name + ')_{x}'
        #return var.name + '_x'
        
def get_var_name_from_deriv(sym):
    start = sym.name.find('(')
    end = sym.name.find(')')
    if start != -1 and end != -1:
        sym_name = sym.name[start+1:end]
        return sym_name
        #orders.append(len(sym.name) - 4 - sym.name.find(')_{'))
    else:
        sym_name = sym.name
        return sym_name
        #orders.append(0)

def get_order_from_deriv(sym):
    start = sym.name.find('(')
    end = sym.name.find(')')
    if start != -1 and end != -1:
        sym_name = sym.name[start+1:end]
        return len(sym.name) - 4 - sym.name.find(')_{')
    else:
        return 0
        
def deriv(poly, append_to_list=True):
    res = 0
    original = differentiable_symbols.copy()
    for sym in original:
        deriv_term = Derivative(poly, sym).doit()
        if deriv_term != 0:
            #print("looking at: ", sym)
            newName = var_deriv_name(sym)

            if newName in [s.name for s in differentiable_symbols]:
                dsym = differentiable_symbols[[s.name for s in differentiable_symbols].index(newName)]
                #print("newName found: ", dsym)
            else:
                dsym = Symbol(newName, real=True)
                if append_to_list:
                    print("new differentiable symbol: ", dsym)
                    differentiable_symbols.append(dsym)
                #print("newName not found: ", dsym)
            #print("syms: ", syms)
            res += deriv_term * dsym
            #print("res: ", res)
    
    return res
    

def higher_deriv(var, n):
    if n > 0:
        return deriv(higher_deriv(var, n-1))
    else:
        return var
    
def variation(expr, sym):
    if not hasattr(expr, 'free_symbols'):
        return 0
    
    res = 0
    order = 0

    syms = []
    orders = []

    start = sym.name.find('(')
    end = sym.name.find(')')
    if start != -1 and end != -1:
        sym_name = sym.name[start+1:end]
        syms.append(sym)
        orders.append(len(sym.name) - 4 - sym.name.find(')_{'))
    else:
        sym_name = sym.name
        syms.append(sym)
        orders.append(0)

    for s in expr.free_symbols:
        start = s.name.find('(')
        end = s.name.find(')')
        if start != -1 and end != -1 and sym_name == s.name[start+1:end]:
            if s.name not in [sym.name for sym in syms]:
                syms.append(s)
                orders.append(len(s.name) - 4 - s.name.find(')_{'))    
    
    for (sym, order) in zip(syms, orders):
        res += (-1)**order * higher_deriv(Derivative(expr, sym).doit(), order)

    return simplify(res)

def polynomize(expr):
    return simplify(Poly(expr, epsilon, epsiloninv).subs(epsiloninv, 1/epsilon))

def depolynomize(poly):
    monoms = poly.monoms()
    coeffs = poly.coeffs()
    
    X = 0
    for (k, m) in enumerate(monoms):
        X += epsilon**m[0] * epsiloninv**m[1] * coeffs[k]
            
    return X

def poly_simplify(expr):
    return depolynomize(polynomize(expr))

def substituter(expr, var, sub, magnitude=1, scale=1):
    if not hasattr(expr, 'subs'):
        return expr
    for subvar in sub.free_symbols:
        if var in differentiable_symbols and subvar not in differentiable_symbols:
            print("Warning: Substituting differentiable by non-differentiable symbol.")
            #differentiable_symbols.append(subvar)
            pass
        
    original = differentiable_symbols.copy()
    expr = expr.subs(var, magnitude * sub)
    
    cont = True
    m = 0
    while cont:
        #print("var", var, "sub", sub)
        dvar = deriv(var, append_to_list=False)
        m += 1
        
        if dvar in differentiable_symbols:
            dsub = simplify(deriv(sub))
            #print("new sub variable from", sub, "to", dsub)
            expr = simplify(expr.subs(dvar, magnitude * scale**m * dsub))
                    
            var = dvar
            sub = dsub
                    
            #print("substited", var, "by", sub)
        else: 
            cont = False
            
    if hasattr(expr, 'applyfunc'):
        return expr.applyfunc(poly_simplify)
    else:
        return poly_simplify(expr)
    
def multi_substituter(expr, data, magnitude=1, scale=1):
    for (var, sub) in data:
        expr = substituter(expr, var, sub, magnitude=magnitude, scale=scale)
    return expr

def extract_deriv(expr, k): 
    expr = simplify(expr).expand()
    #print("expr =", expr) 
    res = 0
    if isinstance(expr, Add):
        for mon in expr.args:
            #print("mon =", mon)
            count = 0
            if hasattr(mon, "name"):
                #print(mon.name)
                if "_{x" in mon.name:
                    count += 1
                    #print("count =", count)
            else:
                for fac in mon.args:
                    #print("fac =", fac)
                    if hasattr(fac, "name"):
                        #print(fac.name)
                        if "_{x" in fac.name:
                            count += 1
                            #print("count =", count)
                    elif isinstance(fac, Pow):
                        #print(fac.base.name, fac.exp)
                        if "_{x" in fac.base.name and fac.exp > 0:
                            count += fac.exp     
                            #print("count =", count)
            #print("final count =", count)
            if count == k:
                res += mon

    else:
        mon = expr
        #print("mon =", mon)
        count = 0
        if hasattr(mon, "name"):
            #print(mon.name)
            if "_{x" in mon.name:
                count += 1
                #print("count =", count)
        elif hasattr(mon, "args"):
            for fac in mon.args:
                #print("fac =", fac)
                if hasattr(fac, "name"):
                    #print(fac.name)
                    if "_{x" in fac.name:
                        count += 1
                        #print("count =", count)
                elif isinstance(fac, Pow):
                    #print(fac.base.name, fac.exp)
                    if "_{x" in fac.base.name and fac.exp > 0:
                        count += fac.exp     
                        #print("count =", count)
        #print("final count =", count)
        if count == k:
            res += mon

    #print("res =", res)
    return res
        
def extract_deriv_alt(expr, k):
    expr = simplify(expr).expand()
    #print("expr =", expr) 
    res = 0
    if isinstance(expr, Add):
        for mon in expr.args:
            presence_deriv_q_conj = False
            #print("mon =", mon)
            count = 0
            if hasattr(mon, "name"):
                #print(mon.name)
                if "_{x" in mon.name:
                    count += 1
                    if get_var_name_from_deriv(mon) == q_conj.name:
                        presence_deriv_q_conj = True
            else:
                for fac in mon.args:
                    #print("fac =", fac)
                    if hasattr(fac, "name"):
                        #print(fac.name)
                        if "_{x" in fac.name:
                            count += 1
                            if get_var_name_from_deriv(fac) == q_conj.name:
                                presence_deriv_q_conj = True
                            #print("count =", count)
                    elif isinstance(fac, Pow):
                        #print(fac.base.name, fac.exp)
                        if "_{x" in fac.base.name and fac.exp > 0:
                            count += fac.exp     
                            if get_var_name_from_deriv(fac.base) == q_conj.name:
                                presence_deriv_q_conj = True
                            #print("count =", count)
            #print("final count =", count)
            if count == k and presence_deriv_q_conj:
                res += mon

    else:
        presence_deriv_q_conj = False
        mon = expr
        #print("mon =", mon)
        count = 0
        if hasattr(mon, "name"):
            #print(mon.name)
            if "_{x" in mon.name:
                count += 1
                if get_var_name_from_deriv(mon) == q_conj.name:
                    presence_deriv_q_conj = True
                #print("count =", count)
        elif hasattr(mon, "args"):
            for fac in mon.args:
                #print("fac =", fac)
                if hasattr(fac, "name"):
                    #print(fac.name)
                    if "_{x" in fac.name:
                        count += 1
                        if get_var_name_from_deriv(fac) == q_conj.name:
                            presence_deriv_q_conj = True
                        #print("count =", count)
                elif isinstance(fac, Pow):
                    #print(fac.base.name, fac.exp)
                    if "_{x" in fac.base.name and fac.exp > 0:
                        count += fac.exp   
                        if get_var_name_from_deriv(fac.base) == q_conj.name:
                            presence_deriv_q_conj = True
                        #print("count =", count)
        #print("final count =", count)
        if count == k and presence_deriv_q_conj:
            res += mon

    #print("res =", res)
    return res

N = 7

In [19]:
X = q * deriv(sin(q)**2) * deriv(exp(u))
display(multi_substituter(X, [(u, v), (q, deriv(a))]))


(a)_{xx}*(a)_{x}*(v)_{x}*exp(v)*sin(2*(a)_{x})

In [8]:
def get_expansion(eqn, x0, var, num):
    ser = series(eqn, x0=0, x=var, n=num).removeO()
    return [simplify(ser.coeff(var**n) if n > 0 else ser.subs(var, 0)) for n in range(0, num)]

delta = Symbol('\\delta')



In [ ]:
if w_plus not in differentiable_symbols: 
    differentiable_symbols.append(w_plus) 
if w_minus not in differentiable_symbols:
    differentiable_symbols.append(w_minus) 

print(differentiable_symbols)

a_subs_q = sqrt(q * q_conj)
v_subs_q = (deriv(q) / q - deriv(q_conj) / q_conj) / (2 * I)


expr = deriv(deriv(w_plus - w_minus))
display(expr)
expr2 = multi_substituter(expr, [(w_plus, v_subs_q/2 + a_subs_q-1), (w_minus, v_subs_q/2 - a_subs_q+1)])
display(simplify(expr2.subs(q_conj, abs(q)**2 / q)))


#print(differentiable_symbols)

In [16]:
N = 10
for n in range(N):
    print("n =", n)
    display(Eq(sig_sym[n], polynomize(poly_simplify(sig_eq[n]))))
    display(Eq(sig_sym[n], polynomize(poly_simplify(multi_substituter(sig_eq[n], [(u, epsilon**2 * u), (v, epsilon**2 * v)])))))   

n = 0


Eq(\sigma_{0}, Poly((-sqrt(2)*I*u/4 - sqrt(2)*I*v/4)*\epsilon, \epsilon, 1/\epsilon, domain='EX'))

Eq(\sigma_{0}, Poly((-sqrt(2)*I*u/4 - sqrt(2)*I*v/4)*\epsilon**3, \epsilon, 1/\epsilon, domain='EX'))

n = 1


Eq(\sigma_{1}, Poly((-u**2/8 + u*v/4 - v**2/8)*\epsilon**2 + (-sqrt(2)*I*(u)_{x}/4 + sqrt(2)*I*(v)_{x}/4)*\epsilon + u/2 - v/2, \epsilon, 1/\epsilon, domain='EX'))

...

Eq(\sigma_{7}, Poly((429*u**8/4096 - 33*u**7*v/512 - 21*u**6*v**2/1024 - 7*u**5*v**3/512 - 25*u**4*v**4/2048 - 7*u**3*v**5/512 - 21*u**2*v**6/1024 - 33*u*v**7/512 + 429*v**8/4096)*\epsilon**8 + (2*sqrt(2)*I*(u)_{x}*u**6 - 2*sqrt(2)*I*(v)_{x}*v**6)*\epsilon**7 + (-693*(u)_{xx}*u**5/128 - 315*(u)_{xx}*u**4*v/128 - 105*(u)_{xx}*u**3*v**2/64 - 75*(u)_{xx}*u**2*v**3/64 - 105*(u)_{xx}*u*v**4/128 - 63*(u)_{xx}*v**5/128 - 5775*(u)_{x}**2*u**4/256 - 525*(u)_{x}**2*u**3*v/64 - 525*(u)_{x}**2*u**2*v**2/128 - 125*(u)_{x}**2*u*v**3/64 - 175*(u)_{x}**2*v**4/256 - 315*(u)_{x}*(v)_{x}*u**4/128 - 105*(u)_{x}*(v)_{x}*u**3*v/32 - 225*(u)_{x}*(v)_{x}*u**2*v**2/64 - 105*(u)_{x}*(v)_{x}*u*v**3/32 - 315*(u)_{x}*(v)_{x}*v**4/128 + 63*(v)_{xx}*u**5/128 + 105*(v)_{xx}*u**4*v/128 + 75*(v)_{xx}*u**3*v**2/64 + 105*(v)_{xx}*u**2*v**3/64 + 315*(v)_{xx}*u*v**4/128 + 693*(v)_{xx}*v**5/128 + 245*(v)_{x}**2*u**4/256 + 175*(v)_{x}**2*u**3*v/64 + 735*(v)_{x}**2*u**2*v**2/128 + 735*(v)_{x}**2*u*v**3/64 + 8085*(v)_{x}**2*v*

Eq(\sigma_{7}, Poly((429*u**8/4096 - 33*u**7*v/512 - 21*u**6*v**2/1024 - 7*u**5*v**3/512 - 25*u**4*v**4/2048 - 7*u**3*v**5/512 - 21*u**2*v**6/1024 - 33*u*v**7/512 + 429*v**8/4096)*\epsilon**24 + (2*sqrt(2)*I*(u)_{x}*u**6 - 2*sqrt(2)*I*(v)_{x}*v**6)*\epsilon**21 + (-231*u**7/256 + 105*u**6*v/256 + 21*u**5*v**2/256 + 5*u**4*v**3/256 - 5*u**3*v**4/256 - 21*u**2*v**5/256 - 105*u*v**6/256 + 231*v**7/256)*\epsilon**20 + (-693*(u)_{xx}*u**5/128 - 315*(u)_{xx}*u**4*v/128 - 105*(u)_{xx}*u**3*v**2/64 - 75*(u)_{xx}*u**2*v**3/64 - 105*(u)_{xx}*u*v**4/128 - 63*(u)_{xx}*v**5/128 - 5775*(u)_{x}**2*u**4/256 - 525*(u)_{x}**2*u**3*v/64 - 525*(u)_{x}**2*u**2*v**2/128 - 125*(u)_{x}**2*u*v**3/64 - 175*(u)_{x}**2*v**4/256 - 315*(u)_{x}*(v)_{x}*u**4/128 - 105*(u)_{x}*(v)_{x}*u**3*v/32 - 225*(u)_{x}*(v)_{x}*u**2*v**2/64 - 105*(u)_{x}*(v)_{x}*u*v**3/32 - 315*(u)_{x}*(v)_{x}*v**4/128 + 63*(v)_{xx}*u**5/128 + 105*(v)_{xx}*u**4*v/128 + 75*(v)_{xx}*u**3*v**2/64 + 105*(v)_{xx}*u**2*v**3/64 + 315*(v)_{xx}*u*v**4/128

n = 8


Eq(\sigma_{8}, Poly((-715*sqrt(2)*I*u**9/8192 + 429*sqrt(2)*I*u**8*v/8192 + 33*sqrt(2)*I*u**7*v**2/2048 + 21*sqrt(2)*I*u**6*v**3/2048 + 35*sqrt(2)*I*u**5*v**4/4096 + 35*sqrt(2)*I*u**4*v**5/4096 + 21*sqrt(2)*I*u**3*v**6/2048 + 33*sqrt(2)*I*u**2*v**7/2048 + 429*sqrt(2)*I*u*v**8/8192 - 715*sqrt(2)*I*v**9/8192)*\epsilon**9 + (4*(u)_{x}*u**7 - 4*(v)_{x}*v**7)*\epsilon**8 + (3003*sqrt(2)*I*(u)_{xx}*u**6/512 + 693*sqrt(2)*I*(u)_{xx}*u**5*v/256 + 945*sqrt(2)*I*(u)_{xx}*u**4*v**2/512 + 175*sqrt(2)*I*(u)_{xx}*u**3*v**3/128 + 525*sqrt(2)*I*(u)_{xx}*u**2*v**4/512 + 189*sqrt(2)*I*(u)_{xx}*u*v**5/256 + 231*sqrt(2)*I*(u)_{xx}*v**6/512 + 15015*sqrt(2)*I*(u)_{x}**2*u**5/512 + 5775*sqrt(2)*I*(u)_{x}**2*u**4*v/512 + 1575*sqrt(2)*I*(u)_{x}**2*u**3*v**2/256 + 875*sqrt(2)*I*(u)_{x}**2*u**2*v**3/256 + 875*sqrt(2)*I*(u)_{x}**2*u*v**4/512 + 315*sqrt(2)*I*(u)_{x}**2*v**5/512 + 693*sqrt(2)*I*(u)_{x}*(v)_{x}*u**5/256 + 945*sqrt(2)*I*(u)_{x}*(v)_{x}*u**4*v/256 + 525*sqrt(2)*I*(u)_{x}*(v)_{x}*u**3*v**2/128 + 525*sq

KeyboardInterrupt: 

In [63]:
def miura(expr):
    return substituter(expr, u, deriv(v) + v**2)

for n in range(0, N, 2):
    print("n =", n)
    KdV = - simplify(Rational(1,2) * deriv(variation(SS_eq[n+1], u)))
    KdVMiura = simplify(miura(KdV))
    mKdV = simplify(multi_substituter(variation(sigq_eq[n+1], q_conj), [(q, v), (q_conj, v)]))
    display(Eq(Symbol('\\text{KdV}(u)'), KdV))
    display(Eq(Symbol('\\text{KdV}(\\text{Miura}(v))'), KdVMiura))
    display(Eq(Symbol('\\text{mKdV}(v)'), mKdV))
    display(Eq(Symbol('(\\text{mKdV}(v))_{x} + 2 \cdot v \cdot \\text{mKdV}(v)'), simplify(deriv(mKdV) + 2 * v * mKdV)))
    display(Eq(Symbol('(\\text{mKdV}(v))_{x} + 2 \cdot v \cdot \\text{mKdV}(v) - \\text{KdV}(\\text{Miura}(v))'), simplify(deriv(mKdV) + 2 * v * mKdV - KdVMiura)))

n = 0


Eq(\text{KdV}(u), 0)

Eq(\text{KdV}(\text{Miura}(v)), 0)

Eq(\text{mKdV}(v), -v)

Eq((\text{mKdV}(v))_{x} + 2 \cdot v \cdot \text{mKdV}(v), -(v)_{x} - 2*v**2)

...

Eq(\text{KdV}(u), -(u)_{xxxxxxxxx} + 18*(u)_{xxxxxxx}*u + 72*(u)_{xxxxxx}*(u)_{x} + 168*(u)_{xxxxx}*(u)_{xx} - 126*(u)_{xxxxx}*u**2 + 252*(u)_{xxxx}*(u)_{xxx} - 756*(u)_{xxxx}*(u)_{x}*u - 1260*(u)_{xxx}*(u)_{xx}*u - 966*(u)_{xxx}*(u)_{x}**2 + 420*(u)_{xxx}*u**3 - 1302*(u)_{xx}**2*(u)_{x} + 2520*(u)_{xx}*(u)_{x}*u**2 + 1260*(u)_{x}**3*u - 630*(u)_{x}*u**4)

Eq(\text{KdV}(\text{Miura}(v)), -(v)_{xxxxxxxxxx} - 2*(v)_{xxxxxxxxx}*v + 18*(v)_{xxxxxxxx}*v**2 + 180*(v)_{xxxxxxx}*(v)_{x}*v + 36*(v)_{xxxxxxx}*v**3 + 480*(v)_{xxxxxx}*(v)_{xx}*v + 462*(v)_{xxxxxx}*(v)_{x}**2 + 288*(v)_{xxxxxx}*(v)_{x}*v**2 - 126*(v)_{xxxxxx}*v**4 + 840*(v)_{xxxxx}*(v)_{xxx}*v + 2376*(v)_{xxxxx}*(v)_{xx}*(v)_{x} + 672*(v)_{xxxxx}*(v)_{xx}*v**2 + 636*(v)_{xxxxx}*(v)_{x}**2*v - 2016*(v)_{xxxxx}*(v)_{x}*v**3 - 252*(v)_{xxxxx}*v**5 + 504*(v)_{xxxx}**2*v + 3696*(v)_{xxxx}*(v)_{xxx}*(v)_{x} + 1008*(v)_{xxxx}*(v)_{xxx}*v**2 + 2706*(v)_{xxxx}*(v)_{xx}**2 + 2808*(v)_{xxxx}*(v)_{xx}*(v)_{x}*v - 4032*(v)_{xxxx}*(v)_{xx}*v**3 - 10668*(v)_{xxxx}*(v)_{x}**2*v**2 - 3024*(v)_{xxxx}*(v)_{x}*v**4 + 420*(v)_{xxxx}*v**6 + 3498*(v)_{xxx}**2*(v)_{xx} + 1788*(v)_{xxx}**2*(v)_{x}*v - 2520*(v)_{xxx}**2*v**3 + 2604*(v)_{xxx}*(v)_{xx}**2*v - 36792*(v)_{xxx}*(v)_{xx}*(v)_{x}*v**2 - 5040*(v)_{xxx}*(v)_{xx}*v**4 - 21672*(v)_{xxx}*(v)_{x}**3*v - 12264*(v)_{xxx}*(v)_{x}**2*v**3 + 7560*(v)_{xxx}*(v)

Eq(\text{mKdV}(v), -(v)_{xxxxxxxxxx} + 22*(v)_{xxxxxxxx}*v**2 - 18*(v)_{xxxxxxxx} + 176*(v)_{xxxxxxx}*(v)_{x}*v + 484*(v)_{xxxxxx}*(v)_{xx}*v + 462*(v)_{xxxxxx}*(v)_{x}**2 - 198*(v)_{xxxxxx}*v**4 + 324*(v)_{xxxxxx}*v**2 - 126*(v)_{xxxxxx} + 836*(v)_{xxxxx}*(v)_{xxx}*v + 2376*(v)_{xxxxx}*(v)_{xx}*(v)_{x} - 2376*(v)_{xxxxx}*(v)_{x}*v**3 + 1944*(v)_{xxxxx}*(v)_{x}*v + 506*(v)_{xxxx}**2*v + 3696*(v)_{xxxx}*(v)_{xxx}*(v)_{x} + 2706*(v)_{xxxx}*(v)_{xx}**2 - 5016*(v)_{xxxx}*(v)_{xx}*v**3 + 4104*(v)_{xxxx}*(v)_{xx}*v - 11220*(v)_{xxxx}*(v)_{x}**2*v**2 + 3780*(v)_{xxxx}*(v)_{x}**2 + 924*(v)_{xxxx}*v**6 - 2268*(v)_{xxxx}*v**4 + 1764*(v)_{xxxx}*v**2 - 420*(v)_{xxxx} + 3498*(v)_{xxx}**2*(v)_{xx} - 3036*(v)_{xxx}**2*v**3 + 2484*(v)_{xxx}**2*v - 39336*(v)_{xxx}*(v)_{xx}*(v)_{x}*v**2 + 13608*(v)_{xxx}*(v)_{xx}*(v)_{x} - 21120*(v)_{xxx}*(v)_{x}**3*v + 11088*(v)_{xxx}*(v)_{x}*v**5 - 18144*(v)_{xxx}*(v)_{x}*v**3 + 7056*(v)_{xxx}*(v)_{x}*v - 9372*(v)_{xx}**3*v**2 + 3276*(v)_{xx}**3 - 44748*(v)_{xx}**2*(v

Eq((\text{mKdV}(v))_{x} + 2 \cdot v \cdot \text{mKdV}(v), -(v)_{xxxxxxxxxxx} - 2*(v)_{xxxxxxxxxx}*v + 22*(v)_{xxxxxxxxx}*v**2 - 18*(v)_{xxxxxxxxx} + 220*(v)_{xxxxxxxx}*(v)_{x}*v + 44*(v)_{xxxxxxxx}*v**3 - 36*(v)_{xxxxxxxx}*v + 660*(v)_{xxxxxxx}*(v)_{xx}*v + 638*(v)_{xxxxxxx}*(v)_{x}**2 + 352*(v)_{xxxxxxx}*(v)_{x}*v**2 - 198*(v)_{xxxxxxx}*v**4 + 324*(v)_{xxxxxxx}*v**2 - 126*(v)_{xxxxxxx} + 1320*(v)_{xxxxxx}*(v)_{xxx}*v + 3784*(v)_{xxxxxx}*(v)_{xx}*(v)_{x} + 968*(v)_{xxxxxx}*(v)_{xx}*v**2 + 924*(v)_{xxxxxx}*(v)_{x}**2*v - 3168*(v)_{xxxxxx}*(v)_{x}*v**3 + 2592*(v)_{xxxxxx}*(v)_{x}*v - 396*(v)_{xxxxxx}*v**5 + 648*(v)_{xxxxxx}*v**3 - 252*(v)_{xxxxxx}*v + 1848*(v)_{xxxxx}*(v)_{xxxx}*v + 6908*(v)_{xxxxx}*(v)_{xxx}*(v)_{x} + 1672*(v)_{xxxxx}*(v)_{xxx}*v**2 + 5082*(v)_{xxxxx}*(v)_{xx}**2 + 4752*(v)_{xxxxx}*(v)_{xx}*(v)_{x}*v - 7392*(v)_{xxxxx}*(v)_{xx}*v**3 + 6048*(v)_{xxxxx}*(v)_{xx}*v - 18348*(v)_{xxxxx}*(v)_{x}**2*v**2 + 5724*(v)_{xxxxx}*(v)_{x}**2 - 4752*(v)_{xxxxx}*(v)_{x}*v**4 + 3888*(v)_

Eq((\text{mKdV}(v))_{x} + 2 \cdot v \cdot \text{mKdV}(v) - \text{KdV}(\text{Miura}(v)), -(v)_{xxxxxxxxxxx} - 2*(v)_{xxxxxxxxxx}*v + (v)_{xxxxxxxxxx} + 22*(v)_{xxxxxxxxx}*v**2 + 2*(v)_{xxxxxxxxx}*v - 18*(v)_{xxxxxxxxx} + 220*(v)_{xxxxxxxx}*(v)_{x}*v + 44*(v)_{xxxxxxxx}*v**3 - 18*(v)_{xxxxxxxx}*v**2 - 36*(v)_{xxxxxxxx}*v + 660*(v)_{xxxxxxx}*(v)_{xx}*v + 638*(v)_{xxxxxxx}*(v)_{x}**2 + 352*(v)_{xxxxxxx}*(v)_{x}*v**2 - 180*(v)_{xxxxxxx}*(v)_{x}*v - 198*(v)_{xxxxxxx}*v**4 - 36*(v)_{xxxxxxx}*v**3 + 324*(v)_{xxxxxxx}*v**2 - 126*(v)_{xxxxxxx} + 1320*(v)_{xxxxxx}*(v)_{xxx}*v + 3784*(v)_{xxxxxx}*(v)_{xx}*(v)_{x} + 968*(v)_{xxxxxx}*(v)_{xx}*v**2 - 480*(v)_{xxxxxx}*(v)_{xx}*v + 924*(v)_{xxxxxx}*(v)_{x}**2*v - 462*(v)_{xxxxxx}*(v)_{x}**2 - 3168*(v)_{xxxxxx}*(v)_{x}*v**3 - 288*(v)_{xxxxxx}*(v)_{x}*v**2 + 2592*(v)_{xxxxxx}*(v)_{x}*v - 396*(v)_{xxxxxx}*v**5 + 126*(v)_{xxxxxx}*v**4 + 648*(v)_{xxxxxx}*v**3 - 252*(v)_{xxxxxx}*v + 1848*(v)_{xxxxx}*(v)_{xxxx}*v + 6908*(v)_{xxxxx}*(v)_{xxx}*(v)_{x} + 1672*(v

In [49]:
def miura(expr):
    return substituter(expr, u, deriv(v) + v**2)

for n in range(0, N, 2):
    print("n =", n)
    KdV = - simplify(Rational(1,2) * deriv(variation(SS_eq[n+1], u)))
    KdVMiura = simplify(miura(KdV))
    mKdV = simplify(multi_substituter(variation(sigs_eq[n], q_conj), [(q, v), (q_conj, v)]))
    display(Eq(Symbol('\\text{KdV}(u)'), KdV))
    display(Eq(Symbol('\\text{KdV}(\\text{Miura}(v))'), KdVMiura))
    display(Eq(Symbol('\\text{mKdV}(v)'), mKdV))
    display(Eq(Symbol('(\\text{mKdV}(v))_{x} + 2 \cdot v \cdot \\text{mKdV}(v)'), simplify(deriv(mKdV) + 2 * v * mKdV)))
    display(Eq(Symbol('(\\text{mKdV}(v))_{x} + 2 \cdot v \cdot \\text{mKdV}(v) - \\text{KdV}(\\text{Miura}(v))'), simplify(deriv(mKdV) + 2 * v * mKdV - KdVMiura)))

n = 0


Eq(\text{KdV}(u), 0)

Eq(\text{KdV}(\text{Miura}(v)), 0)

Eq(\text{mKdV}(v), 0)

Eq((\text{mKdV}(v))_{x} + 2 \cdot v \cdot \text{mKdV}(v), 0)

...

Eq(\text{KdV}(u), -(u)_{xxxxxxxxx} + 18*(u)_{xxxxxxx}*u + 72*(u)_{xxxxxx}*(u)_{x} + 168*(u)_{xxxxx}*(u)_{xx} - 126*(u)_{xxxxx}*u**2 + 252*(u)_{xxxx}*(u)_{xxx} - 756*(u)_{xxxx}*(u)_{x}*u - 1260*(u)_{xxx}*(u)_{xx}*u - 966*(u)_{xxx}*(u)_{x}**2 + 420*(u)_{xxx}*u**3 - 1302*(u)_{xx}**2*(u)_{x} + 2520*(u)_{xx}*(u)_{x}*u**2 + 1260*(u)_{x}**3*u - 630*(u)_{x}*u**4)

Eq(\text{KdV}(\text{Miura}(v)), -(v)_{xxxxxxxxxx} - 2*(v)_{xxxxxxxxx}*v + 18*(v)_{xxxxxxxx}*v**2 + 180*(v)_{xxxxxxx}*(v)_{x}*v + 36*(v)_{xxxxxxx}*v**3 + 480*(v)_{xxxxxx}*(v)_{xx}*v + 462*(v)_{xxxxxx}*(v)_{x}**2 + 288*(v)_{xxxxxx}*(v)_{x}*v**2 - 126*(v)_{xxxxxx}*v**4 + 840*(v)_{xxxxx}*(v)_{xxx}*v + 2376*(v)_{xxxxx}*(v)_{xx}*(v)_{x} + 672*(v)_{xxxxx}*(v)_{xx}*v**2 + 636*(v)_{xxxxx}*(v)_{x}**2*v - 2016*(v)_{xxxxx}*(v)_{x}*v**3 - 252*(v)_{xxxxx}*v**5 + 504*(v)_{xxxx}**2*v + 3696*(v)_{xxxx}*(v)_{xxx}*(v)_{x} + 1008*(v)_{xxxx}*(v)_{xxx}*v**2 + 2706*(v)_{xxxx}*(v)_{xx}**2 + 2808*(v)_{xxxx}*(v)_{xx}*(v)_{x}*v - 4032*(v)_{xxxx}*(v)_{xx}*v**3 - 10668*(v)_{xxxx}*(v)_{x}**2*v**2 - 3024*(v)_{xxxx}*(v)_{x}*v**4 + 420*(v)_{xxxx}*v**6 + 3498*(v)_{xxx}**2*(v)_{xx} + 1788*(v)_{xxx}**2*(v)_{x}*v - 2520*(v)_{xxx}**2*v**3 + 2604*(v)_{xxx}*(v)_{xx}**2*v - 36792*(v)_{xxx}*(v)_{xx}*(v)_{x}*v**2 - 5040*(v)_{xxx}*(v)_{xx}*v**4 - 21672*(v)_{xxx}*(v)_{x}**3*v - 12264*(v)_{xxx}*(v)_{x}**2*v**3 + 7560*(v)_{xxx}*(v)

Eq(\text{mKdV}(v), -(v)_{xxxxxxxxx} + 18*(v)_{xxxxxxx}*v**2 + 144*(v)_{xxxxxx}*(v)_{x}*v + 336*(v)_{xxxxx}*(v)_{xx}*v + 318*(v)_{xxxxx}*(v)_{x}**2 - 126*(v)_{xxxxx}*v**4 + 504*(v)_{xxxx}*(v)_{xxx}*v + 1404*(v)_{xxxx}*(v)_{xx}*(v)_{x} - 1512*(v)_{xxxx}*(v)_{x}*v**3 + 894*(v)_{xxx}**2*(v)_{x} + 1302*(v)_{xxx}*(v)_{xx}**2 - 2520*(v)_{xxx}*(v)_{xx}*v**3 - 6132*(v)_{xxx}*(v)_{x}**2*v**2 + 420*(v)_{xxx}*v**6 - 8484*(v)_{xx}**2*(v)_{x}*v**2 - 9408*(v)_{xx}*(v)_{x}**3*v + 5040*(v)_{xx}*(v)_{x}*v**5 - 798*(v)_{x}**5 + 6300*(v)_{x}**3*v**4 - 630*(v)_{x}*v**8)

Eq((\text{mKdV}(v))_{x} + 2 \cdot v \cdot \text{mKdV}(v), -(v)_{xxxxxxxxxx} - 2*(v)_{xxxxxxxxx}*v + 18*(v)_{xxxxxxxx}*v**2 + 180*(v)_{xxxxxxx}*(v)_{x}*v + 36*(v)_{xxxxxxx}*v**3 + 480*(v)_{xxxxxx}*(v)_{xx}*v + 462*(v)_{xxxxxx}*(v)_{x}**2 + 288*(v)_{xxxxxx}*(v)_{x}*v**2 - 126*(v)_{xxxxxx}*v**4 + 840*(v)_{xxxxx}*(v)_{xxx}*v + 2376*(v)_{xxxxx}*(v)_{xx}*(v)_{x} + 672*(v)_{xxxxx}*(v)_{xx}*v**2 + 636*(v)_{xxxxx}*(v)_{x}**2*v - 2016*(v)_{xxxxx}*(v)_{x}*v**3 - 252*(v)_{xxxxx}*v**5 + 504*(v)_{xxxx}**2*v + 3696*(v)_{xxxx}*(v)_{xxx}*(v)_{x} + 1008*(v)_{xxxx}*(v)_{xxx}*v**2 + 2706*(v)_{xxxx}*(v)_{xx}**2 + 2808*(v)_{xxxx}*(v)_{xx}*(v)_{x}*v - 4032*(v)_{xxxx}*(v)_{xx}*v**3 - 10668*(v)_{xxxx}*(v)_{x}**2*v**2 - 3024*(v)_{xxxx}*(v)_{x}*v**4 + 420*(v)_{xxxx}*v**6 + 3498*(v)_{xxx}**2*(v)_{xx} + 1788*(v)_{xxx}**2*(v)_{x}*v - 2520*(v)_{xxx}**2*v**3 + 2604*(v)_{xxx}*(v)_{xx}**2*v - 36792*(v)_{xxx}*(v)_{xx}*(v)_{x}*v**2 - 5040*(v)_{xxx}*(v)_{xx}*v**4 - 21672*(v)_{xxx}*(v)_{x}**3*v - 12264*(v)_{xxx}*(v)_{x}**2

Eq((\text{mKdV}(v))_{x} + 2 \cdot v \cdot \text{mKdV}(v) - \text{KdV}(\text{Miura}(v)), 0)

In [10]:


def q_coeff(n):
    if n == -1:
        return epsilon / (I * sqrt(2))
    if n < -1 or (n % 2 == 0 and n >= 0):
        return 0
    else:
        return I * catalan((n - 1) // 2) / sqrt(2)**n * epsiloninv**n
       
def q_coeff_plus(n):
    if n == 0:
        return 1
    return q_coeff(n)

def q_coeff_minus(n):
    if n == 0:
        return 1
    return - q_coeff(n)
    
def qinv_coeff(n):
    if n < 0 or n % 2 == 0:
        return 0
    return binomial(- Rational(1,2), (n-1)//2) * sqrt(2)**n * I**n * epsiloninv**n

def sqrtt(x):
    return I*sqrt(-x)


X = Symbol('X')

def get_expansion(eqn, x0, var, num):
    epsilonpos = Symbol('epsilon_2', positive=True)
    epsiloninvpos = Symbol('eta_2', positive=True)
    
    eqn = eqn.subs(epsilon, epsilonpos).subs(epsiloninv, epsiloninvpos)
    ser = series(eqn, x0=0, x=var, n=num).removeO().subs(epsilonpos, epsilon).subs(epsiloninvpos, epsiloninv)
    return [simplify(ser.coeff(var**n) if n > 0 else ser.subs(var, 0)) for n in range(0, num)]
    
N = 8

h_plus = get_expansion(1 + 1 / sqrtt(1 - Rational(1, 2) / epsiloninv**2 / X**2), 0, X, N+1)
h_minus = get_expansion(1 - 1 / sqrtt(1 - Rational(1, 2) / epsiloninv**2 / X**2), 0, X, N+1)

    
#h_plus = [binomial(-Rational(1, 2), n//2) * (sqrt(2) * I * epsiloninv)**n if n % 2 == 1 else 0 for n in range(N+1)]
#h_plus[0] = 1
#h_minus = [- binomial(-Rational(1, 2), n//2) * (sqrt(2) * I * epsiloninv)**n if n % 2 == 1 else 0 for n in range(N+1)]
#h_minus[0] = 1

h_plus_mod = get_expansion(1 + 1 / sqrtt(1 + Rational(1, 2) / epsiloninv**2 / X**2), 0, X, N+1)
h_minus_mod = get_expansion(1 - 1 / sqrtt(1 + Rational(1, 2) / epsiloninv**2 / X**2), 0, X, N+1)

exp1 = [get_expansion((X / sqrtt(1 - 1 / (8 * epsiloninv**2 * X**2)))**n, 0, X, N+1) for n in range(N+1)]

E_plus_sym = [Symbol('E^+_{' + str(n) + '}') for n in range(N+1)]
E_plus_eq = [E_plus_sym[n] for n in range(N+1)]

E_minus_sym = [Symbol('E^-_{' + str(n) + '}') for n in range(N+1)]
E_minus_eq = [E_plus_sym[n] for n in range(N+1)]

F_plus_sym = [Symbol('F^+_{' + str(n) + '}') for n in range(N+1)]
F_plus_eq = [E_plus_sym[n] for n in range(N+1)]

F_minus_sym = [Symbol('F^-_{' + str(n) + '}') for n in range(N+1)]
F_minus_eq = [E_plus_sym[n] for n in range(N+1)]

Ft_plus_sym = [Symbol('\\tilde{F}^+_{' + str(n) + '}') for n in range(N+1)]
Ft_plus_eq = [E_plus_sym[n] for n in range(N+1)]

Ft_minus_sym = [Symbol('\\tilde{F}^-_{' + str(n) + '}') for n in range(N+1)]
Ft_minus_eq = [E_plus_sym[n] for n in range(N+1)]

G_sym = [Symbol('G_{' + str(n) + '}') for n in range(N+1)]
G_eq = [G_sym[n] for n in range(N+1)]

r_factor = 1 #Symbol('lambda', real=True) #1

r_sym = [Symbol('r_{' + str(n) + '}') for n in range(N+1)]
r_eq = [r_sym[n] for n in range(N+1)]
r_poly = [r_sym[n] for n in range(N+1)]

r_eq[0] = r_factor * I
r_eq[1] = r_factor * epsilon / sqrt(2) * (v - u + 2 * epsiloninv**2)
r_poly[0] = polynomize(r_eq[0])
r_poly[1] = polynomize(r_eq[1])

sig_sym = [Symbol('\sigma_{' + str(n) + '}') for n in range(N+1)]
sig_eq = [sig_sym[n] for n in range(N+1)]
sig_poly = [sig_sym[n] for n in range(N+1)]

sig_eq[0] = q_coeff(-1) / 2 * (v + u)
sig_eq[1] = q_coeff(-1) / 2 * deriv(v + u) + q_coeff(-1)**2 / 4 * (v + u)**2 
sig_eq[1] += epsilon**2 / 2 * u * v + (u - v) / 2 
sig_eq[1] += epsilon / sqrt(2) * deriv(v) * r_eq[0]  

sig_poly[0] = polynomize(sig_eq[0])
sig_poly[1] = polynomize(sig_eq[1])

sig_plus_sym = [Symbol('\sigma_+^{' + str(n) + '}') for n in range(N+1)]
sig_plus_eq = [sig_plus_sym[n] for n in range(N+1)]

sig_minus_sym = [Symbol('\sigma_-^{' + str(n) + '}') for n in range(N+1)]
sig_minus_eq = [sig_minus_sym[n] for n in range(N+1)]

sig_plus_eq[0] = 0
sig_plus_eq[1] = - deriv(sig_plus_eq[0]) + sig_plus_eq[0]**2 - v

sig_minus_eq[0] = 0
sig_minus_eq[1] = deriv(sig_minus_eq[0]) + sig_minus_eq[0]**2 + u

for n in range(1, N):
    print("n =", n)

    X = - deriv(sig_plus_eq[n])
    
    for k in range(n+1):
        X += sig_plus_eq[k] * sig_plus_eq[n-k]
        
    sig_plus_eq[n+1] = simplify(X)
        
    X = deriv(sig_minus_eq[n])

    for k in range(n+1):
        X += sig_minus_eq[k] * sig_minus_eq[n-k]

    sig_minus_eq[n+1] = simplify(X)
        
    X = 0
    
    X += deriv(r_eq[n])

    X += - q_coeff(n) / sqrt(2) * epsiloninv
    
    for k in range(0, n+1):
        X += epsilon / sqrt(2) * (- v - epsiloninv**2) * r_eq[n-k] * r_eq[k]

    for m in range(0, n+1):
        for k in range(0, m+1):     
            X += - epsiloninv / sqrt(2) * q_coeff(n-m) * r_eq[m-k] * r_eq[k]

    for k in range(1, n+1):
        X += - q_coeff(-1) / sqrt(2) * epsiloninv * r_eq[n+1-k] * r_eq[k]

    r_eq[n+1] = r_factor * X
    r_poly[n+1] = polynomize(r_eq[n+1])
    r_eq[n+1] = depolynomize(r_poly[n+1])
    
    X = 0
    
    X += deriv(sig_eq[n])
    
    for k in range(0, n+1):
        X += sig_eq[n-k] * sig_eq[k]
        
    X += q_coeff(n) / 2 * (u + v)
    
    X += epsilon / sqrt(2) * deriv(v) * r_eq[n]
    
    sig_eq[n+1] = X
    sig_poly[n+1] = polynomize(sig_eq[n+1])
    sig_eq[n+1] = depolynomize(sig_poly[n+1])

for n in range(0, N+1):
    E_plus_eq[n] = 0
    for k in range(0, n+1):
        E_plus_eq[n] += re(sig_eq[k] * h_plus[n-k])
        #E_plus_eq[n] += re(epsilon * r_eq[k] * h_plus[n-k])
        #E_plus_eq[n] += re(sig_eq[k] * q_coeff_plus(n-k))
    E_minus_eq[n] = 0
    for k in range(0, n+1):
        E_minus_eq[n] += re(sig_eq[k] * h_minus[n-k])
        #E_minus_eq[n] += re(epsilon * r_eq[k] * h_minus[n-k])
        #E_minus_eq[n] += re(sig_eq[k] * q_coeff_minus(n-k))
        
    F_plus_eq[n] = 0
    for k in range(0, n+1):
        F_plus_eq[n] += re(epsilon * r_eq[k] * h_plus[n-k])
        
    F_minus_eq[n] = 0
    for k in range(0, n+1):
        F_minus_eq[n] += re(epsilon * r_eq[k] * h_minus[n-k])
        
    G_eq[n] = 0
    for k in range(0, n+1):
        G_eq[n] += im(sig_eq[k]) * qinv_coeff(n-k) / I
    

n = 1
n = 2
n = 3
n = 4
n = 5
new differentiable symbol:  (u)_{xxxxxx}
new differentiable symbol:  (v)_{xxxxxx}
n = 6
new differentiable symbol:  (u)_{xxxxxxx}
new differentiable symbol:  (v)_{xxxxxxx}
n = 7
new differentiable symbol:  (u)_{xxxxxxxx}
new differentiable symbol:  (v)_{xxxxxxxx}


In [7]:
for n in range(5):
    display(Eq(sig_sym[n], polynomize(sig_eq[n])))
    display(Eq(r_sym[n], polynomize(r_eq[n] / I)))

Eq(\sigma_{0}, Poly((-sqrt(2)*I*u/4 - sqrt(2)*I*v/4)*\epsilon, \epsilon, 1/\epsilon, domain='EX'))

Eq(r_{0}, Poly(1, \epsilon, 1/\epsilon, domain='ZZ'))

Eq(\sigma_{1}, Poly((-u**2/8 + u*v/4 - v**2/8)*\epsilon**2 + (-sqrt(2)*I*(u)_{x}/4 + sqrt(2)*I*(v)_{x}/4)*\epsilon + u/2 - v/2, \epsilon, 1/\epsilon, domain='EX'))

Eq(r_{1}, Poly((sqrt(2)*I*u/2 - sqrt(2)*I*v/2)*\epsilon - sqrt(2)*I*(1/\epsilon), \epsilon, 1/\epsilon, domain='EX'))

Eq(\sigma_{2}, Poly((sqrt(2)*I*u**3/16 - sqrt(2)*I*u**2*v/16 - sqrt(2)*I*u*v**2/16 + sqrt(2)*I*v**3/16)*\epsilon**3 + (-(u)_{x}*u/2 + (v)_{x}*v/2)*\epsilon**2 + (-sqrt(2)*I*(u)_{xx}/4 + sqrt(2)*I*(v)_{xx}/4 - sqrt(2)*I*u**2/4 + sqrt(2)*I*v**2/4)*\epsilon + (sqrt(2)*I*u/4 + sqrt(2)*I*v/4)*(1/\epsilon) + (u)_{x}/2 + (v)_{x}/2, \epsilon, 1/\epsilon, domain='EX'))

Eq(r_{2}, Poly((u**2/4 + u*v/2 - 3*v**2/4)*\epsilon**2 + (sqrt(2)*I*(u)_{x}/2 - sqrt(2)*I*(v)_{x}/2)*\epsilon - (1/\epsilon)**2 - 2*v, \epsilon, 1/\epsilon, domain='EX'))

Eq(\sigma_{3}, Poly((5*u**4/64 - u**3*v/16 - u**2*v**2/32 - u*v**3/16 + 5*v**4/64)*\epsilon**4 + (sqrt(2)*I*(u)_{x}*u**2/2 - sqrt(2)*I*(v)_{x}*v**2/2)*\epsilon**3 + (-3*(u)_{xx}*u/4 - (u)_{xx}*v/4 - 5*(u)_{x}**2/8 - (u)_{x}*(v)_{x}/4 + (v)_{xx}*u/4 + 3*(v)_{xx}*v/4 + 7*(v)_{x}**2/8 - 3*u**3/8 + u**2*v/8 - u*v**2/8 + 3*v**3/8)*\epsilon**2 + (-sqrt(2)*I*(u)_{xxx}/4 - sqrt(2)*I*(u)_{x}*u + sqrt(2)*I*(v)_{xxx}/4 - sqrt(2)*I*(v)_{x}*v)*\epsilon + (sqrt(2)*I*(u)_{x}/4 - sqrt(2)*I*(v)_{x}/4)*(1/\epsilon) + (u)_{xx}/2 + (v)_{xx}/2 + u**2/2 + v**2/2, \epsilon, 1/\epsilon, domain='EX'))

Eq(r_{3}, Poly((-sqrt(2)*I*u**3/8 - sqrt(2)*I*u**2*v/8 - 3*sqrt(2)*I*u*v**2/8 + 5*sqrt(2)*I*v**3/8)*\epsilon**3 + ((u)_{x}*u + (u)_{x}*v - 2*(v)_{x}*v)*\epsilon**2 + (sqrt(2)*I*(u)_{xx}/2 - sqrt(2)*I*(v)_{xx}/2 + sqrt(2)*I*u**2/4 - sqrt(2)*I*u*v/2 + 9*sqrt(2)*I*v**2/4)*\epsilon + 2*sqrt(2)*I*v*(1/\epsilon) - 2*(v)_{x}, \epsilon, 1/\epsilon, domain='EX'))

Eq(\sigma_{4}, Poly((-7*sqrt(2)*I*u**5/128 + 5*sqrt(2)*I*u**4*v/128 + sqrt(2)*I*u**3*v**2/64 + sqrt(2)*I*u**2*v**3/64 + 5*sqrt(2)*I*u*v**4/128 - 7*sqrt(2)*I*v**5/128)*\epsilon**5 + ((u)_{x}*u**3 - (v)_{x}*v**3)*\epsilon**4 + (15*sqrt(2)*I*(u)_{xx}*u**2/16 + 3*sqrt(2)*I*(u)_{xx}*u*v/8 + 3*sqrt(2)*I*(u)_{xx}*v**2/16 + 25*sqrt(2)*I*(u)_{x}**2*u/16 + 5*sqrt(2)*I*(u)_{x}**2*v/16 + 3*sqrt(2)*I*(u)_{x}*(v)_{x}*u/8 + 3*sqrt(2)*I*(u)_{x}*(v)_{x}*v/8 - 3*sqrt(2)*I*(v)_{xx}*u**2/16 - 3*sqrt(2)*I*(v)_{xx}*u*v/8 - 15*sqrt(2)*I*(v)_{xx}*v**2/16 - 7*sqrt(2)*I*(v)_{x}**2*u/16 - 35*sqrt(2)*I*(v)_{x}**2*v/16 + 5*sqrt(2)*I*u**4/16 - sqrt(2)*I*u**3*v/8 + sqrt(2)*I*u*v**3/8 - 5*sqrt(2)*I*v**4/16)*\epsilon**3 + (-(u)_{xxx}*u - (u)_{xxx}*v/2 - 9*(u)_{xx}*(u)_{x}/4 - 3*(u)_{xx}*(v)_{x}/4 + (u)_{x}*(v)_{xx}/4 - 3*(u)_{x}*u**2 + (v)_{xxx}*u/2 + (v)_{xxx}*v + 11*(v)_{xx}*(v)_{x}/4 - 3*(v)_{x}*v**2)*\epsilon**2 + (-sqrt(2)*I*(u)_{xxxx}/4 - 3*sqrt(2)*I*(u)_{xx}*u/2 - 5*sqrt(2)*I*(u)_{x}**2/4 + sqrt(2)*I*(v)_{xxxx}

Eq(r_{4}, Poly((-5*u**4/32 - u**3*v/8 - 3*u**2*v**2/16 - 5*u*v**3/8 + 35*v**4/32)*\epsilon**4 + (-sqrt(2)*I*(u)_{x}*u**2 - sqrt(2)*I*(u)_{x}*u*v - sqrt(2)*I*(u)_{x}*v**2 + 3*sqrt(2)*I*(v)_{x}*v**2)*\epsilon**3 + (3*(u)_{xx}*u/2 + 3*(u)_{xx}*v/2 + 5*(u)_{x}**2/4 + (u)_{x}*(v)_{x}/2 - (v)_{xx}*u/2 - 5*(v)_{xx}*v/2 - 7*(v)_{x}**2/4 + u**3/2 - 3*u*v**2/2 + 5*v**3)*\epsilon**2 + (sqrt(2)*I*(u)_{xxx}/2 + sqrt(2)*I*(u)_{x}*u - sqrt(2)*I*(u)_{x}*v - sqrt(2)*I*(v)_{xxx}/2 + 6*sqrt(2)*I*(v)_{x}*v)*\epsilon - 1/2*(1/\epsilon)**4 + 2*v*(1/\epsilon)**2 + 2*sqrt(2)*I*(v)_{x}*(1/\epsilon) - 2*(v)_{xx} - u**2/4 - u*v/2 + 27*v**2/4, \epsilon, 1/\epsilon, domain='EX'))

In [6]:
def q_coeff(n):
    if n == -1:
        #return Symbol('q_{' + str(n) + '}', real=True) * epsilon / I
        return epsilon / (I * sqrt(2))
    if n < -1 or (n % 2 == 0 and n >= 0):
        return 0
    else:
        #if n > 1:
            #return Symbol('q_{' + str(n) + '}', real=True) * I * epsiloninv**n
        return I * catalan((n - 1) // 2) / sqrt(2)**n * epsiloninv**n
       
def alt_q_coeff(n):
    if n == -1:
        return epsilon / I * Symbol('q_{-1}', real=True)
    if n < -1 or (n % 2 == 0 and n >= 0):
        return 0
    else:
        return I * epsiloninv**n * Symbol('q_{' + str(n) + '}', real=True)
    
def plus_beta_coeff(n):
    if n == 0:
        return - 1 * Rational(1, 2)
    else:
        return - q_coeff(n) * Rational(1, 2)
    
def minus_beta_coeff(n):
    if n == 0:
        return - 1 * Rational(1, 2)
    else:
        return q_coeff(n) * Rational(1, 2)

def alt_beta_coeff(n):
    if n == 0:
        return - 1 #Rational(1, 2)
    else:
        return 0
    
N = 8

S_sym = [Symbol('S_{' + str(n) + '}') for n in range(N+1)]
S_eq = [S_sym[n] for n in range(N+1)]

SS_sym = [Symbol('SS_{' + str(n) + '}') for n in range(N+1)]
SS_eq = [S_sym[n] for n in range(N+1)]

S_plus_sym = [Symbol('S^+_{' + str(n) + '}') for n in range(N+1)]
S_plus_eq = [S_plus_sym[n] for n in range(N+1)]

S_minus_sym = [Symbol('S^-_{' + str(n) + '}') for n in range(N+1)]
S_minus_eq = [S_plus_sym[n] for n in range(N+1)]

sigq_sym = [Symbol('\\sigma^q_{' + str(n) + '}') for n in range(N+1)]
sigq_eq = [sigq_sym[n] for n in range(N+1)]

rq_sym = [Symbol('r^q_{' + str(n) + '}') for n in range(N+1)]
rq_eq = [rq_sym[n] for n in range(N+1)]

Rq_sym = [Symbol('R^q_{' + str(n) + '}') for n in range(N+1)]
Rq_eq = [rq_sym[n] for n in range(N+1)]

sigs_sym = [Symbol('\\sigma^s_{' + str(n) + '}') for n in range(N+1)]
sigs_eq = [sigs_sym[n] for n in range(N+1)]

S_eq[0] = 0
S_eq[1] = u

SS_eq[0] = 0
SS_eq[1] = - u

S_plus_eq[0] = 0
S_plus_eq[1] = u+v

S_minus_eq[0] = 0
S_minus_eq[1] = u-v

sigq_eq[0] = 0
sigq_eq[1] = 1 - q * q_conj

sigs_eq[0] = 0
sigs_eq[1] = - q * q_conj

rq_eq[0] = 0
rq_eq[1] = q_conj

Rq_eq[0] = 0
Rq_eq[1] = q_conj

for n in range(1, N):
    print("n =", n)
    
    X = deriv(S_eq[n])
    
    for k in range(n+1):
        X += S_eq[k] * S_eq[n-k]

    S_eq[n+1] = poly_simplify(X)
    
    X = deriv(SS_eq[n])
    
    for k in range(n+1):
        X += SS_eq[k] * SS_eq[n-k]

    SS_eq[n+1] = poly_simplify(X)
    
    
    
    X = deriv(S_plus_eq[n])
    
    for k in range(n+1):
        X += (S_plus_eq[k] + S_minus_eq[k])/2 * S_plus_eq[n-k]
        
    S_plus_eq[n+1] = poly_simplify(X)
        
    X = - deriv(S_minus_eq[n])

    for k in range(n+1):
        X += - (S_plus_eq[k] + S_minus_eq[k])/2 * S_minus_eq[n-k]

    S_minus_eq[n+1] = poly_simplify(X)

    
    X = 0
    
    for m in range(0, n//2 + 1):
        X += - deriv(rq_eq[n-2*m]) * binomial(m - Rational(n,2), m) * 4**m
        
    for k in range(0, n + 1):
        for m1 in range(0, k//2 + 1):
            for m2 in range(0, (n-k)//2 + 1):
                X += - q * rq_eq[k-2*m1] * binomial(m1 - Rational(k,2), m1) * 4**m1 * rq_eq[n-k-2*m2] * binomial(m2 - Rational(n-k,2), m2) * 4**m2
    
    for m in range(1, (n+1)//2 + 1):
        X += - rq_eq[n+1-2*m] * binomial(m - Rational(n+1,2), m) * 4**m
        
    rq_eq[n+1] = simplify(X)
    
    X = - deriv(Rq_eq[n])
    
    for k in range(0, n + 1):
        X += - q * (Rq_eq[k] * Rq_eq[n-k])
        
    Rq_eq[n+1] = simplify(X)
    
    X = deriv(sigq_eq[n])
    
    for k in range(0, n + 1):
        X += sigq_eq[k] * sigq_eq[n-k]
        
    X += - deriv(q) * rq_eq[n]
    
    sigq_eq[n+1] = simplify(X)
    
    X = deriv(sigs_eq[n])
    
    for k in range(0, n + 1):
        X += sigs_eq[k] * sigs_eq[n-k]
        
    X += - deriv(q) * Rq_eq[n]
    
    sigs_eq[n+1] = simplify(X)
    
    #display(Eq(S_sym[n], S_eq[n]))
    #display(Eq(S_plus_sym[n], S_plus_eq[n]))
    #display(Eq(S_minus_sym[n], S_minus_eq[n]))
    
    #display(Eq(S_sym[n], simplify(deriv(variation(S_eq[n], u)) / 2)))
    #display(Eq(S_plus_sym[n], simplify(deriv(variation(S_plus_eq[n], u)) / 2)))
    #display(Eq(S_minus_sym[n], simplify(deriv(variation(S_minus_eq[n], v)) / 2)))


n = 1
n = 2
n = 3
n = 4
n = 5
n = 6
n = 7
new differentiable symbol:  (q)_{xxxxxxx}


In [ ]:
for n in range(10):
    error = 0
    
    if n % 2 == 0:
        m = n // 2
        for k in range(m+1):
            error += binomial(-k, m-k) * (-4)**(m-k) * sigs_eq[2*k]
            
    elif n % 2 == 1:
        m = (n - 1) // 2
        for k in range(m+1):
            error += binomial(-k-Rational(1,2), m-k) * (-4)**(m-k) * sigs_eq[2*k+1]
        error += catalan(m)
      
    error -= sigq_eq[n]
    
    if simplify(error) != 0:
        print("sigs error n =", n)
        display(simplify(error))

print("Check 1 done")

In [8]:
for m in range(10):
    X = 0
    for l in range(m+1):
        X += (-1)**l * (catalan(l+1) - 2 * catalan(l)) * 4**(-l) * binomial(m, m-l)
    X += 4**(-m) * catalan(m+1) * (m+1)
    display(X)

0

0

0

0

0

0

0

0

0

0

In [61]:
N = 20

def AA(n, j):
    return (-1)**(n+1) * 4**j * binomial(Rational(n,2) - 1, j)

def AAt(n, j):
    return - 4**j * binomial(Rational(n,2) - 1, j)

def BB(n, j):
    return (-1)**(n+1) * sum([(-1)**l * (catalan(l+1) - 2 * catalan(l)) * 4**(j-l) * binomial(Rational(n-1,2) - 1, j - l) for l in range(0, j+1)])

def BBt(n, j):
    return - sum([(-1)**l * (catalan(l+1) - 2 * catalan(l)) * 4**(j-l) * binomial(Rational(n-1,2) - 1, j - l) for l in range(0, j+1)])

def DD(n, j):
    return 4**j * binomial(Rational(n,2) - 1, j)

def DDt(n, j):
    return 4**j * binomial(Rational(n,2) - 1, j)

def EE(n, j):
    return - 4**j * binomial(Rational(n,2) - 1, j) - sum([(-1)**l * catalan(l) * 4**(j-l) * binomial(Rational(n,2) - 1, j - l) for l in range(0, j+1)])

def EEt(n, j):
    #return sum([catalan(l+1)* (-4)**(j-l) * binomial(1+j-Rational(n,2), j-l) for l in range(0,j+1)])
    return sum([(-1)**l * (catalan(l+1) - 2 * catalan(l)) * 4**(j-l) * binomial(Rational(n-1,2) - 1, j - l) for l in range(0, j+1)])

def FFt(n, j):
    if n % 2 == 0:
        return sum([4*(n//2-j-1) * catalan(k) * 4**(j-1-k) * binomial(Rational(n-1,2)-1-k, j-1-k) for k in range(j)])
    if n % 2 == 1:
        Y = 0
        Y += - 4**j * Rational(1,2) * binomial(Rational(n,2)-1, j-1)
        Y += 4**j * (2*j+1) * binomial(Rational(n,2)-1, j)
        Y += sum([(j+1) * catalan(k) * 4**(j-k) * binomial(Rational(n-1,2)-1-k, j-k) for k in range(j+1)])
        return Y 

def FFtt(n, j):
    Y = (-1)**(n+1) * sum([4*((n-1)//2-j) * catalan(k) * 4**(j-1-k) * binomial((n-1)//2-Rational(1,2)-k, j-1-k) for k in range(j)])
    if n % 2 == 1:
        Y += 2 * 4**(j-1) * binomial(Rational(n,2)-1, j-1)
    return Y

def GGt(n, j):
    if n % 2 == 0:
        return 0
    if n % 2 == 1:
        return - 2 * (4*((n-1)//2-j)-3) * 4**j * binomial(Rational(n,2)-1, j)

A = [[0 for j in range(n//2)] for n in range(N+1)]
At = [[0 for j in range(n//2)] for n in range(N+1)]
B = [[0 for j in range(n//2-1)] for n in range(N+1)]
Bt = [[0 for j in range(n//2-1)] for n in range(N+1)]
D = [[0 for j in range(n//2)] for n in range(N+1)]
Dt = [[0 for j in range(n//2)] for n in range(N+1)]
E = [[0 for j in range(n//2)] for n in range(N+1)]
Et = [[0 for j in range(n//2-1)] for n in range(N+1)]

A[2][0] = -1

for n in range(2, N):
    for j in range((n+1)//2):
        A[n+1][j] = 0
        for k in range(j):
            A[n+1][j] += 2 * catalan(k) * A[n-2*k-1][j-1-k]
        if n%2 == 1 and j == (n-1)//2:
            A[n+1][j] += - catalan((n-1)//2) * Rational(n+1, 2)
        else:
            A[n+1][j] += - A[n][j]

At[2][0] = -1

for n in range(2, N):
    for j in range((n+1)//2):
        At[n+1][j] = 0
        for k in range(j):
            At[n+1][j] += 2 * catalan(k) * At[n-2*k-1][j-1-k]
        if n%2 == 1 and j == (n-1)//2:
            At[n+1][j] += - catalan((n-1)//2) * Rational(n+1, 2)
        else:
            At[n+1][j] += At[n][j]
            
        
A_error = 0
At_error = 0

for n in range(0, N+1):
    for j in range(n//2):
        A_error += abs(A[n][j] - AA(n, j))
        At_error += abs(At[n][j] - AAt(n, j))
print("A_error", A_error)
print("At_error", At_error)

B[4][0] = 1

for n in range(4, N):
    for j in range((n+1)//2 - 1):
        B[n+1][j] = 0
        for k in range(j):
            B[n+1][j] += 2 * catalan(k) * B[n-2*k-1][j-1-k]
        if n%2 == 1 and j == (n-3)//2:
            B[n+1][j] += catalan((n-1)//2) * Rational(n-1, 2)
        else:
            B[n+1][j] += - B[n][j]
    
Bt[4][0] = 1

for n in range(4, N):
    for j in range((n+1)//2 - 1):
        Bt[n+1][j] = 0
        for k in range(j):
            Bt[n+1][j] += 2 * catalan(k) * Bt[n-2*k-1][j-1-k]
        if n%2 == 1 and j == (n-3)//2:
            Bt[n+1][j] += catalan((n-1)//2) * Rational(n-1, 2)
        else:
            Bt[n+1][j] += Bt[n][j]
            
B_error = 0
Bt_error = 0
for n in range(0, N+1):
    for j in range(n//2 - 1):
        B_error += abs(B[n][j] - BB(n, j))
        Bt_error += abs(Bt[n][j] - BBt(n, j))
print("B_error", B_error)
print("Bt_error", Bt_error)

D[2][0] = 1

for n in range(2, N):
    for j in range((n+1)//2):
        D[n+1][j] = 0
        for k in range(j):
            D[n+1][j] += 2 * catalan(k) * D[n-2*k-1][j-1-k]
        if n%2 == 1 and j == (n-1)//2:
            D[n+1][j] += catalan((n-1)//2) * Rational(n+1, 2)
        else:
            D[n+1][j] += D[n][j]
            
Dt[2][0] = 1

for n in range(2, N):
    for j in range((n+1)//2):
        Dt[n+1][j] = 0
        for k in range(j):
            Dt[n+1][j] += 2 * catalan(k) * Dt[n-2*k-1][j-1-k]
        if n%2 == 1 and j == (n-1)//2:
            Dt[n+1][j] += catalan((n-1)//2) * Rational(n+1, 2)
        else:
            Dt[n+1][j] += Dt[n][j]
         
D_error = 0
Dt_error = 0
for n in range(0, N+1):
    for j in range(n//2):
        D_error += abs(D[n][j] - DD(n, j))
        Dt_error += abs(Dt[n][j] - DDt(n, j))
print("D_error", D_error)
print("Dt_error", Dt_error)

for n in range(0, N+1):
    #print([D[n][j] for j in range(n//2)])
    pass
for n in range(0, N+1):
    #print([Dt[n][j] for j in range(n//2)])
    pass
        
E[2][0] = -2

for n in range(2, N):
    for j in range((n+1)//2):
        E[n+1][j] = 0
        for k in range(j):
            E[n+1][j] += 2 * catalan(k) * E[n-2*k-1][j-1-k]
        if n%2 == 1 and j == (n-1)//2:
            E[n+1][j] += - catalan((n-1)//2) * Rational(n+3, 2)
        else:
            E[n+1][j] += E[n][j]
  
E_error = 0

for n in range(0, N+1):
    for j in range(n//2 - 1):
        E_error += abs(E[n][j] - EE(n, j))
print("E_error", E_error)

for n in range(0, N+1):
    #print([E[n][j] for j in range(n//2)])
    pass
        
        
F = [[[0 for t in range(n-1-2*j)] for j in range((n-1)//2)] for n in range(N+1)]
G = [[[0 for t in range((n-1)//2-j)] for j in range((n-1)//2-1)] for n in range(N+1)]
Ft = [[0 for j in range((n-1)//2)] for n in range(N+1)]
Gt = [[0 for j in range((n-1)//2-1)] for n in range(N+1)]

F[5][1][1] = -6

for n in range(3, N):
    for j in range(1, n//2):
        for t in range(1, n-2*j):
            F[n+1][j][t] = 0
            for k in range(j):
                F[n+1][j][t] += 2 * catalan(k) * F[n-2*k-1][j-1-k][t]
            for i in range(j-1):
                F[n+1][j][t] += 2 * DDt(t+1+2*i, i) * EEt(n-1-2*i-t, j-2-i)
            if j != Rational(n,2)-1 and t != n-1-2*j:
                F[n+1][j][t] += F[n][j][t]
            if j != Rational(n,2)-1 and t != 1:
                F[n+1][j][t] += F[n][j][t-1]
            if t == 1:
                F[n+1][j][t] += (j + 1) * EEt(n, j-1)
            if t == n-1-2*j:
                F[n+1][j][t] += - j * DDt(n, j)

for n in range(5, N+1):
    for j in range(1, (n-1)//2):
        Ft[n][j] = sum([(-1)**t * F[n][j][t] for t in range(1, n-1-2*j)])
        
for n in range(5, N):
    for j in range(1, n//2):
        X = 2 * sum([FFtt(n-2*k-1, j-1-k) * catalan(k) for k in range(j)])
        if n % 2 == 0:
            X += - 2 * EEt(n, j-2)
        X += - (j+1) * EEt(n, j-1)
        X += (-1)**n * j * DDt(n, j)
        
        print((n, j), X - FFtt(n+1, j))
        
for n in range(0, N+1):
    print([Ft[n][j] for j in range(1, (n-1)//2)])

for n in range(0, N+1):
    print([FFtt(n, j) for j in range(1, (n-1)//2)])
  
for n in range(0, N+1):
    print("n =", n)
    for j in range(0, (n-1)//2):
        print([F[n][j][t] for t in range(n-1-2*j)])
        #print([F_coeffs[n][j][t] for t in range(n-1-2*j)])
        pass

G[5][0][1] = 5


for n in range(5, N):
    for j in range(0, n//2-1):
        for t in range(1,n//2-j):
            G[n+1][j][t] = 0
            for k in range(j):
                G[n+1][j][t] += 2 * catalan(k) * G[n-2*k-1][j-1-k][t]
            for i in range(j+1):
                G[n+1][j][t] += D[t+1+2*i][i] * D[n-1-2*i-t][j-i]
                if t != Rational(n, 2) - 1 - j:
                    G[n+1][j][t] += D[t+1+2*i][i] * D[n-1-2*i-t][j-i]
            if j != Rational(n,2)-2:
                if t != Rational(n, 2) - 1 - j:
                    G[n+1][j][t] += G[n][j][t]
                if t != 1:
                    G[n+1][j][t] += G[n][j][t-1]
                if t == Rational(n-1, 2) - 1 - j:
                    G[n+1][j][t] += G[n][j][t]
            if t == 1:
                G[n+1][j][t] += (j + 1) * D[n][j+1]

for n in range(5, N+1):
    for j in range(0, (n-1)//2-1):
        Gt[n][j] = (1 - (-1)**n) * sum([(-1)**t * G[n][j][t] for t in range(1, (n-1)//2-j)])
        
for n in range(0, N+1):
    print("n =", n)
    for j in range(0, (n-1)//2 - 1):
        print([G[n][j][t] for t in range((n-1)//2-j)])
        #print([G_coeffs[n][j][t] for t in range((n-1)//2-j)])
        pass
        



    


A_error 0
At_error 0
B_error 0
Bt_error 0
D_error 0
Dt_error 0
E_error 0
(5, 1) 0
(6, 1) 0
(6, 2) 0
(7, 1) 0
(7, 2) 0
(8, 1) 0
(8, 2) 0
(8, 3) 0
(9, 1) 0
(9, 2) 0
(9, 3) 0
(10, 1) 0
(10, 2) 0
(10, 3) 0
(10, 4) 0
(11, 1) 0
(11, 2) 0
(11, 3) 0
(11, 4) 0
(12, 1) 0
(12, 2) 0
(12, 3) 0
(12, 4) 0
(12, 5) 0
(13, 1) 0
(13, 2) 0
(13, 3) 0
(13, 4) 0
(13, 5) 0
(14, 1) 0
(14, 2) 0
(14, 3) 0
(14, 4) 0
(14, 5) 0
(14, 6) 0
(15, 1) 0
(15, 2) 0
(15, 3) 0
(15, 4) 0
(15, 5) 0
(15, 6) 0
(16, 1) 0
(16, 2) 0
(16, 3) 0
(16, 4) 0
(16, 5) 0
(16, 6) 0
(16, 7) 0
(17, 1) 0
(17, 2) 0
(17, 3) 0
(17, 4) 0
(17, 5) 0
(17, 6) 0
(17, 7) 0
(18, 1) 0
(18, 2) 0
(18, 3) 0
(18, 4) 0
(18, 5) 0
(18, 6) 0
(18, 7) 0
(18, 8) 0
(19, 1) 0
(19, 2) 0
(19, 3) 0
(19, 4) 0
(19, 5) 0
(19, 6) 0
(19, 7) 0
(19, 8) 0
[]
[]
[]
[]
[]
[6]
[-4]
[10, 64]
[-8, -44]
[14, 148, 468]
[-12, -120, -328]
[18, 264, 1388, 2900]
[-16, -228, -1136, -2060]
[22, 412, 3012, 10512, 16356]
[-20, -368, -2616, -8664, -11736]
[26, 592, 5532, 26892, 70140, 86800]
[-2

In [ ]:
for n in range(min(10, N+1)):
    error = 0
    for j in range(n//2):
        error += DD(n, j) * (-q)**(j+1) * q_conj**j * higher_deriv(q_conj, n-1-2*j)
    for j in range(n//2):
        error += EE(n, j) * (-q)**j * q_conj**(j+1) * higher_deriv(q, n-1-2*j)
    
    error -= extract_deriv(sigs_eq[n], 1)
    
    if simplify(error) != 0:
        print("sigs error n = ", n)
        display(simplify(error))

print("Check 1 done")

for n in range(min(10, N+1)):
    error = 0
    for j in range((n-1)//2):
        for t in range(1, n-1-2*j):
            error += F[n][j][t] * (-q)**j * q_conj**j * higher_deriv(q, n-1-2*j-t) * higher_deriv(q_conj, t)
    for j in range((n-1)//2-1):
        for t in range(1, (n-1)//2-j):
            error += G[n][j][t] * (-q)**(j+2) * q_conj**j * higher_deriv(q_conj, n-3-2*j-t) * higher_deriv(q_conj, t)
    
    error -= extract_deriv_alt(sigs_eq[n], 2)
    
    if simplify(error) != 0:
        print("sigs error n = ", n)
        display(simplify(error))
        
print("Check 2 done")

for n in range(min(10, N+1)):
    error = 0
    for j in range((n-1)//2-1):
        error += GGt(n, j) * (-q)**(j+2) * q_conj**j * higher_deriv(q_conj, n-3-2*j)
    for j in range((n-1)//2):
        error += FFt(n, j) * (-q)**j * q_conj**j * higher_deriv(q, n-1-2*j)
    
    error -= extract_deriv(variation(extract_deriv_alt(sigs_eq[n], 2), q_conj), 1)
    
    if simplify(error) != 0:
        print("sigs error n = ", n)
        display(simplify(error))

print("Check 3 done")


In [ ]:
for n in range(min(9, N+1)):
    error = 0
    for j in range((n-1)//2-1):
        error += GGt(n, j) * (-q)**(j+2) * q_conj**j * higher_deriv(q_conj, n-3-2*j)
    for j in range((n-1)//2):
        error += FFt(n, j) * (-q)**j * q_conj**j * higher_deriv(q, n-1-2*j)
    for j in range(n//2):
        error += extract_deriv(variation(DD(n, j) * (-q)**(j+1) * q_conj**j * higher_deriv(q_conj, n-1-2*j), q_conj), 1)
    for j in range(n//2):
        error += extract_deriv(variation(EE(n, j) * (-q)**j * q_conj**(j+1) * higher_deriv(q, n-1-2*j), q_conj), 1)
    
    error -= extract_deriv(variation(sigs_eq[n], q_conj), 1)
    
    if simplify(error) != 0:
        print("sigs error n = ", n)
        display(simplify(error))

print("Check 4 done")

for n in range(min(9, N+1)):
    #print("n =", n)
    error = 0
    for j in range((n-1)//2-1):
        error += GGt(n, j) * (-q)**(j+2) * q_conj**j * higher_deriv(q_conj, n-3-2*j)
    for j in range((n-1)//2):
        error += FFt(n, j) * (-q)**j * q_conj**j * higher_deriv(q, n-1-2*j)
    for j in range(n//2-1):
        error += (1-(-1)**n) * (j+1) * DD(n, j+1) * (-q)**(j+2) * q_conj**j * higher_deriv(q_conj, n-3-2*j)
    for j in range(n//2):
        error += (j+1) * (EE(n, j) + (-1)**n * DD(n, j)) * (-q)**j * q_conj**j * higher_deriv(q, n-1-2*j)
    
    #display(error)
    error -= extract_deriv(variation(sigs_eq[n], q_conj), 1)
    
    if simplify(error) != 0:
        print("sigs error n = ", n)
        display(simplify(error))

print("Check 5 done")


In [ ]:
for n in range(min(9, N+1)):
    error = 0
    for j in range(n//2-1):
        error += ((GGt(n, j) if j <= (n-1)//2-2 else 0) + 2 * (n%2) * (j+1) * DD(n, j+1))  * (-q)**(j+2) * q_conj**j * higher_deriv(q_conj, n-3-2*j)
    for j in range(n//2):
        error += ((FFt(n, j) if j <= (n-1)//2-1 else 0) + (j+1) * (EE(n, j) + (-1)**n * DD(n, j))) * (-q)**j * q_conj**j * higher_deriv(q, n-1-2*j)

    #display(error)
    error -= extract_deriv(variation(sigs_eq[n], q_conj), 1)
    
    if simplify(error) != 0:
        print("sigs error n = ", n)
        display(simplify(error))

print("Check 6 done")

def JJ(n, j):
    if n%2 == 0:
        return 0
    m = Rational(n-1, 2)
    return (1 - (-1)**n) * 4**j * binomial(Rational(n,2) - 1, j)

def KK(n, j):
    if n%2 == 0:
        m = Rational(n, 2)
        return - 4**j * binomial(m - Rational(1,2), j)
    elif n%2 == 1:
        m = Rational(n-1, 2)
        return - Rational(1,2) * 4**j * (binomial(m - Rational(1,2), j) + binomial(m + Rational(1,2), j))

for n in range(min(11, N+1)):
    print("n =", n)
    error = 0
    for j in range(n//2-1):
        error += JJ(n, j) * (-q)**(j+2) * q_conj**j * higher_deriv(q_conj, n-3-2*j)
    for j in range(n//2):
        error += KK(n, j) * (-q)**j * q_conj**j * higher_deriv(q, n-1-2*j)

    #display(error)
    error -= extract_deriv(variation(sigs_eq[n], q_conj), 1)
    
    if simplify(error) != 0:
        print("sigs error n = ", n)
        display(simplify(error))

print("Check 7 done")

In [ ]:



def KK(n, j):
    return (FFt(n, j) if j <= (n-1)//2-1 else 0) + (j+1) * (EE(n, j) + (-1)**n * DD(n, j))

def KKo(m, j):
    n = 2 * m + 1
    return (FFt(n, j) if j <= (n-1)//2-1 else 0) + (j+1) * (EE(n, j) + (-1)**n * DD(n, j))

def KKo2(m, j):
    X = 0
    X += - 4**j * binomial(m-Rational(1,2), j)
    X += - 4**j * Rational(1,2) * binomial(m-Rational(1,2), j-1)
    for k in range(j+1):
        X += catalan(k) * 4**(j-k) * (j+1) * binomial(m-1-k, j-k)
        X += - catalan(k) * 4**(j-k) * (j+1) * (-1)**k * binomial(m-Rational(1,2), j-k)
        
    return X

def KKo3(m, j):
    X = 0
    X += - 4**j * binomial(m-Rational(1,2), j)
    X += - 4**j * Rational(1,2) * binomial(m-Rational(1,2), j-1)

    return X

for m in range(16):
    print([KKo(m, j) for j in range(m-1)])
    print([KKo(m, j) - KKo3(m, j) for j in range(m-1)])


def KKe(m, j):
    n = 2 * m
    return (FFt(n, j) if j <= (n-1)//2-1 else 0) + (j+1) * (EE(n, j) + (-1)**n * DD(n, j))

def KKe2(m, j):
    X = 0
    for k in range(j):
        X += catalan(k) * 4**(j-k) * (m-1-j) * binomial(m-1-k-Rational(1,2), j-1-k)
    for l in range(j+1):
        X += - (j+1) * (-1)**l * catalan(l) * 4**(j-l) * binomial(m-1, j-l)
    return X

def KKe3(m, j):
    X = 0
    for k in range(j+1):
        X += 4**j * Rational(catalan(k), 4**k) * ((m-1-j) * binomial(m-1-k-Rational(1,2), j-1-k) - (-1)**k * (j+1) * binomial(m-1, j-k))
    return X

def KKe4(m, j):
    return - 2 * 4**j * sum([binomial(-Rational(3,2), k) * binomial(m, j-k) for k in range(j+1)])

def KKe5(m, j):
    return - 4**j * binomial(m - Rational(1,2), j)

for m in range(16):
    print([KKe(m, j) for j in range(m-1)])
    print([KKe5(m, j) for j in range(m-1)])

    

In [72]:
for n in range(8):
    display(sigs_eq[n])

0

-\tilde{q}*q

-(\tilde{q})_{x}*q - 2*(q)_{x}*\tilde{q}

-(\tilde{q})_{xx}*q - 2*(\tilde{q})_{x}*(q)_{x} - 2*(q)_{xx}*\tilde{q} + \tilde{q}**2*q**2

-(\tilde{q})_{xxx}*q - 4*(\tilde{q})_{xx}*(q)_{x} - 4*(\tilde{q})_{x}*(q)_{xx} + 4*(\tilde{q})_{x}*\tilde{q}*q**2 - 2*(q)_{xxx}*\tilde{q} + 7*(q)_{x}*\tilde{q}**2*q

-(\tilde{q})_{xxxx}*q - 4*(\tilde{q})_{xxx}*(q)_{x} - 8*(\tilde{q})_{xx}*(q)_{xx} + 6*(\tilde{q})_{xx}*\tilde{q}*q**2 + 5*(\tilde{q})_{x}**2*q**2 - 6*(\tilde{q})_{x}*(q)_{xxx} + 26*(\tilde{q})_{x}*(q)_{x}*\tilde{q}*q - 2*(q)_{xxxx}*\tilde{q} + 11*(q)_{xx}*\tilde{q}**2*q + 10*(q)_{x}**2*\tilde{q}**2 - 2*\tilde{q}**3*q**3

-(\tilde{q})_{xxxxx}*q - 6*(\tilde{q})_{xxxx}*(q)_{x} - 12*(\tilde{q})_{xxx}*(q)_{xx} + 8*(\tilde{q})_{xxx}*\tilde{q}*q**2 + 18*(\tilde{q})_{xx}*(\tilde{q})_{x}*q**2 - 14*(\tilde{q})_{xx}*(q)_{xxx} + 56*(\tilde{q})_{xx}*(q)_{x}*\tilde{q}*q + 45*(\tilde{q})_{x}**2*(q)_{x}*q - 8*(\tilde{q})_{x}*(q)_{xxxx} + 60*(\tilde{q})_{x}*(q)_{xx}*\tilde{q}*q + 60*(\tilde{q})_{x}*(q)_{x}**2*\tilde{q} - 16*(\tilde{q})_{x}*\tilde{q}**2*q**3 - 2*(q)_{xxxxx}*\tilde{q} + 15*(q)_{xxx}*\tilde{q}**2*q + 40*(q)_{xx}*(q)_{x}*\tilde{q}**2 - 26*(q)_{x}*\tilde{q}**3*q**2

-(\tilde{q})_{xxxxxx}*q - 6*(\tilde{q})_{xxxxx}*(q)_{x} - 18*(\tilde{q})_{xxxx}*(q)_{xx} + 10*(\tilde{q})_{xxxx}*\tilde{q}*q**2 + 28*(\tilde{q})_{xxx}*(\tilde{q})_{x}*q**2 - 26*(\tilde{q})_{xxx}*(q)_{xxx} + 76*(\tilde{q})_{xxx}*(q)_{x}*\tilde{q}*q + 19*(\tilde{q})_{xx}**2*q**2 + 176*(\tilde{q})_{xx}*(\tilde{q})_{x}*(q)_{x}*q - 22*(\tilde{q})_{xx}*(q)_{xxxx} + 136*(\tilde{q})_{xx}*(q)_{xx}*\tilde{q}*q + 120*(\tilde{q})_{xx}*(q)_{x}**2*\tilde{q} - 30*(\tilde{q})_{xx}*\tilde{q}**2*q**3 + 113*(\tilde{q})_{x}**2*(q)_{xx}*q + 98*(\tilde{q})_{x}**2*(q)_{x}**2 - 50*(\tilde{q})_{x}**2*\tilde{q}*q**3 - 10*(\tilde{q})_{x}*(q)_{xxxxx} + 106*(\tilde{q})_{x}*(q)_{xxx}*\tilde{q}*q + 276*(\tilde{q})_{x}*(q)_{xx}*(q)_{x}*\tilde{q} - 196*(\tilde{q})_{x}*(q)_{x}*\tilde{q}**2*q**2 - 2*(q)_{xxxxxx}*\tilde{q} + 19*(q)_{xxxx}*\tilde{q}**2*q + 62*(q)_{xxx}*(q)_{x}*\tilde{q}**2 + 44*(q)_{xx}**2*\tilde{q}**2 - 52*(q)_{xx}*\tilde{q}**3*q**2 - 94*(q)_{x}**2*\tilde{q}**3*q + 5*\tilde{q}**4*q**4

In [71]:
for n in range(0, 10):
    m = n // 2
    X = 0
    
    if n % 2 == 0:
        for j in range(m):
            X += - 4**j * binomial(m - Rational(1,2), j) * (-q)**j * q_conj**j * higher_deriv(q, 2*m-1-2*j)
    if n % 2 == 1:
        for j in range(m-1): 
            X += 2 * 4**j * binomial(m - Rational(1,2), j) * (-q)**(j+2) * q_conj**j * higher_deriv(q_conj, 2*m-2-2*j)
        for j in range(m):
            X += - Rational(1,2) * 4**j * (binomial(m - Rational(1,2), j) + binomial(m + Rational(1,2), j)) * (-q)**j * q_conj**j * higher_deriv(q, 2*m-2*j)

    X = simplify(X)
    
    Y = simplify(extract_deriv(variation(sigs_eq[n], q_conj), 1))
    
    display(Eq(X, Y, evaluate=True))

for n in range(0, 10):
    m = n // 2
    X = 0
    
    if n % 2 == 0:
        for k in range(m+1):
            X += 4**(m-k) * binomial(m - 1, m - k) * sigs_eq[2*k]
    if n % 2 == 1:
        for k in range(m+1):
            X += 4**(m-k) * binomial(m - Rational(1,2), m - k) * sigs_eq[2*k+1]
        X += catalan(m)
        
    Y = sigq_eq[n]

    print("n =", n)
    display(simplify(X - Y))

print("Check 0 done")

for n in range(0, 10):
    m = n // 2
    X = 0
    
    if n % 2 == 0:
        for k in range(m+1):
            X += 4**(m-k) * binomial(m - 1, m - k) * sigs_eq[2*k]
    if n % 2 == 1:
        for k in range(m+1):
            X += 4**(m-k) * binomial(m - Rational(1,2), m - k) * sigs_eq[2*k+1]
        X += catalan(m)
        
    Y = sigq_eq[n]

    print("n =", n)
    display(simplify(X - Y))

print("Check 1 done")



n = 0


0

n = 1


0

n = 2


...

n = 8


0

n = 9


0

Check 1 done


In [ ]:
for n in range(-1, 11):
    display(variation(I**n * sigs_eq[n+1], q_conj))

In [ ]:

for n in range(0, 8):
    m = n // 2
    X = 0
    if n % 2 == 1:
        X += binomial(2*m, m) * q_conj**m * (- q)**(m+1)
    
    Y = extract_deriv(variation(sigs_eq[n], q_conj), 0)
    display(Eq(X, Y))
    
print("Check 1 done")

for n in range(0, 8):
    m = n // 2
    X = 0
    
    if n % 2 == 0:
        for j in range((n-1)//2):
            for k in range(j): 
                X += catalan(k) * 4**(j-k) * (m-1-j) * binomial(m-1-k-Rational(1,2), j-1-k) * (-q)**j * q_conj**j * higher_deriv(q, 2*m-1-2*j)
        
    if n % 2 == 1:
        for j in range((n-1)//2):
            C = 4**j * (2*j+1) * binomial(m-Rational(1,2), j) - 4**j * Rational(1,2) * binomial(m-Rational(1,2), j-1)
            for k in range(j+1):
                C += catalan(k) * 4**(j-k) * (j+1) * binomial(m-1-k, j-k)
            X += C * (-q)**j * q_conj**j * higher_deriv(q, n-1-2*j)

        
        for j in range((n-1)//2-1):
            X += - (8*m-8*j-6) * 4**j * binomial(m-Rational(1,2), j) * (-q)**(j+2) * q_conj**j * higher_deriv(q_conj, n-3-2*j)
            
            
    X = simplify(X)
    
    Y = simplify(extract_deriv(variation(extract_deriv_alt(sigs_eq[n], 2), q_conj), 1))
    
    display(Eq(X - Y))
    
print("Check 2 done")

for n in range(0, 10):
    m = n // 2
    X = 0
    
    if n % 2 == 0:
        for j in range(m):
            X += - 4**j * binomial(m - Rational(1,2), j) * (-q)**j * q_conj**j * higher_deriv(q, 2*m-1-2*j)
    if n % 2 == 1:
        for j in range(m-1): 
            X += 2 * 4**j * binomial(m - Rational(1,2), j) * (-q)**(j+2) * q_conj**j * higher_deriv(q_conj, 2*m-2-2*j)
        for j in range(m):
            X += - Rational(1,2) * 4**j * (binomial(m - Rational(1,2), j) + binomial(m + Rational(1,2), j)) * (-q)**j * q_conj**j * higher_deriv(q, 2*m-2*j)

    X = simplify(X)
    
    Y = simplify(extract_deriv(variation(sigs_eq[n], q_conj), 1))
    
    display(Eq(X, Y, evaluate=True))
    
print("Check 3 done")

    
for n in range(10):
    X = extract_deriv_alt(deriv(extract_deriv_alt(sigs_eq[n], 2)), 2)
    #print("A", simplify(X))
    X += extract_deriv_alt(deriv(extract_deriv(sigs_eq[n], 1)), 2)
    #print("B", simplify(X))
    for k in range(1, n):
        X += 2 * extract_deriv_alt(sigs_eq[k], 2) * extract_deriv(sigs_eq[n-k], 0)
        #print("C", k, simplify(X))
        X += extract_deriv_alt(extract_deriv(sigs_eq[k], 1) * extract_deriv(sigs_eq[n-k], 1), 2)
        #print("D", k, simplify(X))
    X += - deriv(q) * extract_deriv_alt(Rq_eq[n], 1)
    
    Y = deriv(extract_deriv_alt(sigs_eq[n], 2))
    Y += deriv(extract_deriv(sigs_eq[n], 1))

    for k in range(1, n):
        Y += 2 * extract_deriv_alt(sigs_eq[k], 2) * extract_deriv(sigs_eq[n-k], 0)
        #display("C", k, X)
        Y += extract_deriv(sigs_eq[k], 1) * extract_deriv(sigs_eq[n-k], 1)
        
    Y += - deriv(q) * extract_deriv_alt(Rq_eq[n], 1)
    
    #display(Eq(sigs_sym[n+1], sigs_eq[n+1]))
    #display(Eq(sigs_sym[n+1], simplify(X)))
    #display(Eq(sigs_sym[n+1], extract_deriv_alt(simplify(Y), 2)))
    #display(Eq(sigs_sym[n+1], extract_deriv_alt(sigs_eq[n+1], 2)))


In [ ]:


num = 20

for n in range(num):
    L = [0 for j in range(n//2)]
    for j in range(n//2):
        for k in range(j):
            for t in range(1, n-2*j):
                L[j] += (-1)**t * D[t+1+2*k][k] * E[n-1-2*k-t][j-1-k]
    print(L)
    LL = [0 for j in range(n//2)]
    for j in range(n//2):
        for k in range(j):
            for t in range(1, n-2*j):
                LL[j] += - 4**(j-1) * (-1)**t * binomial(Rational(t+1,2)+k-1,k) * binomial(Rational(n-t-1,2)-k-1,j-1-k)
        for k in range(j):
            for t in range(1, n-2*j):
                for l in range(j-k):
                    LL[j] += - 4**(j-1-l) * (-1)**t * binomial(Rational(t+1,2)+k-1,k) * (-1)**l * catalan(l) * binomial(Rational(n-t-1,2)-k-1,j-1-k-l)
    print(LL)
    LLL = [0 for j in range(n//2)]
    for j in range(n//2):
        for k in range(j):
            for t in range(1, n-2*j):
                LLL[j] += - 4**(j-1) * (-1)**t * (-1)**k * binomial(-Rational(t+1,2),k) * (-1)**(j-1-k) * binomial(-Rational(n-t-1,2)+j-1,j-1-k)
        for k in range(j):
            for t in range(1, n-2*j):
                for l in range(j-k):
                    LLL[j] += - 4**(j-1-l) * (-1)**t * (-1)**k * binomial(-Rational(t+1,2),k) * catalan(l) * (-1)**(j-1-k) * binomial(-Rational(n-t-1,2)+j-1-l,j-1-k-l)
    print(LLL)
    print([4**(j-1)*binomial(Rational(n,2)-1,j-1) + 4**(j-1) * sum([catalan(l)/(-4)**l * binomial(Rational(n,2)-1,j-1-l) for l in range(j)]) if n%2==0 else 0 for j in range(n//2)])

print("-----")

for m in range(-num, num):
    L = [0 for j in range(-num, num)]
    for j in range(0, num):
        X = 0
        for k in range(j):
            X += 2 * catalan(k) * ((8*(m-j)-6) * Rational(1,4**(k+1)) * binomial(m-k-Rational(3,2), j-1-k))
        X += 2 * (-1)**j * binomial(j-m, j)
        X += 8 * (j + 1) * binomial(m-1, j+1)
        
        X += - (8*(m-j)-6) * binomial(m-Rational(1,2), j)
        
        L[n] = X
        
    print(L)
         
print("-----")

num = 30
for m in range(num):
    L = [0 for j in range(num)]
    for j in range(num):
        X = 0
        for k in range(j):
            X += 2 * (8 * (m-j) - 6) * Rational(1,4**(k+1)) * binomial(m-k-Rational(3,2), j-k-1) * catalan(k)
        X += (2 * (-1)**j * binomial(j - m, j)) if j <= m - 2 else 0
        X += 2 * (j+1) * 4**(j+1) * binomial(m-1, j+1)
        L[j] = X
    print("m =", m)
    print([L[j] - (8 * (m-j) - 6) * binomial(m-Rational(1,2), j) for j in range(num)])
    
print("-----")


In [ ]:
for n in range(1, 14, 2):
    print([binomial(Rational(n,2)-1, j-1) for j in range(10)])
print("---")
for n in range(1, 20, 2):
    print([4**j * (binomial(Rational(n,2)-1, j) - Rational(1,2) * binomial(Rational(n,2), j-1)) for j in range((n-1)//2)])

In [ ]:
def get_expansion(eqn, x0, var, num):
    ser = series(eqn, x0=0, x=var, n=num).removeO()
    return [simplify(ser.coeff(var**n) if n > 0 else ser.subs(var, 0)) for n in range(0, num)]

def coeff(n, j):
    return sum([4*(j//2-1) * catalan(k) * 4**(n-k) * binomial(Rational(j,2)-1+n-k, n-k) for k in range(n+1)])

def F_even_coeff(n, j):
    return sum([4*(n//2-j-1) * catalan(k) * 4**(j-1-k) * binomial(Rational(n-1,2)-1-k, j-1-k) for k in range(j)])

def F_odd_coeff(n, j):
    Y = 0
    Y += - 4**j * Rational(1,2) * binomial(Rational(n,2)-1, j-1)
    Y += 4**j * (2*j+1) * binomial(Rational(n,2)-1, j)
    Y += sum([(j+1) * catalan(k) * 4**(j-k) * binomial(Rational(n-1,2)-1-k, j-k) for k in range(j+1)])
    return Y 

def G_even_coeff(n, j):
    return 0
    #return 2 * 4**j * (-1)**(n//2-j) * binomial((n-1)//2, j) * (((n//2-j)%2) * (4*(n//2-j)-5) - 2*(n//2-j-1) + 2*binomial(n-2*j-3,n//2-j-1))

def G_odd_coeff(n, j):
    return - 2 * (4*((n-1)//2-j)-3) * 4**j * binomial(Rational(n,2)-1, j)

def Gt_coeff(n, j):
    if n%2 == 0:
        return 0
    else:
        return - 2 * (4*((n-1)//2-j)-3) * 4**j * binomial(Rational(n,2)-1, j)

print("---")
N = 60
print("===")
for n in range(3, N-1, 2):
    print([Ft[n][j] - F_odd_coeff(n, j) for j in range(0, (n-1)//2)])
for n in range(4, N+1, 2):
    print([Ft[n][j] - F_even_coeff(n, j) for j in range(0, (n-1)//2)])
print("###")
for n in range(3, N-1, 2):
    print([F_odd_coeff(n, j) for j in range(0, (n-1)//2)])
for n in range(4, N+1, 2):
    print([F_even_coeff(n, j) for j in range(0, (n-1)//2)])
    
print("---")

for m in range(1, N//2):
    L = [0 for j in range(m)]
    T = [0 for j in range(m)]

    for j in range(0, m):
        X = 0
        
        for k in range(j):
            X += 2 * catalan(k) * Rational(1,4**j) * Ft[2*m-2*k-1][j-1-k]
        
        X += Rational(1,2) * binomial(m-1, j-1)
        X += (2*j+1) * binomial(m-1, j)
        X += sum([(-1)**k * catalan(k) * Rational(1,4**k) * Rational(1,2) * binomial(m-1, j-1-k) for k in range(j)])
        X += sum([(-1)**k * catalan(k) * Rational(1,4**k) * (j+1) * binomial(m-1, j-k) for k in range(j+1)])

        Y = Rational(1,4**j) * Ft[2*m+1][j]
        L[j] = X - Y
        
    print(L)

print("---")

for m in range(1, N//2):
    L = [0 for j in range(m)]
    for j in range(0, m):
        X = 0
        for k in range(j):
            X += 2 * catalan(k) * Rational(1,4**j) * Ft[2*m-2*k][j-1-k]
        
        X += - binomial(m-Rational(1,2), j)
        X += (j+1) * sum([(-1)**l * catalan(l) * Rational(1,4**l) * binomial(m-Rational(1,2), j-l) for l in range(j+1)])
        
        Y = Rational(1,4**j) * Ft[2*m+2][j]
        #Y = Rational(1,4**j) * sum([(m-j) * catalan(k) * 4**(j-k) * binomial(m-k-Rational(1,2), j-1-k) for k in range(j)])
        L[j] = X - Y
    print(L)
        
print("---")

for n in range(3, N):
    L = [0 for j in range(n//2)]
    for j in range(0, n//2):
        X = 0
        for k in range(j):
            X += 2 * catalan(k) * Ft[n-2*k-1][j-1-k]
        
        X += ((n+1)%2) * 2 * 4**(j-1) * binomial(Rational(n,2)-1, j-1)
        X += ((n+1)%2) * 2 * 4**(j-1) * sum([catalan(l) / (-4)**l * binomial(Rational(n,2)-1, j-1-l) for l in range(j)])
        
        X += (j + (-1)**n * (j+1)) * 4**j * binomial(Rational(n,2)-1, j)
        X += (j+1) * sum([(-1)**l * catalan(l) * 4**(j-l) * binomial(Rational(n,2)-1, j-l) for l in range(j+1)])
        
        Y = Ft[n+1][j]
        L[j] = X - Y
    print(L)
        
print("---")

for n in range(5, N+1, 2):
    print([Gt[n][j] for j in range(0, (n-1)//2-1)])
for n in range(5, N+1, 2):
    print([G_odd_coeff(n, j) for j in range((n-1)//2-1)])
for n in range(6, N+1, 2):
    print([Gt[n][j] for j in range(0, (n-1)//2-1)])
for n in range(6, N+1, 2):
    print([G_even_coeff(n, j) for j in range((n-1)//2-1)])
    
print("---")

for n in range(5, N):
    for j in range(0, n//2-1):
        X = 0
        for k in range(j):
            X += 2 * ((n+1)%2) * catalan(k) * Gt[n-2*k-1][j-1-k]
        X += - 2 * ((n+1)%2) * (-4)**j * binomial(j-Rational(n,2), j)
        X += - 2 * ((n+1)%2) * (j + 1) * 4**(j+1) * binomial(Rational(n,2)-1, j+1)
        
        Y = Gt[n+1][j]
        print(X - Gt_coeff(n+1, j))
        #print(X - Y)
         
print("---")

for n in range(5, N):
    for j in range(0, n//2-1):
        X = 0
        for k in range(j):
            X += 4 * catalan(k) * Gt[n-2*k-1][j-1-k]
        for t in range(1, n//2-j):
            for i in range(j+1):
                X += 2 * (-1)**t * D[t+1+2*i][i] * D[n-1-2*i-t][j-i]
                if t != Rational(n,2) - 1 - j:
                    X += 2 * (-1)**t * D[t+1+2*i][i] * D[n-1-2*i-t][j-i]
        X += - 2 * (j + 1) * D[n][j+1]
        if n%2 == 1:
            X = 0
            
        Y = Gt[n+1][j]
        #print(X, Y)

x = Symbol('x')

for j in range(1, 15, 2):
    print(get_expansion((1 / (1 - 4*x)**Rational(j,2) - 1) / (2*x), 0, x, 8))

for j in range(5, 14, 2):
    print(get_expansion(4*(j//2-1) * ((1 - sqrt(1 - 4*x))/(2*x)) / (1 - 4*x)**Rational(j,2), 0, x, 12))
print("---")


            

In [ ]:
def generate_sequence(limit_n, limit_r, p):
    # Initialize the array with zeros
    a = [[0 for _ in range(limit_r + 1)] for _ in range(limit_n + 1)]

    # Base case
    for n in range(limit_n + 1):
        a[n][0] = 1
    # Populate the array using the formula
    for n in range(1, limit_n + 1):
        for r in range(1, limit_r + 1):
            if r > (p - 1) * n:
                a[n][r] = 0
            else:
                a[n][r] = a[n][r - 1] + a[n - 1][r]

    # Flatten the array to get the sequence
    sequence = [[a[n][r] for r in range(limit_r + 1)] for n in range(limit_n + 1)]
    return sequence

def curly(n, r, m):
    return binomial(n+r, r) - sum([binomial(m*k,k)*binomial(n+r-m*k-1,n-k)/((m-1)*k+1) for k in range(ceiling(Rational(r,m-1)))])

def curly2(n, r, m):
    if 0 <= r and r < m:
        return binomial(n-1 + r, r)
    if m <= r and r <= n-1 + m - 1:
        return binomial(n-1 + r, r) - binomial(n-1 + r, r - m)
    if r > n-1 + m - 1:
        return 0

def curly3(n, r):
    return binomial(n - 1 + r, r) - 1

for n in range(5, N):
    j = 0
    #print([G_coeffs[n][j][t] for t in range(1, n//2-j-1)])

t = 1
for n in range(2*t, N+1):
    print([G_coeffs[n][j][t] for j in range(0, (n-1)//2-t)])
t = 2
for n in range(2*t, N+1):
    print([G_coeffs[n][j][t] for j in range(0, (n-1)//2-t)])
t = 3
for n in range(2*t, N+1):
    print([G_coeffs[n][j][t] for j in range(0, (n-1)//2-t)])
t = 4
for n in range(2*t, N+1):
    print([G_coeffs[n][j][t] / 2 for j in range(0, (n-1)//2-t)])
t = 5
for n in range(2*t, N+1):
    print([G_coeffs[n][j][t] / 2 for j in range(0, (n-1)//2-t)])

# Compare the sequences
for n in range(5, N):
    j = 0
    print([G_coeffs[n][j][t] for t in range(1, n//2-j-1)])
    print([2*curly3(n-(t+1), t+1) for t in range(1, n//2-j-1)])
    
for n in range(0, 16):
    print([curly3(n, r, 2) for r in range(0, 12)])
for n in range(0, 16):
    print([curly3(n, r, 3) for r in range(0, 12)])
for n in range(0, 16):
    print([curly3(n, r, 4) for r in range(0, 12)])

    

In [ ]:
def get_expansion(eqn, x0, var, num):
    ser = series(eqn, x0=0, x=var, n=num).removeO()
    return [simplify(ser.coeff(var**n) if n > 0 else ser.subs(var, 0)) for n in range(0, num)]

def coeff(n, k):
    return sum([(-2)**(n+2) * binomial(n+k+4, j) * Rational(1,(-2)**j) for j in range(0, n+2)])

for n in range(14):
    print([coeff(n, m) for m in range(14)])
    
print("---")

x = Symbol('x')
for k in range(0, 12):
    print(get_expansion(2 / (1 + x) * 1 / (1 - x)**k, 0, x, 14))
print("---")
for k in range(0, 12):
    print(get_expansion(2 * 1 / ((1 + x)**4 * (1 - x)**k), 0, x, 14))
print("---")

In [33]:
N = 8

F_coeffs = [[[0 for t in range(0,n-1-2*j)] for j in range(0, (n-1)//2)] for n in range(N+1)]
G_coeffs = [[[0 for t in range(0,(n-1)//2-j)] for j in range(0, (n-1)//2 - 1)] for n in range(N+1)]

for n in range(N+1):
    #display(extract_deriv_alt(sigs_eq[n], 2))
    #print(F_coeffs[n])
    #print(G_coeffs[n])
    pass

for n in range(0, N+1):
    
    expr = extract_deriv_alt(sigs_eq[n], 2)
    #display(expr)
    
    if isinstance(expr, Add):
        for mon in expr.args:
            coefficient = 0
            #print("mon =", mon)
            count_q_conj = 0
            t = 0
            j = 0
            for fac in mon.args:
                if isinstance(fac, Pow):
                    var_name = get_var_name_from_deriv(fac.base)
                    order = get_order_from_deriv(fac.base)
                    if var_name == q_conj.name:
                        if order == 0:
                            j = fac.exp
                        if order > 0:
                            count_q_conj += fac.exp
                            t = order
                            
                elif isinstance(fac, Symbol):
                    var_name = get_var_name_from_deriv(fac)
                    order = get_order_from_deriv(fac)
                    if var_name == q_conj.name: 
                        if order == 0:
                            j = 1
                        if order > 0:
                            count_q_conj += 1
                            t = order
                            
                elif isinstance(fac, Number):
                    coefficient = fac
                    
            #print("type =", count_q_conj, "j =", j, "t =", t, "coeff =", coefficient)
            
            if count_q_conj == 1:
                F_coeffs[n][j][t] += (-1)**j * coefficient
            if count_q_conj == 2:
                if t >= (n-1)//2-j:
                    t = n - 3 - 2 * j - t
                print("n =", n, "j =", j, "t =", t)
                G_coeffs[n][j][t] += (-1)**j * coefficient

    if isinstance(expr, Mul):
        mon = expr
        coefficient = 0
        #print("mon =", mon)
        count_q_conj = 0
        t = 0
        j = 0
        for fac in mon.args:
            if isinstance(fac, Pow):
                var_name = get_var_name_from_deriv(fac.base)
                order = get_order_from_deriv(fac.base)
                if var_name == q_conj.name:
                    if order == 0:
                        j = fac.exp
                    if order > 0:
                        count_q_conj += fac.exp
                        t = order

            elif isinstance(fac, Symbol):
                var_name = get_var_name_from_deriv(fac)
                order = get_order_from_deriv(fac)
                if var_name == q_conj.name: 
                    if order == 0:
                        j = 1
                    if order > 0:
                        count_q_conj += 1
                        t = order

            elif isinstance(fac, Number):
                coefficient = fac

        #print("type =", count_q_conj, "j =", j, "t =", t, "coeff =", coefficient)

        if count_q_conj == 1:
            F_coeffs[n][j][t] += (-1)**j * coefficient
        if count_q_conj == 2:
            if t >= (n-1)//2-j:
                t = n - 3 - 2 * j - t
            print("n =", n, "j =", j, "t =", t)
            G_coeffs[n][j][t] += (-1)**j * coefficient
            
    #display(F_coeffs[n])
    #display(G_coeffs[n])
    
print("----")
first = 0
for n in range(0, N+1):
    for j in range(0, (n-1)//2):
        for t in range(1, n-1-2*j):
            #print(n, j, t, F_coeffs[n][j][t])
            first += 1
            pass
print("----")
for t in range(1, N):
    for n in range(t+2, N+1):
        for j in range(0, (n-t)//2):
            #print(n, j, t, F_coeffs[n][j][t])
            pass
print("----")
third = 0
for t in range(1, N):
    for j in range(0, (N-t)//2):
        for n in range(t+2+2*j, N+1):
            #print(n, j, t, F_coeffs[n][j][t])
            third += 1
            pass
        
#print(third - first)

for n in range(0, N+1):
    #print([[F_coeffs[n][j][t] for t in range(1, n-1-2*j)] for j in range(0, (n-1)//2)])
    pass
print("----")
for t in range(1, N):
    #print([[F_coeffs[n][j][t] for j in range(0, (n-t)//2)] for n in range(t+2, N+1)])
    pass
print("----")

def delta_list(A, k):
    if k > 0:
        return delta_list([A[n] - A[n-1] for n in range(1, len(A))], k-1)
    if k == 0:
        return A

#for m in range(10):
    #print([2**n * binomial(-n, m) for n in range((14-m)//2)])

triangle = [[[0 for i in range(j+t+1)] for j in range((N-t)//2)] for t in range(0, N)]

for t in range(1, N):
    for j in range(0, (N-t)//2):
        #print("t =", t, "j =", j)
        for i in range(j+t+1):
            deltas = delta_list([F_coeffs[n][j][t] for n in range(1+t+2+2*j, N+1)], i)
            if len(deltas) > 0:
                #print(deltas)
                triangle[t][j][i] = deltas[0]
            else:
                #print("#")
                break

def coeff(t, j, i):
    if j == 0 and i == 0:
        return sum([2 * k//2 for k in range(1, t+1)])
    else:
        return 0

    
triangle = [[0 for i in range(t+2)] for t in range(0, N)]

for t in range(1, N//2):
    for j in range(0, (N-t)//2):
        #print([F_coeffs[n][j][t] for n in range(1+t+2+2*j, N+1)])
        for i in range(j+t+1):
            if j == 1:
                deltas = delta_list([F_coeffs[n][j][t] / 2 for n in range(1+t+2+2*j, N+1)], i)
                print(deltas)
                if len(deltas) > 0:
                    triangle[t][i] = deltas[0]
        #print("-----")
        #print(delta_list(triangle[t][j][::-1], 0))
        #print(delta_list(triangle[t][j][::-1], 1))
        pass
                                
for t in range(0, N):
    #print([triangle[t][i] for i in range(t+2)][::-1])
    pass

for i in range(0, N+2):
    for d in range(0, i+2):
        print(delta_list([triangle[t][t+1-i] for t in range(i-1, N)], d))
    

n = 5 j = 0 t = 1
n = 6 j = 0 t = 1
n = 7 j = 0 t = 2
n = 7 j = 1 t = 1
n = 7 j = 0 t = 1
n = 8 j = 0 t = 1
n = 8 j = 0 t = 2
n = 8 j = 1 t = 1
----
----
----
----
----
[-30, -53, -82]
[-23, -29]
[-6]
[-68, -137]
[-69]
[]
[]
[-120]
[]
[]
[]
[]
[0, 0, -6, 0, 0, 0, 0, 0, 0]
[0, -6, 6, 0, 0, 0, 0, 0]
[0, -23, 0, 0, 0, 0, 0, 0]
[-23, 23, 0, 0, 0, 0, 0]
[46, -23, 0, 0, 0, 0]
[-30, -69, 0, 0, 0, 0, 0]
[-39, 69, 0, 0, 0, 0]
[108, -69, 0, 0, 0]
[-177, 69, 0, 0]
[-68, 0, 0, 0, 0, 0]
[68, 0, 0, 0, 0]
[-68, 0, 0, 0]
[68, 0, 0]
[-68, 0]
[-120, 0, 0, 0, 0]
[120, 0, 0, 0]
[-120, 0, 0]
[120, 0]
[-120]
[]
[0, 0, 0, 0]
[0, 0, 0]
[0, 0]
[0]
[]
[]
[]
[0, 0, 0]
[0, 0]
[0]
[]
[]
[]
[]
[]
[0, 0]
[0]
[]
[]
[]
[]
[]
[]
[]
[0]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [ ]:
x = Symbol('x')
F_polys = [[(t, j) for j in range(0, (N-t)//2)] for t in range(0, N)]
for t in range(1, N):
    for j in range(0, (N-t)//2):
        X = list(range(t+2+2*j, N+1))
        Y = [F_coeffs[n][j][t] for n in X]
        F_polys[t][j] = simplify(polys.specialpolys.interpolating_poly(len(X), x, X=X, Y=Y))
        if Poly(F_polys[t][j], x).degree()+1 >= len(X):
            F_polys[t][j] = Symbol("\lambda")
    
display(A_polys[0])
display(A_polys[1])
display(A_polys[2])
display(A_polys[3])
display(A_polys[4])
display(A_polys[5])

display(Eq(Symbol("F_{" + str((1,0)) + "}"), F_polys[1][0] + A_polys[1]))
display(Eq(Symbol("F_{" + str((1,1)) + "}"), F_polys[1][1] - 3 * A_polys[2] - 2 * A_polys[1] + 4 * A_polys[0]))
display(Eq(Symbol("F_{" + str((1,2)) + "}"), F_polys[1][2] + 6 * A_polys[3]))

for t in range(1, N):
    for j in range(0, (N-t)//2):
        display(Eq(Symbol("F_{" + str((t,j)) + "}"), F_polys[t][j]))



In [ ]:
expr = sigs_eq[7] + q * q_conj + q**3 * q_conj**7
display(expr)
#display(extract_deriv(expr, 0))
#display(extract_deriv(expr, 1))
#display(extract_deriv(expr, 2))
display(extract_deriv_alt(expr, 2))
display(expr - extract_deriv(expr, 0) - extract_deriv(expr, 1) - extract_deriv(expr, 2))
display(expr - extract_deriv(expr, 0) - extract_deriv(expr, 1) - extract_deriv_alt(expr, 2))

In [ ]:
x = Symbol('x') 
A_polys = [0 for n in range(0, N//2)] 
B_polys = [0 for n in range(0, N//2)]

for k in range(0, N//2):
    X = list(range(2*k+2, N+1)) 
    #print("X =", X) 
    Y = [A_coeffs[n][k] for n in X] 
    #print("Y =", Y) 
    if len(X) >= k:
        #print("For A, k =", k, "ok")
        pass
    else: 
        #print("For A, k =", k, "not ok")
        pass
    A_polys[k] = simplify(polys.specialpolys.interpolating_poly(len(X), x, X=X, Y=Y))
    
for k in range(0, N//2):
    X = list(range(2*k+2, N+1)) 
    #print("X =", X) 
    Y = [B_coeffs[n][k] for n in X] 
    #print("Y =", Y) 
    if len(X) >= k:
        #print("For B, k =", k, "ok")
        pass
    else: 
        #print("For B, k =", k, "not ok")
        pass
    B_polys[k] = simplify(polys.specialpolys.interpolating_poly(len(X), x, X=X, Y=Y))
    
for k in range(N//2):
    display(A_polys[k])
    
for k in range(N//2):
    display(B_polys[k])

display(B_polys[0] + 2 * A_polys[0])
display(B_polys[1] + 2 * A_polys[1] - A_polys[0])
display(B_polys[2] + 2 * A_polys[2] - A_polys[1] + 2 * A_polys[0])
display(B_polys[3] + 2 * A_polys[3] - A_polys[2] + 2 * A_polys[1] - 5 * A_polys[0])
display(B_polys[4] + 2 * A_polys[4] - A_polys[3] + 2 * A_polys[2] - 5 * A_polys[1] + 14 * A_polys[0])
display(B_polys[5] + 2 * A_polys[5] - A_polys[4] + 2 * A_polys[3] - 5 * A_polys[2] + 14 * A_polys[1] - 42 * A_polys[0])


In [ ]:
N = 16

for k in range(0,N//2):
    print([A_coeffs[n+1][k] for n in range(2*k+2, N-1)])
    print([B_coeffs[n][k] for n in range(2*k+4, N+1)])

print("test:")
for k in range(0,N//2):
    
    L = [A_coeffs[n][k] for n in range(2*k+3, N-1)]
    for n in range(2*k+3, N-1):
        L[n-(2*k+3)] -= B_coeffs[n+1][k]
        for j in range(0, k):
            L[n-(2*k+3)] -= binomial(2*j, j-1) * (-1)**(j+1) * B_coeffs[n+1][k-1-j]
    print(L)
    
print("test 2:")
for k in range(0,N//2):
    
    L = [(-1)**k * B_coeffs[n+1][k] for n in range(2*k+3, N)]
    for n in range(2*k+3, N):
        for j in range(0, k+1):
            #L[n-(2*k+3)] += (catalan(j+1) - 2 * catalan(j)) * (-1)**(k-j) * A_coeffs[n][k-j]
            L[n-(2*k+3)] += (catalan(j+1) - 2 * catalan(j)) * (-1)**(n+1) * (-4)**(k-j) * binomial(Rational(n,2) - 1, k-j)
            #L[n-(2*k+3)] += 4 / (j+3) * binomial(2*j+1,j-1) * A_coeffs[n][k-1-j]

    print(L)


In [ ]:
N = 16

for k in range(0,N//2):
    print([A_coeffs[n+1][k] for n in range(2*k+2, N-1)])
    print([B_coeffs[n][k] for n in range(2*k+4, N+1)])

print("test:")
for k in range(0,N//2):
    
    L = [A_coeffs[n][k] for n in range(2*k+3, N-1)]
    for n in range(2*k+3, N-1):
        L[n-(2*k+3)] -= B_coeffs[n+1][k]
        for j in range(0, k):
            L[n-(2*k+3)] -= binomial(2*j, j-1) * (-1)**(j+1) * B_coeffs[n+1][k-1-j]
    print(L)
    
print("test 2:")
for k in range(0,N//2):
    
    L = [(-1)**k * B_coeffs[n+1][k] for n in range(2*k+3, N)]
    for n in range(2*k+3, N):
        for j in range(0, k+1):
            #L[n-(2*k+3)] += (catalan(j+1) - 2 * catalan(j)) * (-1)**(k-j) * A_coeffs[n][k-j]
            L[n-(2*k+3)] += (catalan(j+1) - 2 * catalan(j)) * (-1)**(n+1) * (-4)**(k-j) * binomial(Rational(n,2) - 1, k-j)
            #L[n-(2*k+3)] += 4 / (j+3) * binomial(2*j+1,j-1) * A_coeffs[n][k-1-j]

    print(L)


In [ ]:
x = Symbol('x') 
A_polys = [0 for n in range(0, N//2)] 
B_polys = [0 for n in range(0, (N-1)//2)]

for k in range(0, N//2):
    X = list(range(2*(k+1), N+1)) 
    #print("X =", X) 
    Y = [(-1)**n * A_coeffs[n][k] for n in X] 
    #print("Y =", Y) 
    if len(X) >= k:
        print("For A, k =", k, "ok")
    else: 
        print("For A, k =", k, "not ok")
    A_polys[k] = simplify(polys.specialpolys.interpolating_poly(len(X), x, X=X, Y=Y))
    
    #display(A_polys[k])
    
for k in range(0, (N-1)//2):
    X = list(range(2*(k+2), N+1)) 
    #print("X =", X) 
    Y = [(-1)**n * B_coeffs[n][k] for n in X] 
    #print("Y =", Y) 
    if len(X) >= k:
        print("For B, k =", k, "ok")
    else:
        print("For B, k =", k, "not ok")
    B_polys[k] = simplify(polys.specialpolys.interpolating_poly(len(X), x, X=X, Y=Y))
    
    #display(B_polys[k])
    
display(simplify(- B_polys[0].subs(x, x+1)))
display(simplify(A_polys[0]))

display(simplify(- B_polys[1].subs(x, x+1)))
display(simplify(A_polys[1]))

display(simplify(- B_polys[2].subs(x, x+1)))
display(simplify(A_polys[2] - A_polys[0]))

display(simplify(- B_polys[3].subs(x, x+1)))
display(simplify(A_polys[3] - A_polys[1] - 4 * A_polys[0]))

display(simplify(- B_polys[4].subs(x, x+1)))
display(simplify(A_polys[4] - A_polys[2] - 4 * A_polys[1] - 14 * A_polys[0]))

display(simplify(- B_polys[5].subs(x, x+1)))
display(simplify(A_polys[5] - A_polys[3] - 4 * A_polys[2] - 14 * A_polys[1] - 48 * A_polys[0]))


display(- A_polys[0])
display(simplify(B_polys[0].subs(x, x+1)))

display(- A_polys[1])
display(simplify(B_polys[1].subs(x, x+1)))

display(- A_polys[2])
display(simplify(B_polys[2].subs(x, x+1) + B_polys[0].subs(x, x+1)))

display(- A_polys[3])
display(simplify(B_polys[3].subs(x, x+1) + B_polys[1].subs(x, x+1) + 4 * B_polys[0].subs(x, x+1)))

display(- A_polys[4])
display(simplify(B_polys[4].subs(x, x+1) + B_polys[2].subs(x, x+1) + 4 * B_polys[1].subs(x, x+1) + 15 * B_polys[0].subs(x, x+1)))

display(- A_polys[5])
display(simplify(B_polys[5].subs(x, x+1) + B_polys[3].subs(x, x+1) + 4 * B_polys[2].subs(x, x+1) + 15 * B_polys[1].subs(x, x+1) + 56 * B_polys[0].subs(x, x+1)))




In [ ]:
n = 5
print([binomial(n - Rational(1,2), n - m) * (-4)**(n-m) for m in range(-3, 8)])

def get_expansion(eqn, x0, var, num):
    epsilonpos = Symbol('epsilon_2', positive=True)
    epsiloninvpos = Symbol('eta_2', positive=True)
    
    eqn = eqn.subs(epsilon, epsilonpos).subs(epsiloninv, epsiloninvpos)
    ser = series(eqn, x0=0, x=var, n=num).removeO().subs(epsilonpos, epsilon).subs(epsiloninvpos, epsiloninv)
    return [simplify(ser.coeff(var**n) if n > 0 else ser.subs(var, 0)) for n in range(0, num)]

    
x = Symbol('x')

N = 10
for n in range(1, N+1):
    #display(Eq(rq_sym[n], rq_eq[n]))
    #display(Eq(Rq_sym[n], Rq_eq[n]))
    
    nn = n // 2

    #coefficients = get_expansion((1 - 4*x)**Rational(n-2,2), 0, x, n+1)

    coefficients = [binomial(Rational(n-2,2), m) * (-4)**(m) for m in range(0, n+1)]
    
    Y = 0
    
    if n % 2 == 0:
        for m in range(1, nn+1):
            Y += coefficients[nn-m] * sigq_eq[2*m]
      
    if n % 2 == 1:
        Y += (-1)**(nn+1) * catalan(nn)
        for m in range(0, nn+1):
            Y += coefficients[nn-m] * sigq_eq[2*m+1]
        
    #display(Eq(Rq_sym[n], simplify(Rq_eq[n])))
    
    #display(Eq(SS_sym[n], simplify(deriv(variation(SS_eq[n], u)))))
    
    #display(Eq(sigs_sym[n], sigs_eq[n]))
    #display(Eq(Rq_sym[n], extract_single_deriv(simplify(Rq_eq[n]))))

    display(Eq(sigs_sym[n], simplify(extract_deriv_alt(sigs_eq[n], 2))))

    #display(Eq(Rq_sym[n], extract_single_deriv(simplify(Rq_eq[n]))))

    #display(Eq(rq_sym[n], simplify(rq_eq[n])))
    #display(Eq(sigs_sym[n], simplify(sigs_eq[n])))

    #display(Eq(sigq_sym[n], simplify(sigq_eq[n])))

    #display(Eq(Rq_sym[n], simplify(Rq_eq[n])))
    #display(Eq(sigq_sym[n], simplify(sigs_eq[n])))

    #display(Eq(Symbol('0_{'+ str(n) + '}'), simplify(sigs_eq[n] - Y)))

    Z = 0

   

In [ ]:
for n in range(N+1):
    display(Eq(sig_sym[n], polynomize(variation(sigq_eq[n], q_conj))))
    #display(Eq(E_minus_sym[n], polynomize(E_minus_eq[n])))

In [6]:

def uv_sub(expr):
    return multi_substituter(expr, [(u, W_minus), (v, W_plus)])
def NV_sub(expr):
    return multi_substituter(expr, [(u, V_sym/2 - N_sym), (v, V_sym/2 + N_sym)])
def AV_sub(expr):
    return multi_substituter(expr, [(u, V_sym/2 - epsiloninv**2 * (A_sym - 1)), (v, V_sym/2 + epsiloninv**2 * (A_sym - 1))])

R_sym = [Symbol('R_{' + str(n) + '}') for n in range(N+1)]
R_eq = [R_sym[n] for n in range(N+1)]

EE_plus_sym = [Symbol('\\mathcal{E}_+^{' + str(n) + '}') for n in range(N+1)]
EE_plus_eq = [EE_plus_sym[n] for n in range(N+1)]

EE_minus_sym = [Symbol('\\mathcal{E}_-^{' + str(n) + '}') for n in range(N+1)]
EE_minus_eq = [EE_minus_sym[n] for n in range(N+1)]

HH_sym = [Symbol('\\mathcal{H}^{' + str(n-1) + '}') for n in range(N+1)]
HH_eq = [HH_sym[n] for n in range(N+1)]

EE_sym = [Symbol('\\mathcal{E}^{' + str(n) + '}') for n in range(N+1)]
EE_eq = [EE_sym[n] for n in range(N+1)]

PP_sym = [Symbol('\\mathcal{P}^{' + str(n) + '}') for n in range(N+1)]
PP_eq = [PP_sym[n] for n in range(N+1)]

EEo_sym = [Symbol('\\mathcal{E}_o^{' + str(n) + '}') for n in range(N+1)]
EEo_eq = [EEo_sym[n] for n in range(N+1)]

PPo_sym = [Symbol('\\mathcal{P}_o^{' + str(n) + '}') for n in range(N+1)]
PPo_eq = [PPo_sym[n] for n in range(N+1)]

EEh_sym = [Symbol('\\mathcal{E}_h^{' + str(n) + '}') for n in range(N+1)]
EEh_eq = [EEh_sym[n] for n in range(N+1)]

PPh_sym = [Symbol('\\mathcal{P}_h^{' + str(n) + '}') for n in range(N+1)]
PPh_eq = [PPh_sym[n] for n in range(N+1)]

EEq_sym = [Symbol('\\mathcal{E}_q^{' + str(n) + '}') for n in range(N+1)]
EEq_eq = [EEq_sym[n] for n in range(N+1)]

PPq_sym = [Symbol('\\mathcal{P}_q^{' + str(n) + '}') for n in range(N+1)]
PPq_eq = [PPq_sym[n] for n in range(N+1)]

PPm_sym = [Symbol('\\mathcal{P}_m^{' + str(n) + '}') for n in range(N+1)]
PPm_eq = [PPm_sym[n] for n in range(N+1)]

Y_plus_sym = [Symbol('Y_+^{' + str(n) + '}') for n in range(N+1)]
Y_plus_eq = [Y_plus_sym[n] for n in range(N+1)]

Y_minus_sym = [Symbol('Y_-^{' + str(n) + '}') for n in range(N+1)]
Y_minus_eq = [Y_minus_sym[n] for n in range(N+1)]

Yt_plus_sym = [Symbol('\\tilde{Y}_+^{' + str(n) + '}') for n in range(N+1)]
Yt_plus_eq = [Y_plus_sym[n] for n in range(N+1)]

Yt_minus_sym = [Symbol('\\tilde{Y}_-^{' + str(n) + '}') for n in range(N+1)]
Yt_minus_eq = [Y_minus_sym[n] for n in range(N+1)]

P_plus_sym = [Symbol('P_+^{' + str(n) + '}') for n in range(N+1)]
P_plus_eq = [P_plus_sym[n] for n in range(N+1)]

P_minus_sym = [Symbol('P_-^{' + str(n) + '}') for n in range(N+1)]
P_minus_eq = [P_minus_sym[n] for n in range(N+1)]

a_subs_q = sqrt(q * q_conj)
v_subs_q = (deriv(q) / q - deriv(q_conj) / q_conj) / (2 * I)

for n in range(0, N+1):
    print("n =", n)

    R_eq[n] = epsilon / sqrt(2) * deriv(v) * r_eq[n]
    HH_eq[n] = - I**(n-1) * (sqrt(2) * epsilon)**n * sig_eq[n]

    if 2*n+1 < N+1:
        EE_eq[n] = (-1)**(n+1) * (sqrt(2) * epsilon)**(2*n+1) * sig_eq[2*n+1]

        EE_plus_eq[n] = sig_plus_eq[2*n+1]
        EE_minus_eq[n] = sig_minus_eq[2*n+1]

        
    if 2*n < N+1:
        PP_eq[n] = I * (-1)**n * (sqrt(2) * epsilon)**(2*n) * sig_eq[2*n]

#print("substituting energy")

#EEo_eq = sqrt(2) * epsilon * multi_substituter(Matrix(EE_eq), [(v, w_plus), (u, w_minus)], magnitude=epsiloninv**2, scale=epsiloninv/sqrt(2))
#EEo_eq = [elem for elem in EEo_eq]

#EEh_eq = multi_substituter(Matrix(EEo_eq), [(w_plus, phi_sym / 2 + (a_sym - 1)), (w_minus, phi_sym / 2 - (a_sym - 1))])
#EEh_eq = [elem for elem in EEh_eq]

#EEq_eq = multi_substituter(Matrix(EEh_eq), [(phi_sym, v_subs_q), (a_sym, a_subs_q)])
#EEq_eq = [elem for elem in EEq_eq]

#print("substituting momentum")

#PPo_eq = sqrt(2) * epsilon * multi_substituter(Matrix(PP_eq), [(v, w_plus), (u, w_minus)], magnitude=epsiloninv**2, scale=epsiloninv/sqrt(2))
#PPo_eq = [elem for elem in PPo_eq]

#PPh_eq = multi_substituter(Matrix(PPo_eq), [(w_plus, phi_sym / 2 + (a_sym - 1)), (w_minus, phi_sym / 2 - (a_sym - 1))])
#PPh_eq = [elem for elem in PPh_eq]

#PPq_eq = multi_substituter(Matrix(PPh_eq), [(phi_sym, v_subs_q), (a_sym, a_subs_q)])
#PPq_eq = [elem for elem in PPq_eq]

#print("done")

print("moving on...")
for n in range(0, N+1):
    if 2*n+1 < N+1:
        PPm_eq[n] = 0
        for k in range(n):
            
    if 2*n+1 < N+1:
        Y_plus_eq[n] = EE_eq[n]
        for m in range(0, n+1):
            Y_plus_eq[n] += binomial(- Rational(1,2), m) * 2**(2*m+1) * PP_eq[n-m]
        Yt_plus_eq[n] = (-1)**(n+1)*(epsiloninv/sqrt(2))**(2*n+1) * Y_plus_eq[n]
        
        Y_minus_eq[n] = EE_eq[n]
        for m in range(0, n+1):
            Y_minus_eq[n] += - binomial(- Rational(1,2), m) * 2**(2*m+1) * PP_eq[n-m]
        Yt_minus_eq[n] = (-1)**(n+1)*(epsiloninv/sqrt(2))**(2*n+1) * Y_minus_eq[n]
        
        P_plus_eq[n] = epsiloninv**2 * (Yt_plus_eq[n] - EE_plus_eq[n])
        P_minus_eq[n] = epsiloninv**2 * (Yt_minus_eq[n] - EE_minus_eq[n])


n = 0
n = 1
n = 2
n = 3
n = 4
n = 5
n = 6
n = 7
n = 8
moving on...


In [ ]:
for n in range(0, N+1): 

    #display(Eq(sig_sym[n], polynomize(multi_substituter(sig_eq[n], [(u, V_sym/2 - (A_sym-1)), (v, V_sym/2 + (A_sym-1))]))))
    #display(Eq(HH_sym[n], polynomize(multi_substituter(HH_eq[n], [(u, V_sym/2 - (A_sym-1)), (v, V_sym/2 + (A_sym-1))]))))
    
    #display(Eq(sig_sym[n], polynomize(multi_substituter(sig_eq[n], [(u, W_minus), (v, W_plus)]))))

    #display(Eq(sig_sym[n], polynomize(AV_sub(sig_eq[n]))))

    #display(Eq(Symbol('\\frac{\delta ' + sig_sym[n].name + '}{\delta v}'), polynomize(AV_sub(variation(sig_eq[n], v)))))
    #display(Eq(Symbol('\\frac{\delta ' + sig_sym[n].name + '}{\delta u}'), polynomize(AV_sub(variation(sig_eq[n], u)))))
    #display(Eq(Symbol('\\frac{\delta ' + sig_sym[n].name + '}{\delta N}'), polynomize(AV_sub((variation(sig_eq[n], v) - variation(sig_eq[n], u))))))
    #display(Eq(Symbol('\\frac{\delta ' + sig_sym[n].name + '}{\delta V}'), polynomize(AV_sub(variation(sig_eq[n], v)/2 + variation(sig_eq[n], u)/2))))

    
    #display(Eq(HH_sym[n], polynomize(multi_substituter(HH_eq[n], [(u, W_minus), (v, W_plus)]))))
    pass

    if 2*n < N+1:
        #display(Eq(PP_sym[n], polynomize(multi_substituter(PP_eq[n], [(u, W_minus), (v, W_plus)]))))
        display(Eq(PPo_sym[n], polynomize(PPo_eq[n])))
        display(Eq(PPh_sym[n], polynomize(PPh_eq[n])))
        display(Eq(PPq_sym[n], simplify(simplify(poly_simplify(PPq_eq[n]).subs(q_conj, abs(q)**2 / q))).expand()))
        pass
    if 2*n+1 < N+1:
        #display(Eq(EE_sym[n], polynomize(multi_substituter(EE_eq[n], [(u, W_minus), (v, W_plus)]))))
        display(Eq(EEo_sym[n], polynomize(EEo_eq[n])))
        display(Eq(EEh_sym[n], polynomize(EEh_eq[n])))
        display(Eq(EEq_sym[n], simplify(simplify(poly_simplify(EEq_eq[n]).subs(q_conj, abs(q)**2 / q))).expand()))
        pass
    pass

for n in range(0, N+1): 
    if 2*n < N+1:
        display(Eq(Symbol('\\frac{\delta ' + PPq_sym[n].name + '}{\delta \\overline{q}}'), simplify(simplify(variation(poly_simplify(PPq_eq[n]), q_conj).subs(q_conj, abs(q)**2 / q)))).expand())

    if 2*n+1 < N+1:
        display(Eq(Symbol('\\frac{\delta ' + EEq_sym[n].name + '}{\delta \\overline{q}}'), simplify(simplify(variation(poly_simplify(EEq_eq[n]), q_conj).subs(q_conj, abs(q)**2 / q)))).expand())



In [ ]:
B = 0
B += - epsilon * sqrt(2) / 4 * (EE_plus_eq[0] - EE_minus_eq[0] + epsilon**2 * (P_plus_eq[0] - P_minus_eq[0]))

display(polynomize(uv_sub(PP_eq[0] - B)))

B = 0
B += (sqrt(2) * epsilon)**3 / 4 * (EE_plus_eq[1] - EE_minus_eq[1] + epsilon**2 * (P_plus_eq[1] - P_minus_eq[1]))
B += - 2 * epsilon * sqrt(2) / 4 * (EE_plus_eq[0] - EE_minus_eq[0] + epsilon**2 * (P_plus_eq[0] - P_minus_eq[0]))

display(polynomize(uv_sub(PP_eq[1] - B)))

B = 0
B += - (sqrt(2) * epsilon)**5 / 4 * (EE_plus_eq[2] - EE_minus_eq[2] + epsilon**2 * (P_plus_eq[2] - P_minus_eq[2]))
B += 6 * epsilon * sqrt(2) / 4 * (EE_plus_eq[0] - EE_minus_eq[0] + epsilon**2 * (P_plus_eq[0] - P_minus_eq[0]))
B += 2 * (sqrt(2) * epsilon)**3 / 4 * (EE_plus_eq[1] - EE_minus_eq[1] + epsilon**2 * (P_plus_eq[1] - P_minus_eq[1]))
B += - 4 * epsilon * sqrt(2) / 4 * (EE_plus_eq[0] - EE_minus_eq[0] + epsilon**2 * (P_plus_eq[0] - P_minus_eq[0])) 
display(polynomize(uv_sub(PP_eq[2] - B)))

In [ ]:
#hamiltonian = PP_eq[2]

B = 0
#B += 2 * epsilon * sqrt(2) / 4 * (EE_plus_eq[0] - EE_minus_eq[0] + epsilon**2 * (P_plus_eq[0] - P_minus_eq[0]))
#B += 2 * (sqrt(2) * epsilon)**3 / 4 * (EE_plus_eq[1] - EE_minus_eq[1] + epsilon**2 * (P_plus_eq[1] - P_minus_eq[1]))
B += - (sqrt(2) * epsilon)**5 / 4 * (EE_plus_eq[2] - EE_minus_eq[2] + epsilon**2 * (P_plus_eq[2] - P_minus_eq[2]))

hamiltonian = PP_eq[2]
varis = [poly_simplify(variation(hamiltonian, v)), poly_simplify(variation(hamiltonian, u))]
result = Matrix([0, 0])

T = 1
result[0] += epsiloninv**2 / 2 * (- deriv((varis[0] - varis[1]) * T) - deriv(varis[0] + varis[1]) * T)
result[1] += epsiloninv**2 / 2 * (- deriv((varis[0] - varis[1]) * T) + deriv(varis[0] + varis[1]) * T)

T = - epsilon**2 / 2 * (v - u)
result[0] += epsiloninv**2 / 2 * (- deriv((varis[0] - varis[1]) * T) - deriv(varis[0] + varis[1]) * T)
result[1] += epsiloninv**2 / 2 * (- deriv((varis[0] - varis[1]) * T) + deriv(varis[0] + varis[1]) * T)

result[0] = poly_simplify(result[0])
result[1] = poly_simplify(result[1])


display(polynomize(result[0]) + O(epsilon**5))
display(polynomize(result[1]) + O(epsilon**5))

In [11]:
for n in range(1, N+1):
    display(simplify(Rational(1,2) * deriv(variation(sig_minus_eq[n], u))))
    display(simplify(Rational(1,2) * deriv(variation(sig_plus_eq[n], v))))

0

0

0

0

(u)_{x}

...

0

(u)_{xxxxx} + 10*(u)_{xxx}*u + 20*(u)_{xx}*(u)_{x} + 30*(u)_{x}*u**2

(v)_{xxxxx} - 10*(v)_{xxx}*v - 20*(v)_{xx}*(v)_{x} + 30*(v)_{x}*v**2

0

0

In [12]:
for n in range(0, N+1):
    
    #print("n =", n)
    nn = n - 1
    max_order = 1
    max_display_order = 5
    hamiltonian = HH_eq[nn+1] #re(sig_eq[n]) if n % 2 == 1 else I * im(sig_eq[n])
    evol = evolution(hamiltonian, max_order)
    
    if n % 2 == 0 and nn//2 == 0:
        dtdT = epsilon * sqrt(2) / 2
    elif nn % 2 == 0 and nn//2 > 0:
        dtdT = epsiloninv**(2*(nn//2)-1) / sqrt(2)**(2*(nn//2)-1) / 2
    elif nn % 2 == 1:
        dtdT = epsiloninv / sqrt(2)
        
    display(Eq(Symbol('u_{T_{' + str(nn) + '}}'), polynomize(dtdT * evol[0]).subs(u, 0) + O(epsilon**4)))
    display(Eq(Symbol('v_{T_{' + str(nn) + '}}'), polynomize(dtdT * evol[1]).subs(v, 0) + O(epsilon**4)))

Eq(u_{T_{-1}}, O(\epsilon**4))

Eq(v_{T_{-1}}, O(\epsilon**4))

Eq(u_{T_{0}}, \epsilon**2*(-(u)_{x}*v/4 + (v)_{x}*v/4) + O(\epsilon**4))

Eq(v_{T_{0}}, \epsilon**2*((u)_{x}*u/4 - (v)_{x}*u/4) + O(\epsilon**4))

Eq(u_{T_{1}}, -(v)_{x}*\epsilon**2 + O(\epsilon**4))

...

Eq(v_{T_{5}}, \epsilon**2*(4*(u)_{xxx} + 24*(u)_{x}*u) + 2*(u)_{x} + O(\epsilon**4))

Eq(u_{T_{6}}, \epsilon**2*(-(u)_{xxxxxxx}/4 + 5*(u)_{xxxxx}*v/4 - 5*(u)_{xxxx}*(u)_{x} + 2*(u)_{xxxx}*(v)_{x} - 35*(u)_{xxx}*(u)_{xx}/4 + 5*(u)_{xxx}*(v)_{xx}/4 - 15*(u)_{xxx}*v**2/4 - 5*(u)_{xx}*(v)_{xxx}/4 - 5*(u)_{xx}*(v)_{x}*v - 15*(u)_{x}**3/2 - 5*(u)_{x}**2*(v)_{x}/4 - 5*(u)_{x}*(v)_{xxxx}/2 + 15*(u)_{x}*(v)_{xx}*v/2 + 5*(u)_{x}*(v)_{x}**2 + (v)_{xxxxxxx}/4 - 19*(v)_{xxxxx}*v/4 - 31*(v)_{xxxx}*(v)_{x}/2 - 105*(v)_{xxx}*(v)_{xx}/4 + 30*(v)_{xxx}*v**2 + 255*(v)_{xx}*(v)_{x}*v/2 + 135*(v)_{x}**3/4 - 70*(v)_{x}*v**3) - 30*(v)_{x}*v**2 + 20*(v)_{xx}*(v)_{x} + 10*(v)_{xxx}*v - (v)_{xxxxx} + O(\epsilon**4))

Eq(v_{T_{6}}, \epsilon**2*(-(u)_{xxxxxxx}/4 - 19*(u)_{xxxxx}*u/4 - 31*(u)_{xxxx}*(u)_{x}/2 - 5*(u)_{xxxx}*(v)_{x}/2 - 105*(u)_{xxx}*(u)_{xx}/4 - 5*(u)_{xxx}*(v)_{xx}/4 - 30*(u)_{xxx}*u**2 - 255*(u)_{xx}*(u)_{x}*u/2 + 5*(u)_{xx}*(v)_{xxx}/4 - 15*(u)_{xx}*(v)_{x}*u/2 - 135*(u)_{x}**3/4 - 5*(u)_{x}**2*(v)_{x} + 2*(u)_{x}*(v)_{xxxx} + 5*(u)_{x}*(v)_{xx}*u + 5*(u)_{x}*(v)_{x}**2/4 - 70*(u)_{x}*u**3 + (v)_{xxxxxxx}/4 + 5*(v)_{xxxxx}*u/4 - 5*(v)_{xxxx}*(v)_{x} - 35*(v)_{xxx}*(v)_{xx}/4 + 15*(v)_{xxx}*u**2/4 + 15*(v)_{x}**3/2) + 30*(u)_{x}*u**2 + 20*(u)_{xx}*(u)_{x} + 10*(u)_{xxx}*u + (u)_{xxxxx} + O(\epsilon**4))

Eq(u_{T_{7}}, \epsilon**2*(-4*(v)_{xxx} + 24*(v)_{x}*v) - 4*(v)_{x} + O(\epsilon**4))

Eq(v_{T_{7}}, \epsilon**2*(-4*(u)_{xxx} - 24*(u)_{x}*u) - 4*(u)_{x} + O(\epsilon**4))

In [9]:
W_plus = Symbol('W_+', real=True)
W_minus = Symbol('W_-', real=True)
V_sym = Symbol('V')
N_sym = Symbol('N')
A_sym = Symbol('A')

#differentiable_symbols.extend([W_plus, W_minus, V_sym, N_sym, A_sym])

def uv_sub(expr):
    return multi_substituter(expr, [(u, W_minus), (v, W_plus)])

def evolution(hamiltonian, order):
    varis = [poly_simplify(variation(hamiltonian, v)), poly_simplify(variation(hamiltonian, u))]
    result = Matrix([0, 0])
    for k in range(order+1):

        T = (-epsilon**2 / 2 * (v - u))**k
        result[0] += epsiloninv**2 / 2 * (- deriv((varis[0] - varis[1]) * T) - deriv(varis[0] + varis[1]) * T)
        result[1] += epsiloninv**2 / 2 * (- deriv((varis[0] - varis[1]) * T) + deriv(varis[0] + varis[1]) * T)

        result[0] = poly_simplify(result[0])
        result[1] = poly_simplify(result[1])
        
    return result


def evolution2(hamiltonian, order):
    H = multi_substituter(hamiltonian, [(u, V_sym/2 - N_sym), (v, V_sym/2 + N_sym)])
    HH = epsiloninv / sqrt(2) * (2 * epsilon**6 * deriv(N_sym)**2 + epsilon**4 * (1 + epsilon**2 * N_sym)**2 * V_sym**2 + ((1 + epsilon**2 * N_sym)**2 - 1)**2)
    
    varis = [poly_simplify(variation(H, N_sym)), poly_simplify(variation(H, V_sym))]

    result = Matrix([0, 0])
    
    for k in range(order+1):

        T = (- epsilon**2 * N_sym)**k
        result[0] += epsiloninv**2 * T * deriv(varis[1])
        result[1] += epsiloninv**2 * deriv(T * varis[0])

        result[0] = poly_simplify(result[0])
        result[1] = poly_simplify(result[1])
        
    return result
                  
def evolution3(hamiltonian, order):
    H = multi_substituter(hamiltonian, [(u, V_sym/2 - N_sym), (v, V_sym/2 + N_sym)])
    HH = epsiloninv / sqrt(2) * (2 * epsilon**6 * deriv(N_sym)**2 + epsilon**4 * (1 + epsilon**2 * N_sym)**2 * V_sym**2 + ((1 + epsilon**2 * N_sym)**2 - 1)**2)
    
    varis = [poly_simplify(variation(H, N_sym)), poly_simplify(variation(H, V_sym))]

    result = Matrix([0, 0])
    
    R = Symbol('R')
    result[0] += epsiloninv**2 * 1 / (1 + epsilon**2 * N_sym) * deriv(varis[1])
    result[1] += epsiloninv**2 * deriv(1 / (1 + epsilon**2 * N_sym) * varis[0])

    #result[0] = poly_simplify(result[0])
    #result[1] = poly_simplify(result[1])
        
    return result


print("moving on...")
for n in range(0, N+1):
    
    #print("n =", n)
    nn = n - 1
    max_order = 1
    max_display_order = 5
    hamiltonian = G_eq[nn+1] #re(sig_eq[n]) if n % 2 == 1 else I * im(sig_eq[n])
    evol = evolution(hamiltonian, max_order)
    
    if n % 2 == 0 and nn//2 == 0:
        dtdT = epsilon * sqrt(2) / 2
    elif nn % 2 == 0 and nn//2 > 0:
        dtdT = epsiloninv**(2*(nn//2)-1) / sqrt(2)**(2*(nn//2)-1) / 2
    elif nn % 2 == 1:
        dtdT = epsiloninv / sqrt(2)
    
    dtdT = 1
    
    display(Eq(Symbol('(W_+)_{T_{' + str(nn) + '}}'), polynomize(dtdT * uv_sub(evol[0])) + O(epsilon**4)))
    display(Eq(Symbol('(W_-)_{T_{' + str(nn) + '}}'), polynomize(dtdT * uv_sub(evol[1])) + O(epsilon**4)))

moving on...


C:\Users\rober\AppData\Local\Temp\ipykernel_4936\2110384835.py:83: SymPyDeprecationWarning: 

Mixing Poly with non-polynomial expressions in binary
operations is deprecated. Either explicitly convert
the non-Poly operand to a Poly with as_poly() or
convert the Poly to an Expr with as_expr().

See https://docs.sympy.org/latest/explanation/active-deprecations.html#deprecated-poly-nonpoly-binary-operations
for details.

This has been deprecated since SymPy version 1.6. It
will be removed in a future version of SymPy.

  display(Eq(Symbol('(W_+)_{T_{' + str(nn) + '}}'), polynomize(dtdT * uv_sub(evol[0])) + O(epsilon**4)))


Eq((W_+)_{T_{-1}}, O(\epsilon**4))

Eq((W_-)_{T_{-1}}, O(\epsilon**4))

Eq((W_+)_{T_{0}}, O(\epsilon**4))

...

Eq((W_-)_{T_{5}}, O(\epsilon**4))

Eq((W_+)_{T_{6}}, (-(W_+)_{xxxxx} + 10*(W_+)_{xxx}*W_+ + 20*(W_+)_{xx}*(W_+)_{x} - 30*(W_+)_{x}*W_+**2)/\epsilon**2 + \epsilon**2*(7*(W_+)_{xxxxx}*W_+**2/8 - (W_+)_{xxxxx}*W_+*W_-/2 - 3*(W_+)_{xxxxx}*W_-**2/8 + 27*(W_+)_{xxxx}*(W_+)_{x}*W_+/8 - 17*(W_+)_{xxxx}*(W_+)_{x}*W_-/8 + 3*(W_+)_{xxxx}*(W_-)_{x}*W_+/8 - 13*(W_+)_{xxxx}*(W_-)_{x}*W_-/8 + 35*(W_+)_{xxx}*(W_+)_{xx}*W_+/8 - 35*(W_+)_{xxx}*(W_+)_{xx}*W_-/8 + 15*(W_+)_{xxx}*(W_+)_{x}**2/8 - 5*(W_+)_{xxx}*(W_+)_{x}*(W_-)_{x}/4 + 5*(W_+)_{xxx}*(W_-)_{xx}*W_+/8 - 5*(W_+)_{xxx}*(W_-)_{xx}*W_-/8 - 5*(W_+)_{xxx}*(W_-)_{x}**2/8 - 35*(W_+)_{xxx}*W_+**3/16 + 155*(W_+)_{xxx}*W_+**2*W_-/16 + 35*(W_+)_{xxx}*W_+*W_-**2/16 + 5*(W_+)_{xxx}*W_-**3/16 + 5*(W_+)_{xx}**2*(W_+)_{x}/4 - 5*(W_+)_{xx}**2*(W_-)_{x}/4 - 15*(W_+)_{xx}*(W_+)_{x}*W_+**2/16 + 325*(W_+)_{xx}*(W_+)_{x}*W_+*W_-/8 + 85*(W_+)_{xx}*(W_+)_{x}*W_-**2/16 - 5*(W_+)_{xx}*(W_-)_{xxx}*W_+/8 + 5*(W_+)_{xx}*(W_-)_{xxx}*W_-/8 + 115*(W_+)_{xx}*(W_-)_{x}*W_+**2/16 + 55*(W_+)_{xx}*(W_-)_{x}*W_+*W_-

Eq((W_-)_{T_{6}}, (-(W_-)_{xxxxx} - 10*(W_-)_{xxx}*W_- - 20*(W_-)_{xx}*(W_-)_{x} - 30*(W_-)_{x}*W_-**2)/\epsilon**2 + \epsilon**2*(5*(W_+)_{xxxxx}*W_+**2/8 - 5*(W_+)_{xxxxx}*W_-**2/8 + 27*(W_+)_{xxxx}*(W_+)_{x}*W_+/8 - 17*(W_+)_{xxxx}*(W_+)_{x}*W_-/8 + 3*(W_+)_{xxxx}*(W_-)_{x}*W_+/8 - 13*(W_+)_{xxxx}*(W_-)_{x}*W_-/8 + 35*(W_+)_{xxx}*(W_+)_{xx}*W_+/8 - 35*(W_+)_{xxx}*(W_+)_{xx}*W_-/8 + 15*(W_+)_{xxx}*(W_+)_{x}**2/8 - 5*(W_+)_{xxx}*(W_+)_{x}*(W_-)_{x}/4 + 5*(W_+)_{xxx}*(W_-)_{xx}*W_+/8 - 5*(W_+)_{xxx}*(W_-)_{xx}*W_-/8 - 5*(W_+)_{xxx}*(W_-)_{x}**2/8 + 5*(W_+)_{xxx}*W_+**3/16 + 75*(W_+)_{xxx}*W_+**2*W_-/16 + 75*(W_+)_{xxx}*W_+*W_-**2/16 + 5*(W_+)_{xxx}*W_-**3/16 + 5*(W_+)_{xx}**2*(W_+)_{x}/4 - 5*(W_+)_{xx}**2*(W_-)_{x}/4 + 65*(W_+)_{xx}*(W_+)_{x}*W_+**2/16 + 245*(W_+)_{xx}*(W_+)_{x}*W_+*W_-/8 + 165*(W_+)_{xx}*(W_+)_{x}*W_-**2/16 - 5*(W_+)_{xx}*(W_-)_{xxx}*W_+/8 + 5*(W_+)_{xx}*(W_-)_{xxx}*W_-/8 + 115*(W_+)_{xx}*(W_-)_{x}*W_+**2/16 + 55*(W_+)_{xx}*(W_-)_{x}*W_+*W_-/8 + 15*(W_+)_{xx}*(W_-)_{x

Eq((W_+)_{T_{7}}, O(\epsilon**4))

Eq((W_-)_{T_{7}}, O(\epsilon**4))

In [ ]:

M_sym = [Symbol('M_{' + str(n) + '}') for n in range(N+1)]
M_eq = [M_sym[n] for n in range(N+1)]
Mt_sym = [Symbol('\\tilde{M}_{' + str(n) + '}') for n in range(N+1)]
Mt_eq = [M_sym[n] for n in range(N+1)]
N_sym = [Symbol('N_{' + str(n) + '}') for n in range(N+1)]
N_eq = [N_sym[n] for n in range(N+1)]
St_sym = [Symbol('\\tilde{S}_{' + str(n) + '}') for n in range(N+1)]
St_eq = [M_sym[n] for n in range(N+1)]

for n in range(0, N+1):
        
    M_eq[n] = im(S_eq[n] - 2 * S_plus_eq[n])
    N_eq[n] = re(S_eq[n] - 2 * S_plus_eq[n])
    
for n in range(0, N+1):
    Mt_eq[n] = 0
    for k in range(0, n+1):
        Mt_eq[n] += M_eq[k] * M_eq[n-k]

for n in range(0, N):
    print("n =", n)

    V = 0
    V += deriv(S_plus_eq[n])
    V += S_plus_eq[n+1]
    
    for k in range(n+1):
        V += S_plus_eq[k] * S_plus_eq[n-k]
    
    V += - plus_beta_coeff(n) * u
    
    X = 0
    X += deriv(M_eq[n])
    X += M_eq[n+1]
    
    for k in range(n+1):
        X += S_eq[k] * M_eq[n-k]
        X += - N_eq[k] * M_eq[n-k]
    
    X += - im(q_coeff(n)) * u
    
    XX = 0
    for k in range(n+1):
        XX += deriv(M_eq[k]) * M_eq[n-k]
    for k in range(n+2):
        XX += M_eq[k] * M_eq[n+1-k]
    for k in range(n+1):
        for m in range(k+1):
            XX += S_eq[m] * M_eq[k-m] * M_eq[n-k]
            XX += - N_eq[m] * M_eq[k-m] * M_eq[n-k]

    for k in range(-1, n+1):
        XX += - im(q_coeff(k)) * u * M_eq[n-k]
    
    XXX = 0
    XXX += deriv(Mt_eq[n]) / 2
    XXX += Mt_eq[n+1]
    for k in range(n+1):
        XXX += S_eq[k] * Mt_eq[n-k]
        XXX += - N_eq[k] * Mt_eq[n-k]
        
    for k in range(-1, n+1):
        XXX += - im(q_coeff(k)) * M_eq[n-k] * u
        
    XXXX = XXX
    
    XXXX = 0
    XXXX += deriv(Mt_eq[n]) / 2
    XXXX += Mt_eq[n+1]
    for k in range(n+1):
        XXXX += S_eq[k] * Mt_eq[n-k]
        XXXX += - N_eq[k] * Mt_eq[n-k]
        
    for k in range(-1, n+1):
        XXXX += - im(q_coeff(k)) * M_eq[n-k] * u

    Z = 0
    for k in range(n+1):
        Z += S_eq[k] * S_eq[n-k]
    
    
    Y = 0
    for k in range(n+1):
        Y += im(S_eq[k] - 2*S_plus_eq[k]) * im(S_eq[n-k] - 2*S_plus_eq[n-k])
    YY = 0
    for k in range(n+1):
        YY += im(S_plus_eq[k]) * im(S_plus_eq[n-k])
    
    W = 0
    WW = 0
    for k in range(-1,n+1):
        W += im(q_coeff(k)) * im(S_eq[n-k] - 2*S_plus_eq[n-k])
        WW += im(q_coeff(k)) * im(2*S_plus_eq[n-k])

        
    #display(polynomize(re(2*S_plus_eq[n]) - S_eq[n]) + O(epsilon**2))
    #display(polynomize(WW + S_eq[n]) + O(epsilon**2))
    #display(polynomize(re(2*S_plus_eq[n]) - WW) / 2 + O(epsilon**2))
    display(polynomize(YY))
    #display(polynomize(XXX))
    #display(polynomize(Y + Z))
    #display(polynomize(- W))


In [ ]:
def coeff(n):
    if n < 0 or n % 2 == 0:
        return 0
    return binomial(- Rational(1,2), (n-1)//2) * sqrt(2)**n * I**n * epsiloninv**n

for n in range(1, N): 
    print("n =", n)
    
    X = sig_eq[n+1]
    
    X += deriv(sig_eq[n])
    
    for k in range(0, n+1):
        X += sig_eq[k] * sig_eq[n-k]
        
    X += q_coeff(n) / 2 * (u + v)
    
    X += - deriv(v) / sqrt(2) * epsilon * r_eq[n]
    
    display(polynomize(X))
    

In [ ]:
def coeff(n):
    if n < 0 or n % 2 == 0:
        return 0
    return binomial(- Rational(1,2), (n-1)//2) * sqrt(2)**n * I**n * epsiloninv**n

for n in range(1, N): 
    print("n =", n)
    
    X = re(sig_eq[n+1])
    
    X += deriv(re(sig_eq[n]))
    
    for k in range(0, n+1):
        X += re(sig_eq[k]) * re(sig_eq[n-k])
        X += - im(sig_eq[k]) * im(sig_eq[n-k])
        
    X += re(q_coeff(n)) / 2 * (u + v)
    
    X += - deriv(v) / sqrt(2) * epsilon * re(r_eq[n])
     
        
    Y = im(sig_eq[n+1])
    
    Y += deriv(im(sig_eq[n]))
    
    for k in range(0, n+1):
        Y += 2 * re(sig_eq[k]) * im(sig_eq[n-k])
        
    Y += im(q_coeff(n)) / 2 * (u + v)
    
    Y += - deriv(v) / sqrt(2) * epsilon * im(r_eq[n])
    
    Z = 0
    for m in range(0, n+1):
        ZZ = im(sig_eq[m+1])

        ZZ += deriv(im(sig_eq[m]))

        for k in range(0, m+1):
            ZZ += 2 * re(sig_eq[k]) * im(sig_eq[m-k])

        ZZ += im(q_coeff(m)) / 2 * (u + v)

        ZZ += - deriv(v) / sqrt(2) * epsilon * im(r_eq[m])
    
        Z += coeff(n-k) / I * ZZ 
      
    A = re(sig_eq[n])
    for m in range(0, n+1):
        A += coeff(n-m) / I * im(sig_eq[m])
        

    display(polynomize(X + Z))

In [ ]:
def coeff(n):
    if n < 0 or n % 2 == 0:
        return 0
    return binomial(- Rational(1,2), (n-1)//2) * sqrt(2)**n * I**n * epsiloninv**n

for n in range(0, N): 
    print("n =", n)
    
    X = E_plus_eq[n+1]
    
    X += deriv(E_plus_eq[n])
    
    for k in range(0, n+1):
        X += re(sig_eq[k]) * re(sig_eq[n-k])
        
    for k in range(0, n+1):
        X += - im(sig_eq[k]) * im(sig_eq[n-k])
    
    for k in range(0, n+1):
        for m in range(0, k+1):
            X += - 2 / I * coeff(n) * re(sig_eq[k-m]) * im(sig_eq[n-k])
        
    if n == 0:
        X += u + epsilon**2 / 2 * u * v
        
    X += deriv(v) / sqrt(2) * F_plus_eq[n]
    
    display(polynomize(X))

In [ ]:
for n in range(0, N): 
    print("n =", n)
    
    X = E_plus_eq[n+1]
    
    X += deriv(E_plus_eq[n])
    
    for k in range(0, n+1):
        X += E_plus_eq[k] * E_plus_eq[n-k]
        
    for k in range(0, n+1):
        X += - im(sig_eq[k]) * im(sig_eq[n-k])
    
    #for k in range(0, n+1):
    #    for m in range(0, k+1):
    #        X += - 2**m * epsiloninv**(2*m) * im(sig_eq[k-m]) * im(sig_eq[n-k])
        
    if n == 0:
        X += u + epsilon**2 / 2 * u * v
        
    X += deriv(v) / sqrt(2) * F_plus_eq[n]
    
    display(polynomize(X))

In [ ]:

def G(n):
    if n == 0:
        return 1
    if n % 2 == 1 and n >= 0:
        return binomial(2 * (n//2), n//2) * sqrt(2) * I * epsiloninv**n / 2**(n//2)
    else:
        return 0
    
def q_coeff(n):
    if n % 2 == 0 or n < 0:
        return 0
    else:
        nn = (n - 1) // 2
        if nn % 2 == 0:
            return I**n * epsiloninv**n * catalan(nn) / (sqrt(2) * 2**nn)
        elif nn % 2 == 1:
            return - I**n * epsiloninv**n* catalan(nn) / (sqrt(2) * 2**nn)


def sqrtt(x):
    return I*sqrt(-x)

X = Symbol('X')

def get_expansion(eqn, x0, var, num):
    epsilonpos = Symbol('epsilon_2', positive=True)
    epsiloninvpos = Symbol('eta_2', positive=True)
    
    eqn = eqn.subs(epsilon, epsilonpos).subs(epsiloninv, epsiloninvpos)
    ser = series(eqn, x0=0, x=var, n=num).removeO().subs(epsilonpos, epsilon).subs(epsiloninvpos, epsiloninv)
    return [simplify(ser.coeff(var**n) if n > 0 else ser.subs(var, 0)) for n in range(0, num)]
    
N = 6

#h_plus = get_expansion(1 + 1 / sqrtt(1 - Rational(1, 2) / epsiloninv**2 / X**2), 0, X, N+1)
#h_minus = get_expansion(1 - 1 / sqrtt(1 - Rational(1, 2) / epsiloninv**2 / X**2), 0, X, N+1)

h_plus = [binomial(-Rational(1, 2), n//2) * (sqrt(2) * I * epsiloninv)**n if n % 2 == 1 else 0 for n in range(N+1)]
h_plus[0] = 1
h_minus = [- binomial(-Rational(1, 2), n//2) * (sqrt(2) * I * epsiloninv)**n if n % 2 == 1 else 0 for n in range(N+1)]
h_minus[0] = 1

h_plus_mod = get_expansion(1 + 1 / sqrtt(1 + Rational(1, 2) / epsiloninv**2 / X**2), 0, X, N+1)
h_minus_mod = get_expansion(1 - 1 / sqrtt(1 + Rational(1, 2) / epsiloninv**2 / X**2), 0, X, N+1)

exp1 = [get_expansion((X / sqrtt(1 - 1 / (8 * epsiloninv**2 * X**2)))**n, 0, X, N+1) for n in range(N+1)]

for n in range(N+1):
    #print("n =", n)
    for k in range(N+1):
        #print("k =", k)
        #display(exp1[n][k])
        pass
    pass

E_plus_sym = [Symbol('E^+_{' + str(n) + '}') for n in range(N+1)]
E_plus_eq = [E_plus_sym[n] for n in range(N+1)]

E_minus_sym = [Symbol('E^-_{' + str(n) + '}') for n in range(N+1)]
E_minus_eq = [E_plus_sym[n] for n in range(N+1)]

F_plus_sym = [Symbol('F^+_{' + str(n) + '}') for n in range(N+1)]
F_plus_eq = [E_plus_sym[n] for n in range(N+1)]

F_minus_sym = [Symbol('F^-_{' + str(n) + '}') for n in range(N+1)]
F_minus_eq = [E_plus_sym[n] for n in range(N+1)]

Ft_plus_sym = [Symbol('\\tilde{F}^+_{' + str(n) + '}') for n in range(N+1)]
Ft_plus_eq = [E_plus_sym[n] for n in range(N+1)]

Ft_minus_sym = [Symbol('\\tilde{F}^-_{' + str(n) + '}') for n in range(N+1)]
Ft_minus_eq = [E_plus_sym[n] for n in range(N+1)]

r_sym = [Symbol('r_{' + str(n) + '}') for n in range(N+1)]
r_eq = [r_sym[n] for n in range(N+1)]
r_poly = [r_sym[n] for n in range(N+1)]

sig_sym = [Symbol('\sigma_{' + str(n) + '}') for n in range(N+1)]
sig_eq = [sig_sym[n] for n in range(N+1)]
sig_poly = [sig_sym[n] for n in range(N+1)]

r_factor = 1 #Symbol('\lambda')
r_eq[0] = r_factor * (- I * sqrt(2) * epsiloninv)
r_eq[1] = r_factor * (u - epsiloninv**2 + (Rational(1, 2) + epsilon**2 * v / 2) * r_eq[0]**2)
r_poly[0] = polynomize(r_eq[0])
r_poly[1] = polynomize(r_eq[1])


sig_eq[0] = epsilon / (2 * sqrt(2) * I) * (u + v)
sig_eq[1] = deriv(sig_eq[0]) + sig_eq[0]**2 + u * (1 + epsilon**2 * v / 2) - (u + v) / 2 + epsilon**2 * deriv(v) / 2 * r_eq[0]
sig_poly[0] = polynomize(sig_eq[0])
sig_poly[1] = polynomize(sig_eq[1])

for n in range(1, N):
    
    print("n =", n)
    print(datetime.datetime.now())

    X = 0

    X += - deriv(r_eq[n]) 
    
    X += q_coeff(n) * epsiloninv**2

    for k in range(0, n+1):
        X += (Rational(1, 2) + epsilon**2 * v / 2) * r_eq[k] * r_eq[n-k]

    for k in range(1, n+1):
        X += epsilon / (2 * sqrt(2) * I) * r_eq[k] * r_eq[n+1-k]

    for k in range(0, n+1):
        for j in range(0, k+1):
            X += q_coeff(j) * r_eq[k-j] * r_eq[n-k] / 2
            
    r_eq[n+1] = r_factor * X
    r_poly[n+1] = polynomize(r_eq[n+1])
    r_eq[n+1] = depolynomize(r_poly[n+1])
    
    X = 0

    X += deriv(sig_eq[n]) 
    
    X += q_coeff(n) / 2 * (u + v)
    
    for k in range(0, n+1):
        X += sig_eq[k] * sig_eq[n-k]

    X += epsilon**2 * deriv(v) / 2 * r_eq[n]

    sig_eq[n+1] = X
    sig_poly[n+1] = polynomize(sig_eq[n+1])
    sig_eq[n+1] = depolynomize(sig_poly[n+1])
    
    #display(Eq(r_sym[n], polynomize(r_eq[n])))
    
for n in range(0, N+1):
    E_plus_eq[n] = 0
    for k in range(0, n+1):
        E_plus_eq[n] += re(sig_eq[k] * h_plus[n-k])
    
    E_minus_eq[n] = 0
    for k in range(0, n+1):
        E_minus_eq[n] += re(sig_eq[k] * h_minus[n-k])
        
    F_plus_eq[n] = 0
    for k in range(0, n+1):
        F_plus_eq[n] += im(sig_eq[k] * h_plus[n-k])
        
    F_minus_eq[n] = 0
    for k in range(0, n+1):
        F_minus_eq[n] += im(sig_eq[k] * h_minus[n-k])
        
    Ft_plus_eq[n] = 0
    for k in range(0, N+1):
        Ft_plus_eq[n] += exp1[k][n] * F_plus_eq[k]
        
    Ft_minus_eq[n] = 0
    for k in range(0, N+1):
        Ft_minus_eq[n] += exp1[k][n] * F_minus_eq[k]
       
    display(Eq(sig_sym[n], sig_poly[n]))
    display(Eq(E_plus_sym[n], polynomize(E_plus_eq[n])))
    display(Eq(E_minus_sym[n], polynomize(E_minus_eq[n])))
    display(Eq(F_plus_sym[n], polynomize(F_plus_eq[n])))
    display(Eq(F_minus_sym[n], polynomize(F_minus_eq[n])))
    #display(Eq(Ft_plus_sym[n], polynomize(Ft_plus_eq[n])))
    #display(Eq(Ft_minus_sym[n], polynomize(Ft_minus_eq[n])))



In [ ]:
N = 8

def polynomize_matrix(M):
    return Matrix([[polynomize(M[0,0]), polynomize(M[0,1])], [polynomize(M[1,0]), polynomize(M[1,1])]])
def depolynomize_matrix(M):
    return Matrix([[depolynomize(M[0,0]), depolynomize(M[0,1])], [depolynomize(M[1,0]), depolynomize(M[1,1])]])

def coeff(n):
    return binomial(-Rational(1, 2), n//2) * (sqrt(2) * I * epsiloninv)**n if n % 2 == 1 else 0

M_sym = [Symbol('M_{' + str(n) + '}') for n in range(N+1)]
M_eq = [M_sym[n] for n in range(N+1)]

M_eq[0] = Matrix([[0, - I * epsilon * sqrt(2) * (u + v) / 4], [- I * epsilon * sqrt(2) * (u + v) / 4 , 0]])

P = Matrix([[0, 1], [1, 0]])

for n in range(0, N):

    print("n =", n+1)

    X = Matrix([[deriv(M_eq[n][0,0]), deriv(M_eq[n][0,1])], [deriv(M_eq[n][1,0]), deriv(M_eq[n][1,1])]])
    
    for k in range(0, n+1):
        X += Rational(1, 2) * (M_eq[k] * M_eq[n-k] + P * M_eq[k] * P * M_eq[n-k])
        
    X -= (u + v) / 2 * epsilon**2 / 2 * Matrix([[0, - im(- coeff(n+2)) * I], [im(coeff(n+2)) * I, 0]])
    
    X -= Matrix([[0, (- u * (1 + epsilon**2 * v / 2) + (u + v)) * im(- coeff(n)) * I], [- u * (1 + epsilon**2 * v / 2) * im(coeff(n)) * I, 0]])
    
    if n == 0:
        X -= Matrix([[- u * (1 + epsilon**2 * v / 2), 0], [0 , - u * (1 + epsilon**2 * v / 2) + (u + v)]])

    M_eq[n+1] = Matrix([[depolynomize(polynomize(X[0,0])), 
                         depolynomize(polynomize(X[0,1]))], 
                        [depolynomize(polynomize(X[1,0])), 
                         depolynomize(polynomize(X[1,1]))]])
    
    obtained = polynomize_matrix(M_eq[n+1])
    
    compare = Matrix([[polynomize(E_plus_eq[n+1]) + O(epsilon**2),
                    polynomize(I * F_minus_eq[n+1]) + O(epsilon**2)],
                   [polynomize(I * F_plus_eq[n+1]) + O(epsilon**2),
                    polynomize(E_minus_eq[n+1]) + O(epsilon**2)]])
    
    display(polynomize_matrix(M_eq[n+1]))


In [ ]:

q_coeffs = [I * catalan(n//2) / sqrt(2)**n * epsiloninv**n if n % 2 == 1 else 0 for n in range(0, N+1)]


Eh_plus_sym = [Symbol('\\hat{E}^+_{' + str(n) + '}') for n in range(N+1)]
Eh_plus_eq = [Eh_plus_sym[n] for n in range(N+1)]

Eh_minus_sym = [Symbol('\\hat{E}^-_{' + str(n) + '}') for n in range(N+1)]
Eh_minus_eq = [Eh_minus_sym[n] for n in range(N+1)]

Fh_plus_sym = [Symbol('\\hat{F}^+_{' + str(n) + '}') for n in range(N+1)]
Fh_plus_eq = [Fh_plus_sym[n] for n in range(N+1)]

Fh_minus_sym = [Symbol('\\hat{F}^-_{' + str(n) + '}') for n in range(N+1)]
Fh_minus_eq = [Fh_minus_sym[n] for n in range(N+1)]


sigh_sym = [Symbol('\\hat{\sigma}_{' + str(n) + '}') for n in range(N+1)]
sigh_eq = [sigh_sym[n] for n in range(N+1)]

for n in range(N-1):
    print("n =", n)

    sigh_eq[n] = 0
    
    sigh_eq[n] += - deriv(sig_eq[n])
    
    sigh_eq[n] += sig_eq[n+1]
    
    for k in range(0, n+1):
        sigh_eq[n] += - sig_eq[k] * sig_eq[n-k]
    
    sigh_eq[n] += - q_coeffs[n] * (u + v) / 2
    
    sigh_eq[n] += - epsilon**2 * deriv(v) / 2 * r_eq[n]
    
    if n == 0:
        sigh_eq[n] += - u * (1 + epsilon**2 * v / 2) + (u + v) / 2

    Eh_plus_eq[n] = 0
    
    #Eh_plus_eq[n] += - deriv(E_plus_eq[n])
    
    #Eh_plus_eq[n] += E_plus_eq[n+1]
    
    for k in range(0, n+1):
        #Eh_plus_eq[n] += - E_plus_eq[k] * E_plus_eq[n-k] / 2
        pass
    for k in range(0, n+1):
        Eh_plus_eq[n] += F_plus_eq[k] * F_plus_eq[n-k] / 2
        pass
    for k in range(0, n+1):
        #Eh_plus_eq[n] += - E_plus_eq[k] * E_minus_eq[n-k] / 2
        pass
    for k in range(0, n+1):
        Eh_plus_eq[n] += + F_plus_eq[k] * F_minus_eq[n-k] / 2
        pass
    for k in range(0, n+1):
        #Eh_plus_eq[n] += - epsilon**2 * deriv(v) / 2 * re(r_eq[k] * h_plus[n-k])
        pass
    if n == 0:
        #Eh_plus_eq[n] += - u * (1 + epsilon**2 * v / 2)
        pass
    
    Eh_minus_eq[n] = 0
    
    Eh_minus_eq[n] += - deriv(E_minus_eq[n])
    
    #Eh_minus_eq[n] += E_minus_eq[n+1]
    
    for k in range(0, n+1):
        Eh_minus_eq[n] += - E_minus_eq[k] * E_minus_eq[n-k] / 2
        pass
    for k in range(0, n+1):
        Eh_minus_eq[n] += F_minus_eq[k] * F_minus_eq[n-k] / 2
        pass
    for k in range(0, n+1):
        Eh_minus_eq[n] += - E_minus_eq[k] * E_plus_eq[n-k] / 2
        pass
    for k in range(0, n+1):
        Eh_minus_eq[n] += + F_minus_eq[k] * F_plus_eq[n-k] / 2
        pass
    for k in range(0, n+1):
        Eh_minus_eq[n] += - epsilon**2 * deriv(v) / 2 * re(r_eq[k] * h_minus[n-k])
        pass
    if n == 0:
        Eh_minus_eq[n] += - u * (1 + epsilon**2 * v / 2) + (u + v)
        pass
        
    Fh_plus_eq[n] = 0
    
    Fh_plus_eq[n] += - deriv(F_plus_eq[n])
    
    #Fh_plus_eq[n] += F_plus_eq[n+1]
    
    for k in range(0, n+1):
        Fh_plus_eq[n] += - E_plus_eq[k] * F_plus_eq[n-k]
        pass
    for k in range(0, n+1):
        Fh_plus_eq[n] += - E_plus_eq[k] * F_minus_eq[n-k] / 2
        pass
    for k in range(0, n+1):
        Fh_plus_eq[n] += - F_plus_eq[k] * E_minus_eq[n-k] / 2
        pass
    for k in range(0, n+1):
        Fh_plus_eq[n] += - epsilon**2 * deriv(v) / 2 * im(r_eq[k] * h_plus[n-k])
        pass
    
    Fh_plus_eq[n] += (u + v) / 2 * epsilon**2 / 2 * im(h_plus[n+2])
    
    Fh_plus_eq[n] += - u * (1 + epsilon**2 * v / 2) * (im(h_plus[n]) if n > 0 else 0)
        

    Fh_minus_eq[n] = 0
    
    Fh_minus_eq[n] += - deriv(F_minus_eq[n])
    
    Fh_minus_eq[n] += F_minus_eq[n+1]
    
    for k in range(0, n+1):
        Fh_minus_eq[n] += - E_minus_eq[k] * F_minus_eq[n-k]
        pass
    for k in range(0, n+1):
        Fh_minus_eq[n] += - E_minus_eq[k] * F_plus_eq[n-k] / 2
        pass
    for k in range(0, n+1):
        Fh_minus_eq[n] += - F_minus_eq[k] * E_plus_eq[n-k] / 2
        pass
    for k in range(0, n+1):
        Fh_minus_eq[n] += - epsilon**2 * deriv(v) / 2 * im(r_eq[k] * h_minus[n-k])
        pass
    
    Fh_minus_eq[n] += - (u + v) / 2 * epsilon**2 / 2 * im(h_minus[n+2])
    
    Fh_minus_eq[n] += (- u * (1 + epsilon**2 * v / 2) + (u + v)) * (im(h_minus[n]) if n > 0 else 0)
        
    display(Eq(E_plus_sym[n], polynomize(E_plus_eq[n])))
    display(Eq(E_minus_sym[n], polynomize(E_minus_eq[n])))
    display(Eq(F_plus_sym[n], polynomize(F_plus_eq[n])))
    display(Eq(F_minus_sym[n], polynomize(F_minus_eq[n])))
    
    #display(Eq(Eh_plus_sym[n], polynomize(Eh_plus_eq[n])))
    #display(Eq(Eh_minus_sym[n], polynomize(Eh_minus_eq[n])))
    #display(Eq(Fh_plus_sym[n], polynomize(Fh_plus_eq[n])))
    #display(Eq(Fh_minus_sym[n], polynomize(Fh_minus_eq[n])))

    #display(Eq(sigh_sym[n], polynomize(sigh_eq[n])))



In [ ]:
def p_coeff(n):
    if n == -1:
        return simplify(epsilon / sqrt(2))
    if n >= 0:
        return simplify(1 / sqrt(2) * epsiloninv**(2*n+1) * Rational((-2)**(-n) * (factorial(2*n) / (factorial(n) * factorial(n+1)))))
    return 0

def q_coeff(n):
    if n % 2 == 0 or n < 0:
        return 0
    else:
        nn = (n - 1) // 2
        if nn % 2 == 0:
            return I**n * epsiloninv**n * catalan(nn) / (sqrt(2) * 2**nn)
        elif nn % 2 == 1:
            return - I**n * epsiloninv**n* catalan(nn) / (sqrt(2) * 2**nn)

for k in range(10):
    display(q_coeff(k))

In [ ]:
import pickle
import datetime


output_folder = "res/"

def p_coeff(n):
    if n == -1:
        return simplify(epsilon / sqrt(2))
    if n >= 0:
        return simplify(1 / sqrt(2) * epsiloninv**(2*n+1) * Rational((-2)**(-n) * (factorial(2*n) / (factorial(n) * factorial(n+1)))))
    return 0

def q_coeff(n):
    if n % 2 == 0 or n < 0:
        return 0
    else:
        nn = (n - 1) // 2
        if nn % 2 == 0:
            return I**n * epsiloninv**n * catalan(nn) / (sqrt(2) * 2**nn)
        elif nn % 2 == 1:
            return - I**n * epsiloninv**n* catalan(nn) / (sqrt(2) * 2**nn)

def a_coeff(n):
    return catalan(n) / 2**n
    
N = 7

r_sym = [Symbol('r_{' + str(n) + '}') for n in range(N+1)]
r_eq = [r_sym[n] for n in range(N+1)]
r_poly = [r_sym[n] for n in range(N+1)]

sig_sym = [Symbol('\sigma_{' + str(n) + '}') for n in range(N+1)]
sig_eq = [sig_sym[n] for n in range(N+1)]
sig_poly = [sig_sym[n] for n in range(N+1)]

E_sym = [Symbol('E_{' + str(n) + '}') for n in range(N+1)]
E_eq = [E_sym[n] for n in range(N+1)]
E_poly = [E_sym[n] for n in range(N+1)]

Et_sym = [Symbol('\\tilde{E}_{' + str(n) + '}') for n in range(N+1)]
Et_eq = [Et_sym[n] for n in range(N+1)]
Et_poly = [Et_sym[n] for n in range(N+1)]

variant_sign = 1

r_factor = 1 #Symbol('\lambda')
r_eq[0] = r_factor * (- I * sqrt(2) * epsiloninv)
r_eq[1] = r_factor * (u - epsiloninv**2 + (Rational(1, 2) + epsilon**2 * v**2 / 2) * r_eq[0]**2)
r_poly[0] = polynomize(r_eq[0])
r_poly[1] = polynomize(r_eq[1])

sig_eq[0] = epsilon / (2 * sqrt(2) * I) * (u + v)
sig_eq[1] = deriv(sig_eq[0]) + sig_eq[0]**2 + u * (1 + epsilon**2 * v / 2) - (u + v) / 2 + epsilon**2 * deriv(v) / 2 * r_eq[0]
sig_poly[0] = polynomize(sig_eq[0])
sig_poly[1] = polynomize(sig_eq[1])

E_eq[0] = re(sig_eq[0])
Et_eq[0] = im(sig_eq[0])
E_poly[0] = polynomize(E_eq[0])
Et_poly[0] = polynomize(Et_eq[0])

E_eq[1] = re(sig_eq[1]) - variant_sign * epsiloninv * sqrt(2) * Et_eq[0]
Et_eq[1] = im(sig_eq[1])
E_poly[1] = polynomize(E_eq[1])
Et_poly[1] = polynomize(Et_eq[1])

for n in range(1, N):
    
    print("n =", n)
    print(datetime.datetime.now())

    X = 0

    X += - deriv(r_eq[n]) + q_coeff(n) * epsiloninv**2
    #display(Eq(Symbol('X'), polynomize(X) + O(epsilon)))

    for k in range(0, n+1):
        X += (Rational(1, 2) + epsilon**2 * v**2 / 2) * r_eq[k] * r_eq[n-k]
    #display(Eq(Symbol('X'), polynomize(X) + O(epsilon)))

    for k in range(1, n+1):
        X += epsilon / (2 * sqrt(2) * I) * r_eq[k] * r_eq[n+1-k]
    #display(Eq(Symbol('X'), polynomize(X) + O(epsilon)))

    for k in range(0, n+1):
        for j in range(0, k+1):
            X += q_coeff(j) * r_eq[k-j] * r_eq[n-k] / 2
            
    #display(Eq(Symbol('X'), polynomize(X) + O(epsilon)))

    r_eq[n+1] = r_factor * X
    r_poly[n+1] = polynomize(r_eq[n+1])
    r_eq[n+1] = depolynomize(r_poly[n+1])
    
    X = 0

    X += deriv(sig_eq[n]) + q_coeff(n) / 2 * (u + v)
    
    for k in range(0, n+1):
        X += sig_eq[k] * sig_eq[n-k]

    X += epsilon**2 * deriv(v) / 2 * r_eq[n]

    sig_eq[n+1] = X
    sig_poly[n+1] = polynomize(sig_eq[n+1])
    sig_eq[n+1] = depolynomize(sig_poly[n+1])

    E_eq[n+1] = re(sig_eq[n+1]) - variant_sign * epsiloninv * sqrt(2) * Et_eq[n]
    E_poly[n+1] = polynomize(E_eq[n+1])
    E_eq[n+1] = depolynomize(E_poly[n+1])

    Et_eq[n+1] = im(sig_eq[n+1])
    for k in range(n-1, -1, -2):
        Et_eq[n+1] += Et_eq[k] * epsiloninv**(n+1-k) * a_coeff((n+1-k)//2 - 1)
    Et_poly[n+1] = polynomize(Et_eq[n+1])
    Et_eq[n+1] = depolynomize(Et_poly[n+1])

    
for n in range(0, N+1):
    display(Eq(sig_sym[n], sig_poly[n]))
    #display(Eq(re(r_sym[n]), re(r_poly[n] + O(epsilon**2))))
    #display(Eq(re(sig_sym[n]), re(sig_poly[n] + O(epsilon**2))))
    display(Eq(E_sym[n], E_poly[n]))


In [ ]:
energy = E_poly[7].coeffs()[-1]
monomial = Add.make_args(energy)[0]
display(Add.make_args(monomial))

In [ ]:
energy = sig_eq[6]

def reduce(expr, sym):
    print("REDUCE: ", str(expr))
    reduction = expr
    summands = Add.make_args(expr)
    for summand in expr.args:
        for factor in summand.args:
            print("Summand: ", str(summand), "Factor: ", str(factor))
            coefficient = 1
            if str(sym) not in str(factor):
                coefficient = factor
                print("coefficient: ", coefficient)
            if str(sym) in str(factor) and '{x' in str(factor) and '**' not in str(factor):
                display(str(factor))
                
                factor_integrated = Symbol(str(factor).replace('x', '', 1).replace('('+str(sym)+')_{}', str(sym), 1), real=True)
                print("integrand: ", str(factor_integrated))
                rest_deriv = deriv(simplify(summand / factor))
                print("deriv: ", str(rest_deriv))
                if rest_deriv == 0:
                    reduction = simplify(reduction - summand)
                    return reduction
                else:
                    new = - factor_integrated * rest_deriv
                    print("new: ", str(new))
                    for new_summand in Add.make_args(new):
                        new_coefficient = 1
                        if str(sym) not in str(new_summand.args[0]):
                                new_coefficient = new_summand.args[0]
                                print("New coefficient: ", new_coefficient)
                                
                        for other_summand in Add.make_args(expr):
                            print("Other summand: ", other_summand)
                            other_coefficient = 1
                            if str(sym) not in str(other_summand.args[0]):
                                other_coefficient = other_summand.args[0]
                                print("Other coefficient: ", other_coefficient)

                            print("Compare: ", other_summand / other_coefficient, "with", new_summand / new_coefficient)
                            print("Find: ", simplify(other_summand / other_coefficient - new_summand / new_coefficient))
                            if simplify(other_summand / other_coefficient - new_summand / new_coefficient) == 0:
                                print("We have a match: ")
                                print(other_summand / other_coefficient, other_coefficient)

                                reduction = simplify(reduction - new / new_coefficient * other_coefficient)
                                return reduction
                                
energy = reduce(energy, Symbol('v', real=True))
energy = reduce(energy, Symbol('v', real=True))
energy = reduce(energy, Symbol('v', real=True))
energy = reduce(energy, Symbol('v', real=True))
energy = reduce(energy, Symbol('v', real=True))
energy = reduce(energy, Symbol('v', real=True))
energy = reduce(energy, Symbol('v', real=True))
energy = reduce(energy, Symbol('v', real=True))


In [ ]:
import pickle
import datetime


output_folder = "res/"

def p_coeff(n):
    if n == -1:
        return simplify(epsilon / sqrt(2))
    if n >= 0:
        return simplify(1 / sqrt(2) * epsiloninv**(2*n+1) * Rational((-2)**(-n) * (factorial(2*n) / (factorial(n) * factorial(n+1)))))
    return 0

def q_coeff(n):
    if n % 2 == 0 or n < 0:
        return 0
    else:
        nn = (n - 1) // 2
        if nn % 2 == 0:
            return I**n * epsiloninv**n * catalan(nn) / (sqrt(2) * 2**nn)
        elif nn % 2 == 1:
            return - I**n * epsiloninv**n * catalan(nn) / (sqrt(2) * 2**nn)
        
def qt_coeff(n):
    if n % 2 == 0 or n < 0:
        return 0
    else:
        nn = (n - 1) // 2
        if nn % 2 == 0:
            return (-1)**n * epsiloninv**n * catalan(nn) / (sqrt(2) * 2**nn)
        elif nn % 2 == 1:
            return - (-1)**n * epsiloninv**n * catalan(nn) / (sqrt(2) * 2**nn)
        
        
def a_coeff(n):
    return catalan(n) / 2**n
    
N = 18

r_sym = [Symbol('r_{' + str(n) + '}') for n in range(N+1)]
r_eq = [r_sym[n] for n in range(N+1)]
r_poly = [r_sym[n] for n in range(N+1)]

R_sym = [Symbol('R_{' + str(n) + '}') for n in range(N+1)]
R_eq = [R_sym[n] for n in range(N+1)]
R_poly = [R_sym[n] for n in range(N+1)]

r_factor = 1 
lam, mu = symbols('\lambda, mu')

r_eq[0] = r_factor * (- I * sqrt(2) * epsiloninv)
r_eq[1] = r_factor * (u - epsiloninv**2 + (Rational(1, 2) + epsilon**2 * v**2 / 2) * r_eq[0]**2)
r_poly[0] = polynomize(r_eq[0])
r_poly[1] = polynomize(r_eq[1])

R_eq[0] = sqrt(2) * epsilon
R_eq[1] = 2 #epsilon**2 * u + 1 + (Rational(1, 2) * epsiloninv**2 - v**2 / 2) * R_eq[0]**2
R_poly[0] = polynomize(R_eq[0])
R_poly[1] = polynomize(R_eq[1])


for n in range(1, N):
    
    print("n =", n)
    print(datetime.datetime.now())

    #X = 0
    #
    #X += - deriv(r_eq[n]) + q_coeff(n) * epsiloninv**2
    #
    #for k in range(0, n+1):
    #    X += (Rational(1, 2) + epsilon**2 * v**2 / 2) * r_eq[k] * r_eq[n-k]
    #
    #for k in range(1, n+1):
    #    X += epsilon / (2 * sqrt(2) * I) * r_eq[k] * r_eq[n+1-k]
    #
    #for k in range(0, n+1):
    #    for j in range(0, k+1):
    #        X += q_coeff(j) * r_eq[k-j] * r_eq[n-k] / 2
    #        
    #r_eq[n+1] = r_factor * X
    #r_poly[n+1] = polynomize(r_eq[n+1])
    #r_eq[n+1] = depolynomize(r_poly[n+1])
    
    #display(Eq(r_sym[n+1], r_poly[n+1]))
    
    X = 0

    #X += - deriv(R_eq[n]) 
    
    X += - qt_coeff(n)
    
    #display(Eq(Symbol('A'), polynomize(- qt_coeff(n))))
    
    Y = 0
    for k in range(0, n+1):
        X += (epsiloninv**2 / 2 - v**2 / 2) * R_eq[k] * R_eq[n-k]
        Y += (epsiloninv**2 / 2 - v**2 / 2) * R_eq[k] * R_eq[n-k]
        
    display(Eq(Symbol('B'), polynomize(Y) + O(epsilon)))
    
    Y = 0
    for k in range(1, n+1):
        X -= epsiloninv / (2 * sqrt(2)) * R_eq[k] * R_eq[n+1-k]
        Y -= epsiloninv / (2 * sqrt(2)) * R_eq[k] * R_eq[n+1-k]

    display(Eq(Symbol('C'), polynomize(Y) + O(epsilon)))

    Y = 0
    for k in range(0, n+1):
        for j in range(0, k+1):
            X += epsiloninv**2 / 2 * qt_coeff(j) * R_eq[k-j] * R_eq[n-k]
            Y += epsiloninv**2 / 2 * qt_coeff(j) * R_eq[k-j] * R_eq[n-k]
            
    display(Eq(Symbol('D'), polynomize(Y) + O(epsilon)))

    R_eq[n+1] = r_factor * X
    R_poly[n+1] = polynomize(R_eq[n+1])
    R_eq[n+1] = depolynomize(R_poly[n+1])
    
    display(Eq(R_sym[n+1], R_poly[n+1] + O(epsilon)))


In [ ]:
#pickle.dump([polynomize(s) for s in sig_eq], open("data/poly_sig_eq_" + str(7) + ".p", "wb"))
#pickle.dump([polynomize(r) for r in r_eq], open("data/poly_r_eq_" + str(7) + ".p", "wb"))


for n in range(N+1):
    
    display(Eq(sig_sym[n], polynomize(sig_eq[n])))

        
    if n == 0:
        F_eq[0] = re(sig_eq[0])
        display(Eq(F_sym[0], polynomize(F_eq[0])))
        
        Ft_eq[0] = im(sig_eq[0])
        display(Eq(Ft_sym[0], polynomize(Ft_eq[0])))
        
    if n == 1:
        F_eq[1] = re(sig_eq[1]) - epsiloninv * sqrt(2) * Ft_eq[0]
        display(Eq(F_sym[1], polynomize(F_eq[1])))
        
        Ft_eq[1] = im(sig_eq[1])
        display(Eq(Ft_sym[1], polynomize(Ft_eq[1])))
        
    if n == 2:
        F_eq[2] = re(sig_eq[2]) - epsiloninv * sqrt(2) * Ft_eq[1]
        display(Eq(F_sym[2], polynomize(F_eq[2])))
        
        Ft_eq[2] = im(sig_eq[2]) + epsiloninv**2 * Ft_eq[0]
        display(Eq(Ft_sym[2], polynomize(Ft_eq[2])))
       
    if n == 3:
        F_eq[3] = re(sig_eq[3]) - epsiloninv * sqrt(2) * Ft_eq[2]
        display(Eq(F_sym[3], polynomize(F_eq[3])))
        
        Ft_eq[3] = im(sig_eq[3]) + epsiloninv**2 * Ft_eq[1]
        display(Eq(Ft_sym[3], polynomize(Ft_eq[3])))
        
    if n == 4:
        F_eq[4] = re(sig_eq[4]) - epsiloninv * sqrt(2) * Ft_eq[3]
        display(Eq(F_sym[4], polynomize(F_eq[4])))
        
        Ft_eq[4] = im(sig_eq[4]) + epsiloninv**2 * Ft_eq[2] + epsiloninv**4 / 2 * Ft_eq[0]
        display(Eq(Ft_sym[4], polynomize(Ft_eq[4])))
        
    if n == 5:
        F_eq[5] = re(sig_eq[5]) - epsiloninv * sqrt(2) * Ft_eq[4]
        display(Eq(F_sym[5], polynomize(F_eq[5])))
        
        Ft_eq[5] = im(sig_eq[5]) + epsiloninv**2 * Ft_eq[3] + epsiloninv**4 / 2 * Ft_eq[1]
        display(Eq(Ft_sym[5], polynomize(Ft_eq[5])))
        
    if n == 6:
        F_eq[6] = re(sig_eq[6]) - epsiloninv * sqrt(2) * Ft_eq[5]
        display(Eq(F_sym[6], polynomize(F_eq[6])))
        
        Ft_eq[6] = im(sig_eq[6]) + epsiloninv**2 * Ft_eq[4] + epsiloninv**4 / 2 * Ft_eq[2] + epsiloninv**6 / 2 * Ft_eq[0]
        display(Eq(Ft_sym[6], polynomize(Ft_eq[6])))
        
    if n == 7:
        F_eq[7] = re(sig_eq[7]) - epsiloninv * sqrt(2) * Ft_eq[6]
        display(Eq(F_sym[7], polynomize(F_eq[7])))
        
        Ft_eq[7] = im(sig_eq[7]) + epsiloninv**2 * Ft_eq[5] + epsiloninv**4 / 2 * Ft_eq[3] + epsiloninv**6 / 2 * Ft_eq[1]
        display(Eq(Ft_sym[7], polynomize(Ft_eq[7])))
        
    #display(Eq(r_sym[n], polynomize(r_eq[n])))



In [ ]:
X = Symbol('X')

epsilon = Symbol('\\epsilon', positive=True)

eqn = 1 / (2 + sqrt(1 - epsilon**2 / (2 * X**2)) + 1 / sqrt(1 - epsilon**2 / (2 * X**2)))

ser = series(eqn, x0=0, x=X, n=None)

display(simplify(next(ser)))

for n in range(15):
    print("n =", n)
    display(simplify(next(ser)))
    #display(polynomize(q_coeff(2*n+1)))
    #display(polynomize(epsiloninv**(2*n+1) * I / (sqrt(2)**(2*n+1)) * catalan(n)))

In [ ]:

def sqrtt(x):
    return I*sqrt(-x)

X = Symbol('X')

epsilon = Symbol('epsilon', positive=True)

eqn = sqrtt(- (u + (sqrtt(1 - epsilon**2 / 2 / X**2) - 1) / epsilon**2) / (1 + epsilon**2 * v + sqrtt(1 - epsilon**2 / 2 / X**2)))
#eqn = 1 / (1 - sqrt(1 - epsilon**2 / 2 * X**2))
ser = series(eqn, x0 = 0, x=X, n=None)

for j in range(15):
    display(simplify(next(ser)))


In [ ]:
def L(n):
    if n == 0:
        return - I
    if n == 1: 
        return sqrt(2) / 4
    if n >= 2 and n % 2 == 0:
        nn = n // 2
        return I * catalan(nn-1) / sqrt(2)**(5*n-2)
    return 0

def Lt(n):
    if n == 0:
        return - I
    if n == 1: 
        return - sqrt(2) / 4
    if n >= 2 and n % 2 == 0:
        nn = n // 2
        return I * catalan(nn-1) / sqrt(2)**(5*n-2)
    return 0

def G(n):
    if n == 0:
        return 1
    if n % 2 == 1 and n >= 0:
        return - binomial(2 * (n//2), n//2) * sqrt(2) * I * epsiloninv**n / 2**(n//2)
    else:
        return 0
    
def sqrtt(x):
    return I*sqrt(-x)

epsilon = Symbol('epsilon', positive=True)

X = Symbol('X')
theta = Symbol('theta')

eqn = 1 - 1 / (epsilon / (sqrt(2) * I) * X * sqrt(1 - 2 / (epsilon**2 * X**2)))
ser = series(eqn, x0 = oo, x=X, n=None)
for n in range(-1,11):
    print("n =", n)
    display(next(ser))
    if n == -1:
        display(polynomize(G(0)))
    else:
        display(polynomize(G(2*n+1)))
    
for n in range(-5, 5):
    display(binomial(Rational(1,2), n) * (-epsilon**2/2)**n)
    
for n in range(-1, 14):
    display(polynomize(I * (-1)**n * p_coeff(n)))
    pass

eqn = I / sqrt(1 + 2 / (sqrtt(1 + 2 * (4) * theta**2) - 1))
ser = series(eqn, x0 = oo, x=theta, n=17)
#display(Eq(eqn, ser))

eqn2 = I / sqrtt(1 - 2 / (sqrtt(1 + 2 * (4) * theta**2) + 1))
ser2 = series(eqn2, x0 = oo, x=theta, n=17)
#display(Eq(eqn2, ser2))

for n in range(14):
    #display(L(n))
    pass



In [ ]:
def binomial2(n, k):
    return factorial2(n) / (factorial2(k) * factorial2(n - k))

def PPP(l, n):
    if l == 0:
        if n == 0:
            return -1
        return 0
    return (-1)**((l-1)//2 + (n+1)//2) * (2 * sqrt(2))**(l - n) * binomial(-Rational(1,2) - n//2, l - n)

def PPPt(n, k):
    if n - k < 0:
        return 0
    if n - k == 0:
        return -1
    else:
        return (-1)**(1+(n-1)//2+(k-1)//2) * (2 * sqrt(2))**(n-k) * binomial(n - 1 - k//2, (k-1)//2)

def BB3(n, s, k):
    if k % 2 == 0:
        return (-1)**(1+s//2+(n+1)//2) * (2 * sqrt(2))**(n-s) * binomial(n - (s+1)//2, s//2)
    if k % 2 == 1:
        return (-1)**(1+(s-1)//2+(n+1)//2) * (2 * sqrt(2))**(n-s) * binomial(n - s//2 - 1, (s-1)//2)
    
def BBt3(n, s, k):
    if k % 2 == 0:
        return (-1)**(1+(s-1)//2+(n+1)//2) * (2 * sqrt(2))**(n-s) * binomial(n - s//2 - 1, (s-1)//2)
    if k % 2 == 1:
        return (-1)**(1+s//2+(n+1)//2) * (2 * sqrt(2))**(n-s) * binomial(n - (s+1)//2, s//2)

def BBtt3(n, s, k):
    if k % 2 == 0:
        return (-1)**(s//2+n//2) * (2 * sqrt(2))**(n-s) * binomial2(2*(n - (s+1)//2) - 1, 2*(s//2) - 1)
    if k % 2 == 1:
        return (-1)**((s+1)//2+n//2) * (2 * sqrt(2))**(n-s) * binomial2(2*(n - s//2) - 1, 2*((s+1)//2) - 1)

def BBttt3(n, s, k):
    if k % 2 == 0:
        return (-1)**((s+1)//2+n//2) * (2 * sqrt(2))**(n-s) * binomial2(2*(n - s//2) - 1, 2*((s+1)//2) - 1)
    if k % 2 == 1:
        return (-1)**(s//2+n//2) * (2 * sqrt(2))**(n-s) * binomial2(2*(n - (s+1)//2) - 1, 2*(s//2) - 1)

def JJ(n, s):
    return PPPt(n, s-1)
    return (-1)**((n-1)//2 + s//2) * (2 * sqrt(2))**(n+1-s) *  binomial(n - (s+1)//2, s//2 - 1)

def JJt(n, s):
    return PPP(n, s-1)
    if s == 0:
        return - (-1)**(n//2 + (s+1)//2) / sqrt(2)**(n-1-s) * catalan(n)
    return (-1)**(n//2 + (s+1)//2) / sqrt(2)**(n+1-s) * binomial(n-s//2, (s-1)//2) * binomial(2*(n-s//2), n-s//2) / binomial(2*((s-1)//2), (s-1)//2)

def L(n):
    if n == 0:
        return I
    if n == 1: 
        return - sqrt(2)
    if n >= 2 and n % 2 == 0:
        nn = n // 2
        return - I * catalan(nn-1) / 2**(nn-1)
    return 0

N = 5

lam = Symbol('\lambda', real=True)

F_sym = [Symbol('F_{' + str(n) + '}') for n in range(N+1)]
Ft_sym = [Symbol('\\tilde{F}_{' + str(n) + '}') for n in range(N+1)]

F_eq = [F_sym[n] for n in range(N+1)]
Ft_eq = [Ft_sym[n] for n in range(N+1)]

E_sym = [Symbol('E_{' + str(n) + '}') for n in range(N+1)]
Et_sym = [Symbol('\\tilde{E}_{' + str(n) + '}') for n in range(N+1)]

E_eq = [E_sym[n] for n in range(N+1)]
Et_eq = [Et_sym[n] for n in range(N+1)]

E_alt = [E_sym[n] for n in range(N+1)]
Et_alt = [Et_sym[n] for n in range(N+1)]


H_sym = [Symbol('H_{' + str(n) + '}') for n in range(N+1)]
Ht_sym = [Symbol('\\tilde{H}_{' + str(n) + '}') for n in range(N+1)]

H_eq = [H_sym[n] for n in range(N+1)]
Ht_eq = [Ht_sym[n] for n in range(N+1)]

omega_sym = [Symbol('\\omega_{' + str(n) + '}') for n in range(N+1)]
omegat_sym = [Symbol('\\tilde{\\omega}_{' + str(n) + '}') for n in range(N+1)]

omega_eq = [omega_sym[n] for n in range(N+1)]
omegat_eq = [omegat_sym[n] for n in range(N+1)]

H_alt = [H_sym[n] for n in range(N+1)]
Ht_alt = [Ht_sym[n] for n in range(N+1)]

r_sym = [Symbol('r_{' + str(n) + '}') for n in range(N+1)]
r_eq = [r_sym[n] for n in range(N+1)]

R_sym = [Symbol('R_{' + str(n) + '}') for n in range(N+1)]
R_eq = [R_sym[n] for n in range(N+1)]

Rt_sym = [Symbol('\\tilde{R}_{' + str(n) + '}') for n in range(N+1)]
Rt_eq = [Rt_sym[n] for n in range(N+1)]

r_eq[0] = - sqrt(2) * I * epsiloninv
r_eq[1] = u - v - 2 * epsiloninv**2

R_eq[0] = lam * im(r_eq[0])
R_eq[1] = lam * re(r_eq[1])

Rt_eq[0] = 0
Rt_eq[1] = 0

F_eq[0] = - epsilon / (2 * sqrt(2)) * (u + v)
F_eq[1] = (u - v) / 2 + epsilon**2 * (u * v / 2 - (u + v)**2 / 8)
Ft_eq[0] = 0
Ft_eq[1] = - epsilon / (2 * sqrt(2)) * deriv(u + v + lam * 2 * v)

E_eq[0] = F_eq[0]
E_eq[1] = F_eq[1] + epsiloninv * PPP(1, 0) * E_eq[0]
Et_eq[0] = 0
Et_eq[1] = Ft_eq[1] + epsiloninv * PPPt(1, 0) * Et_eq[0]

H_eq[0] = Et_eq[0]
H_eq[1] = E_eq[1]
Ht_eq[0] = E_eq[0]
Ht_eq[1] = Et_eq[1]

S_sym = [Symbol('S_{' + str(n) + '}') for n in range(N+1)]

S_eq = [S_sym[n] for n in range(N+1)]

S_eq[0] = 0
S_eq[1] = u

for n in range(1, N):
    X = deriv(S_eq[n])
    
    for k in range(n+1):
        X += S_eq[k] * S_eq[n-k]
    
    #S_eq[n+1] = X


for n in range(1, N):
    print(n)
    
    X = 0
    
    X -= deriv(r_eq[n])
    
    for k in range(0, n+1):
        X += (1 + epsilon**2 / 2 * v) * r_eq[k] * r_eq[n-k]
    
    for k in range(1, n+1):
        X -= I * epsilon / (2 * sqrt(2)) * r_eq[k] * r_eq[n+1-k]
    
    r_eq[n+1] = X
    
    if (n+1) % 2 == 0:
        R_eq[n+1] = lam * im(r_eq[n+1])
        Rt_eq[n+1] = lam * re(r_eq[n+1])
    elif (n+1) % 2 == 1:
        R_eq[n+1] = lam * re(r_eq[n+1])
        Rt_eq[n+1] = lam * im(r_eq[n+1])
        
    X = 0
    
    X += epsilon**2 / 2 * deriv(v) * Rt_eq[n]
    
    for k in range(0, n+1):
        X += epsiloninv**(n+1-k) * PPP(n+1, k) * E_eq[k] - epsiloninv**(n-k) * PPPt(n, k) * deriv(Et_eq[k])
                
        for j in range(k, n+1):
            for m in range(0, n-j+1):
                X += epsiloninv**(n - k - m) * (n%2 + ((n+1)%2)*(-1)**j) * PPPt(j, k) * PPPt(n-j, m) * Et_eq[k] * Et_eq[m]
                pass
            
            for m in range(0, n-j+1):
                X += epsiloninv**(n - k - m) * (n%2 - ((n+1)%2)*(-1)**j) * PPP(j, k) * PPP(n-j, m) * E_eq[k] * E_eq[m]
                pass
            
        for j in range(1, n+2-k): 
            X -= epsiloninv**(n+1-k) * E_eq[k] * 2 * catalan(j-1) / sqrt(2)**j * (-1)**((j+n%2)//2) * PPP(n+1-j, k)
            pass
        
    E_eq[n+1] = X
    
    X = 0
    
    X += epsilon**2 / 2 * deriv(v) * R_eq[n]
    
    for k in range(0, n+1):
        X += epsiloninv**(n+1-k) * PPPt(n+1, k) * Et_eq[k] - epsiloninv**(n-k) * PPP(n, k) * deriv(E_eq[k])
        
        for j in range(k, n+1):
            for m in range(0, n-j+1):
                X += epsiloninv**(n - k - m) * ((n+1)%2 + (n%2)*(-1)**j) * PPPt(j, k) * PPP(n-j, m) * Et_eq[k] * E_eq[m]
                pass
            
            for m in range(0, n-j+1):
                X += epsiloninv**(n - k - m) * ((n+1)%2 - (n%2)*(-1)**j) * PPP(j, k) * PPPt(n-j, m) * E_eq[k] * Et_eq[m]
                pass
            
        for j in range(1, n+2-k): 
            X -= epsiloninv**(n+1-k) * Et_eq[k] * 2 * catalan(j-1) / sqrt(2)**j * (-1)**((j+(n+1)%2)//2) * PPPt(n+1-j, k)
            pass    
    
    Et_eq[n+1] = X
    
    
for n in range(0, N // 2):
    
    
    X = 0
    
    X += epsilon**2 * deriv(v) / 2 * R_eq[2*n+1]
    
    for s in range(0, n+1):
        X += epsiloninv**(2*n+2-2*s) * JJt(2*n+1, 2*s) * H_eq[2*s]
        X -= epsiloninv**(2*n+1-2*s) * PPP(2*n+1, 2*s) * deriv(Ht_eq[2*s])
        
        for k in range(0, 2*s+1):
            X += 2 * BBtt3(2*n+1, 2*s, 0) * epsiloninv**(2*n+1-2*s) * H_eq[k] * Ht_eq[2*s-k]
            
    for s in range(0, n+1):
        X += epsiloninv**(2*n+2-2*s-1) * JJt(2*n+1, 2*s+1) * Ht_eq[2*s+1]
        X -= epsiloninv**(2*n+1-2*s-1) * PPP(2*n+1, 2*s+1) * deriv(H_eq[2*s+1])
        
        for k in range(0, 2*s+2):
            X += BBtt3(2*n+1, 2*s+1, 0) * epsiloninv**(2*n+1-2*s-1) * H_eq[k] * H_eq[2*s+1-k]
            X += BBttt3(2*n+1, 2*s+1, 0) * epsiloninv**(2*n+1-2*s-1) * Ht_eq[k] * Ht_eq[2*s+1-k]
        
    if 2*n+2 <= N:
        H_eq[2*n+2] = X

    
    X = 0
    
    X += epsilon**2 * deriv(v) / 2 * Rt_eq[2*n+1]
    
    for s in range(0, n+1):
        X += epsiloninv**(2*n+2-2*s) * JJ(2*n+1, 2*s) * Ht_eq[2*s]
        X -= epsiloninv**(2*n+1-2*s) * PPPt(2*n+1, 2*s) * deriv(H_eq[2*s])
        
        for k in range(0, 2*s+1):
            X += BB3(2*n+1, 2*s, 0) * epsiloninv**(2*n+1-2*s) * Ht_eq[k] * Ht_eq[2*s-k]
            X += BBt3(2*n+1, 2*s, 0) * epsiloninv**(2*n+1-2*s) * H_eq[k] * H_eq[2*s-k]
        
    for s in range(0, n+1):
        X += epsiloninv**(2*n+2-2*s-1) * JJ(2*n+1, 2*s+1) * H_eq[2*s+1]
        X -= epsiloninv**(2*n+1-2*s-1) * PPPt(2*n+1, 2*s+1) * deriv(Ht_eq[2*s+1])
        
        for k in range(0, 2*s+2):
            X += 2 * BB3(2*n+1, 2*s+1, 0) * epsiloninv**(2*n+1-2*s-1) * Ht_eq[k] * H_eq[2*s+1-k]
       
    if 2*n+2 <= N:
        Ht_eq[2*n+2] = X
    
    
    X = 0
    
    X += epsilon**2 * deriv(v) / 2 * Rt_eq[2*n+2]
    
    for s in range(0, n+2):
        X += epsiloninv**(2*n+3-2*s) * JJ(2*n+2, 2*s) * Ht_eq[2*s]
        X -= epsiloninv**(2*n+2-2*s) * PPPt(2*n+2, 2*s) * deriv(H_eq[2*s])
        
        for k in range(0, 2*s+1):
            X += BB3(2*n+2, 2*s, 0) * epsiloninv**(2*n+2-2*s) * Ht_eq[k] * Ht_eq[2*s-k]
            X += BBt3(2*n+2, 2*s, 0) * epsiloninv**(2*n+2-2*s) * H_eq[k] * H_eq[2*s-k]
        
    for s in range(0, n+1):
        X += epsiloninv**(2*n+3-2*s-1) * JJ(2*n+2, 2*s+1) * H_eq[2*s+1]
        X -= epsiloninv**(2*n+2-2*s-1) * PPPt(2*n+2, 2*s+1) * deriv(Ht_eq[2*s+1])
        
        for k in range(0, 2*s+2):
            X += 2 * BB3(2*n+2, 2*s+1, 0) * epsiloninv**(2*n+2-2*s-1) * Ht_eq[k] * H_eq[2*s+1-k]
        
    if 2*n+3 <= N:
        H_eq[2*n+3] = X
    
    
    X = 0
    
    X += epsilon**2 * deriv(v) / 2 * R_eq[2*n+2]
    
    for s in range(0, n+2):
        X += epsiloninv**(2*n+3-2*s) * JJt(2*n+2, 2*s) * H_eq[2*s]
        X -= epsiloninv**(2*n+2-2*s) * PPP(2*n+2, 2*s) * deriv(Ht_eq[2*s])
        
        for k in range(0, 2*s+1):
            X += 2 * BBtt3(2*n+2, 2*s, 0) * epsiloninv**(2*n+2-2*s) * H_eq[k] * Ht_eq[2*s-k]
            
    for s in range(0, n+1):
        X += epsiloninv**(2*n+3-2*s-1) * JJt(2*n+2, 2*s+1) * Ht_eq[2*s+1]
        X -= epsiloninv**(2*n+2-2*s-1) * PPP(2*n+2, 2*s+1) * deriv(H_eq[2*s+1])
        
        for k in range(0, 2*s+2):
            X += BBtt3(2*n+2, 2*s+1, 0) * epsiloninv**(2*n+2-2*s-1) * H_eq[k] * H_eq[2*s+1-k]
            X += BBttt3(2*n+2, 2*s+1, 0) * epsiloninv**(2*n+2-2*s-1) * Ht_eq[k] * Ht_eq[2*s+1-k]
        
    if 2*n+3 <= N:
        Ht_eq[2*n+3] = X
    


    
for n in range(0, N+1):
    
    omega_eq[n] = H_eq[n]
    for k in range(0, n+1):
        omega_eq[n] += L(k) * epsiloninv**k * Ht_eq[n-k]
    
    omegat_eq[n] = H_eq[n]
    for k in range(0, n+1):
        omegat_eq[n] -= L(k) * epsiloninv**k * Ht_eq[n-k]
    
    #display(Eq(r_sym[n], polynomize(r_eq[n])))
    
    #display(Eq(E_sym[n], polynomize(E_eq[n])))
    #display(Eq(Et_sym[n], polynomize(Et_eq[n])))

    display(Eq(H_sym[n], polynomize(H_eq[n])))
    display(Eq(Ht_sym[n], polynomize(Ht_eq[n])))
    
    display(Eq(omega_sym[n], polynomize(omega_eq[n])))
    display(Eq(omegat_sym[n], polynomize(omegat_eq[n])))

    #display(Eq(S_sym[n], simplify(S_eq[n])))


In [ ]:
import pickle

factors = pickle.load(open("factors_9.p", "rb"))

def binomial2(n, k):
    return factorial2(n) / (factorial2(k) * factorial2(n - k))

def PP(l, n):
    return (-1)**(l//2 + n//2) * (sqrt(2) * 2)**(l - n) * binomial(-Rational(1,2) + l//2 - n, l - n)

def PP2(l, n):
    if l//2 >= n:
        return (-1)**(l + n//2) * (2 * sqrt(2))**(l - n) * binomial2(2 * (l - n), 2 * (l//2 - n) - 1)**(-1) / (2 * (l - l//2) + 1)
    
    elif l//2 < n:
        return (-1)**(1 + (l+1)//2 + (n-1)//2) * (2 * sqrt(2))**(l - n) * binomial2(2 * (l - l//2) - 1, 2 * (n - l//2) - 1)
    

def QQQ(l, n):
    return (-1)**((l-1)//2 + (n+1)//2) * epsiloninv**(l - n) * (2 * sqrt(2))**(l - n) * binomial(-Rational(1,2) - n//2, l - n)

def PPP(l, n):
    if l - n < 0:
        return 0
    if l - n == 0:
        return -1
    else:
        return (-1)**((l-1)//2 + (n+1)//2) * (2 * sqrt(2))**(l - n) * binomial(-Rational(1,2) - n//2, l - n)

def PPPt(n, k):
    if n - k < 0:
        return 0
    if n - k == 0:
        return -1
    else:
        return (-1)**(1+(n-1)//2+(k-1)//2) * (2 * sqrt(2))**(n-k) * binomial(n - 1 - k//2, (k-1)//2)

def PPPt3(n, k):
    if n == k:
        return -1

    def auxilliary(n, k):
        if n == 0:
            return 1
        if k <= n and k >= 0:
            A = 0
            for j in range(0, min(k, n-k)+1):
                A += (-1)**j * 2**(n-j-1) * binomial(k, j) * binomial(n-j, k)
            return A
        else:
            return 0
    A = 0
    for j in range(n-k, n):
        A += (-1)**(1+j+n//2+(k-1)//2) * sqrt(2)**(n-k+2) * binomial(n-1, j) * auxilliary(j, n-k)
    return A
    
def PPPt2(n, k):
    return (-1)**(n*(k+1)) * PPP(n - (k+1)%2, k - (k+1)%2)

def CC(l, n): 
    return 2 * catalan(l - 1 - n) / sqrt(2)**(l - n) * (-1)**((l - n + (l-1)%2) // 2)

def AA(l, n):
    return 2 * catalan(l - n) / sqrt(2)**(l - n + 1) * (-1)**((l - n + 1 + l%2) // 2)

def DD(l, n):
    return (- PP(l+1, n) - CC(l+1, n)) / (- PP(l+1, l) - CC(l+1, l))


def SS(l, k):
    return binomial(2 * l - k, l - k//2) * sqrt(2)**(k//2-l) * (-1)**(l//2 + k//4)

def BB(n, s, k):
    S = 0
    for i in range(0, n-s+1):
        S += (n%2 - ((n+1)%2) * (-1)**(i+k)) * PPP(i+k, k) * PPP(n-i-k,s-k)
    return S

def BBt(n, s, k):
    S = 0
    for i in range(0, n-s+1):
        S += (n%2 + ((n+1)%2) * (-1)**(i+k)) * PPPt(i+k, k) * PPPt(n-i-k,s-k)
    return S

def BBtt(n, s, k):
    S = 0
    for i in range(0, n-s+1):
        S += ((n+1)%2 + (n%2) * (-1)**(i+k)) * PPPt(i+k, k) * PPP(n-i-k,s-k)
    return S

def BBttt(n, s, k):
    S = 0
    for i in range(0, n-s+1):
        S += ((n+1)%2 - (n%2) * (-1)**(i+k)) * PPP(i+k, k) * PPPt(n-i-k,s-k)
    return S

def BB3(n, s, k):
    if k % 2 == 0:
        return (-1)**(1+s//2+(n+1)//2) * (2 * sqrt(2))**(n-s) * binomial(n - (s+1)//2, s//2)
    if k % 2 == 1:
        return (-1)**(1+(s-1)//2+(n+1)//2) * (2 * sqrt(2))**(n-s) * binomial(n - s//2 - 1, (s-1)//2)
    
def BBt3(n, s, k):
    if k % 2 == 0:
        return (-1)**(1+(s-1)//2+(n+1)//2) * (2 * sqrt(2))**(n-s) * binomial(n - s//2 - 1, (s-1)//2)
    if k % 2 == 1:
        return (-1)**(1+s//2+(n+1)//2) * (2 * sqrt(2))**(n-s) * binomial(n - (s+1)//2, s//2)

def BBtt3(n, s, k):
    if k % 2 == 0:
        return (-1)**(s//2+n//2) * (2 * sqrt(2))**(n-s) * binomial2(2*(n - (s+1)//2) - 1, 2*(s//2) - 1)
    if k % 2 == 1:
        return (-1)**((s+1)//2+n//2) * (2 * sqrt(2))**(n-s) * binomial2(2*(n - s//2) - 1, 2*((s+1)//2) - 1)

def BBttt3(n, s, k):
    if k % 2 == 0:
        return (-1)**((s+1)//2+n//2) * (2 * sqrt(2))**(n-s) * binomial2(2*(n - s//2) - 1, 2*((s+1)//2) - 1)
    if k % 2 == 1:
        return (-1)**(s//2+n//2) * (2 * sqrt(2))**(n-s) * binomial2(2*(n - (s+1)//2) - 1, 2*(s//2) - 1)

def CC(n, s):
    S = 0
    for j in range(1, n+2-s): 
        S += 2 * catalan(j-1) / sqrt(2)**j * (-1)**((j+n%2)//2) * PPP(n+1-j, s)
    return S

def CCt(n, s):
    S = 0
    for j in range(1, n+2-s): 
        S += 2 * catalan(j-1) / sqrt(2)**j * (-1)**((j+(n+1)%2)//2) * PPPt(n+1-j, s)
    return S

def expansion(n, k):
    return binomial(n+k, k) * binomial(2*(n+k), n+k) / binomial(2*k, k)

def EE(n, k):
    #Generating function: c(x) / sqrt(1 - 4*x)**(2*k+1)
    return round(binomial(n+k+1, k) * (binomial(2*(n+k+1), n+k+1) / binomial(2*k, k) - k * 4**(n+1) / (n+k+1)) / 2)

def EEt(n, k):
    #Generating function: c(x) / sqrt(1 - 4*x)**(2*k+2)
    if k == -1:
        return catalan(n)
    return round(- binomial(n+k+1, k) * (binomial(2*(n+k+1), n+k+1) / binomial(2*k, k) - 4**(n+1)) / 2)

def CC2(n, s):
    return (-1)**(1+s//2+(n+1)//2) / sqrt(2)**(n-s-1) * EE(n-s, s // 2)

def CCt2(n, s):
    return (-1)**((s+1)//2+n//2) / sqrt(2)**(n-s-1) * EEt(n-s, (s-1) // 2)


def CC3(n, s):
    S = 0
    for j in range(0, n-s+1):
        S += catalan(j) * (-1)**(1+s//2+(n+1)//2) / sqrt(2)**(n-s-1) * expansion(n-s-j, s//2)
    return S

def JJ(n, s):
    return (-1)**((n-1)//2 + s//2) * (2 * sqrt(2))**(n+1-s) *  binomial(n - (s+1)//2, s//2 - 1)

def JJt(n, s):
    if s == 0:
        return - (-1)**(n//2 + (s+1)//2) / sqrt(2)**(n-1-s) * catalan(n)
    return (-1)**(n//2 + (s+1)//2) / sqrt(2)**(n-1-s) * 1 / 2 * binomial(n-s//2, (s-1)//2) * binomial(2*(n-s//2), n-s//2) / binomial(2*((s-1)//2), (s-1)//2)

     
X = Symbol('X')
Y = Symbol('Y')

for s in range(0, 16):
    print("s =", s)
    A = 0
    B = 0
    for n in range(0, 16):
        A += X**n * BBtt3(2*n+2*s+1, 2*s, 0) * (sqrt(2))**(2*n+1) / 2
        #B += X**(n) * PPP(n, s-1)

    display(Poly(simplify(A), X))
    #display(Poly(simplify(B), X))

    #A += X**(n-s) * JJt(n, s)
    #B += X**(n-s) * PPP(n, s-1)        -----> identical for n >= s >= 0

    #A += X**(n-s) * JJ(n, s)
    #B += X**(n-s) * PPPt(n, s-1)        -----> identical for n >= s >= 0
      

In [ ]:

def PPP(l, n):
    if l - n < 0:
        return 0
    if l - n == 0:
        return -1
    else:
        return (-1)**((l-1)//2 + (n+1)//2) * (2 * sqrt(2))**(l - n) * binomial(-Rational(1,2) - n//2, l - n)

    
def custom_display(expr, title):
    print("--- Real: ", title, "---")
    display(polynomize(re(expr)))
    #display(polynomize(multi_substituter(re(expr), [(u, theta/2 + a), (v, theta/2 - a)])))
    print("--- Imag: ", title, "---")
    display(polynomize(im(expr)))
    #display(polynomize(multi_substituter(im(expr), [(u, theta/2 + a), (v, theta/2 - a)])))
    
def p_coeff(n):
    if n == -1:
        return simplify(epsilon / sqrt(2))
    if n >= 0:
        return p_factor * simplify(1 / sqrt(2) * epsiloninv**(2*n+1) * Rational((-2)**(-n) * (factorial(2*n) / (factorial(n) * factorial(n+1)))))
    return 0

def p_coeff_inv(n):
    if n == -1:
        return simplify(sqrt(2) * epsiloninv)
    if n >= 0:
        return p_factor * simplify(sqrt(2) * epsilon**(2*n+1) * Rational((-2)**n * (factorial(n) * factorial(n+1)) / (factorial(2*n))))
    return 0
    
r_factor = 1 
p_factor = 1


r_seq = []

sigma_seq = []

N = 5

F_sym = [Symbol('F_{' + str(n) + '}') for n in range(N+1)]
Ft_sym = [Symbol('\\tilde{F}_{' + str(n) + '}') for n in range(N+1)]

F_eq = [F_sym[n] for n in range(N+1)]
Ft_eq = [Ft_sym[n] for n in range(N+1)]

E_sym = [Symbol('E_{' + str(n) + '}') for n in range(N+1)]
Et_sym = [Symbol('\\tilde{E}_{' + str(n) + '}') for n in range(N+1)]

E_eq = [E_sym[n] for n in range(N+1)]
Et_eq = [Et_sym[n] for n in range(N+1)]

r_sym = [Symbol('r_{' + str(n) + '}') for n in range(N+1)]
r_eq = [r_sym[n] for n in range(N+1)]

sigma_sym = [Symbol('\sigma_{' + str(n) + '}') for n in range(N+1)]
sigma_eq = [sigma_sym[n] for n in range(N+1)] 

r_eq[0] = - sqrt(2) * I * epsiloninv
r_eq[1] = u - v - 2 * epsiloninv**2

sigma_eq[0] = - I * epsilon / (2 * sqrt(2)) * (u + v)
sigma_eq[1] = (u - v) / 2 - epsilon**2 * (u - v)**2 / 8 - I * epsilon / (2 * sqrt(2)) * deriv(u + v) - I * epsilon / sqrt(2) * deriv(v)

for n in range(1, N):
    
    X = 0
    
    X -= deriv(r_eq[n])
    
    for k in range(0, n+1):
        X += (1 + epsilon**2 / 2 * v) * r_eq[k] * r_eq[n-k]
    
    for k in range(1, n+1):
        X -= I * epsilon / (2 * sqrt(2)) * r_eq[k] * r_eq[n+1-k]
    
    r_eq[n+1] = X
        
    X = 0
    
    X += deriv(sigma_eq[n])
    
    X += epsilon**2 / 2 * deriv(v) * r_eq[n]
    
    for k in range(0, n+1):
        X += sigma_eq[k] * sigma_eq[n-k]
        
    for k in range(0, n+1):
        X += sigma_eq[k] * epsiloninv**(n+1-k) * catalan(n-k) * 2 / (sqrt(2) * I)**(n+1-k)

    sigma_eq[n+1] = X
    

        
X = 0
Y = 0
nn_is_even = True

for n in range(-1, N):
    print(" --- nn = ", n+1, " ---")

    if n == -1:
        r_seq.append(r_factor * (- sqrt(2) * I * epsiloninv))
 
        sigma_seq.append((u + v) * epsilon**2 * p_coeff_inv(-1) / (4 * I))
        #custom_display(sigma_seq[-1], "sigma_0")
    
    elif n == 0:
        r_seq.append(r_factor * (u - v - 2 * epsiloninv**2))

        sigma_seq.append(simplify(deriv(sigma_seq[0]) + sigma_seq[0]**2 - 2 * I * p_coeff(0) * sigma_seq[0] + epsilon**2 / 2 * deriv(v) * r_seq[0] 
                                   + u * (1 + epsilon**2 / 2 * v)))
        #custom_display(sigma_seq[-1], "sigma_1")
    else:
        r_square_term_1 = 0
        for k in range(0, n+1):
            r_square_term_1 += r_seq[k] * r_seq[n - k]

        r_square_term_2 = 0
        for k in range(1, n+1):
            r_square_term_2 += r_seq[k] * r_seq[n + 1 - k]

        sigma_square_term = 0
        for k in range(0, n+1):
            sigma_square_term += (sigma_seq[k] + epsiloninv**(k+1) * catalan(k) / (I**(k+1) * sqrt(2)**(k-1))) * sigma_seq[n - k]
        
        r_seq.append(r_factor * (- deriv(r_seq[n]) + (1 + epsilon**2 / 2 * v) * r_square_term_1 + epsiloninv / (sqrt(2) * I) * epsilon**2 / 2 * r_square_term_2))
        sigma_seq.append(simplify(deriv(sigma_seq[n]) + sigma_square_term 
                                  + epsilon**2 / 2 * deriv(v) * r_seq[n]))
        #custom_display(sigma_seq[-1], "sigma_" + str(n+1))


for n in range(N + 1):
    if n % 2 == 0:
        F_eq[n] = im(sigma_eq[n])
    if n % 2 == 1:
        F_eq[n] = re(sigma_eq[n])

    E_eq[n] = F_eq[n]
    for j in range(n):
        E_eq[n] += epsiloninv**(n - j) * PPP(n, j) * E_eq[j]
    
    display(Eq(sigma_sym[n], polynomize(sigma_eq[n])))
    display(Eq(sigma_sym[n], polynomize(sigma_eq[n] - sigma_seq[n])))

    display(Eq(r_sym[n], polynomize(r_eq[n])))
    display(Eq(r_sym[n], polynomize(r_eq[n] - r_seq[n])))

    display(Eq(E_sym[n], polynomize(E_eq[n])))
    